In [1]:
%reset -f
import numpy as np
import scipy
import matplotlib.pyplot as plt
import matplotlib.widgets as wid
import astropy
import regions
import skyfield as sky
#from xpbin import xpbin as _xpbin
import sys
import os
sys.path.insert(0,'/home/jacob/Downloads/heasoft-6.31.1/x86_64-pc-linuc-gnu-libc2.31/lib')

#import heasoftpy as hsp
sys.path.insert(0,os.path.abspath('/home/jacob/Desktop/HerX-1/02003801'))
sys.path.insert(0,os.path.abspath('/home/jacob/Desktop/HerX-1/02004001'))
sys.path.insert(0,os.path.abspath('/home/jacob/Desktop/HerX-1/01001899'))
import event_l1, event_l2, hk, auxil
import math
#from ixpeobssim.utils.environment import PYXSPEC_INSTALLED
#if PYXSPEC_INSTALLED:
#    import ixpeobssim.evt.xspec_.py as xspec
import pandas as pd
import ixpeobssim as ixpe 
from scipy import optimize as opt

import pickle
from scipy import pi as pi
from scipy.special import jn  as jn
import scipy.interpolate, scipy.misc
import IPython.display as ipd
from scipy import signal
from scipy.interpolate import InterpolatedUnivariateSpline
import xlsxwriter
import openpyxl

In [2]:
UseLoadedData = False
OverwriteOldData =False

orbdatafile = '/home/jacob/Desktop/HerX-1/herx1.fits'
orbdataopen = astropy.io.fits.open(orbdatafile)
orbdata1 = orbdataopen[1].data
spindata = orbdataopen[2].data
orbdataopen.close()

RA = orbdata1['RA']
DEC = orbdata1['DEC']
BINARY = orbdata1['BINARY']
PBINARY = orbdata1['PBINARY']
PBDOT = orbdata1['PBDOT']
BINARYEPOCH = orbdata1['BINARYEPOCH']
AXSINI = orbdata1['AXSINI']
ECCENTRICITY = orbdata1['ECCENTRICITY']
EGRESS = orbdata1['EGRESS']
INGRESS = orbdata1['INGRESS']
OMEGA = orbdata1['PERIAPSE']



# set orbital params

obs1date = '01/ 18-21 /2023'
obs2date = '02/ 03-08 /2023'
obs3date = '02/ 17-24 /2022'


#par1 = par2 =par3= [1.7001677,13.185,0,0,2449420.67347]
par1 = par2 =par3= [PBINARY,AXSINI,ECCENTRICITY,OMEGA,BINARYEPOCH-2400000.5]
#par1 = par2 = par3= [1.700167590,13.1831,0.00042,96,46359.871940]
#other needed inputs
#CenterX, CenterY = 305.5,288.5#from ds9 for now, implement a histogram
Radius = 27

Phi0 = 0


Period1= Period3= (1/0.807911292)#1.24#35*86400#1.24
Period2= (1/0.807911292)
Period1=Period2=Period3 = 1/spindata['FREQUENCY']
vdot = NUDOT = 0
LCbin= 250 #LCurveBinSize



#par1=par2 = [8.9643680,113.890,0.0898,153.59,2448895.71860]
c = 29979245800.

# rv=scipy.vectorize(lambda x: round(x))

rv = lambda x: numpy.around(x)

'''
pulse2time=lambda n,p: (1.0/6.0)*p[3]*p[1]**2*n**3+0.5*p[2]*p[1]*n**2+p[1]*n+p[0]


def bincor(pars,t_in,p_orb_e=0.00001,p_time_e=0.00001,t90=True, prec_factor=10,limit=1e-13,maxiter=100, return_correction=False, return_v=False, return_z=False):
#def bincor(pars,t_in,t90=False, prec_factor=10,limit=1e-13,maxiter=100, return_correction=False, return_v=False, return_z=False):

    """modifies t_in (in MJD) according to kepler parameters in pars:

             [orbital_period, - orbital period of the system, 

              a_sin(i),       - large semiaxis projection, light seconds. POSITIVE!!!

              eccentricity,   - eccentricity

              omega,          - longitude of rising node in degrees.

              t0,- time of periastron passage or t90 (t0=t90+(omega_d-90.)/360. * porb) if flag t90 is specified

              ]

       by solving Kepler equation in bessel form. All times shall be in same units.

       Optional argumets p_orb_e,p_time_e are errors for p_orb, periastron_time to limit N in KE.prec_factor

       is precision of summ comparing w. precision of observed anomaly.

       if return_correction=True the correction to input time c=t_in-t_cor is returned. Else, the corrected time itself"""

    #SCIPY ARRAYS ARE OUT OF DATA, UPDATED TO USE NUMPY
    # convert t_in to np.array


    t_in=np.array(t_in,dtype=scipy.float64)

    # just for redability of code.
    
    
    #Period
    p_orb=pars[0]
    
    #Projection onto orbital plane
    asini=pars[1]
    asini_d=asini/86400.0 
    
    #Eccentricity
    ecc=pars[2]

    #Longitude of Rising Node in Degrees -> Radians
    omega=pars[3]*pi/180.0

    if t90:


        tau=pars[4]+(pars[3]-90.)/360.*p_orb #convert t90 to t0


    else:

        tau=pars[4] #periastron passge time



    # the solution for delays is (asini/c)*F(ecc,omega,tau,theta)

    # we need to solve Kepler's equation to obtain tan(v/2) first.

    theta=(t_in-tau)/p_orb

    # E-e*sin(E)=theta - Keplers equation. 

    # Solution is given as E=theta+summ{0}{inf}{(2/n)*Jn(n,e)*sin(n*Theta)}

    # http://mathworld.wolfram.com/KeplersEquation.html

    # let's determine max(n) needed.

    # min error for theta is

    theta_e=np.mean(abs(theta*np.sqrt(((p_time_e/p_orb)**2+(p_orb_e/tau)**2))))



    max_summ_e=theta_e/prec_factor

    # print max_summ_e

    # the summ is solution converges as r=ecc*scipy.exp(scipy.sqrt(1-ecc**2))/(1+scipy.sqrt(1-ecc**2)) geometric

    # the geometric summ from n to inf is r**(n+1)/(1-r) and it shall be less than max_summ_e

    # let's find n max needed for rest summ to be less than theta_err/prec_factor.

    
    
    #If hyperbolic orbit, break
    if ecc<0 or ecc>1:

        return 1e10

    r=ecc*np.exp(np.sqrt(1-ecc**2))/(1+np.sqrt(1-ecc**2))


    #Np.ceil rounds up to nearest int

    n_max=np.ceil(np.log(max_summ_e*(1-r))/np.log(r)) #-1, but it doesn't hurt to have more n :)

    # therefore

    n=np.array(range(int(n_max)))+1

    t=t_in

    sinw=np.sin(omega)

    cosw=np.cos(omega)

    sq=np.sqrt((1.0-ecc)*(1.0+ecc))

    cor=[2.0*limit]

    numiter=0

    while abs(max(cor))>=limit and numiter<= maxiter:

        tper=(t-tau)/p_orb

        theta=2*pi*tper

        eanom=[]

        for each_theta in theta:

            eanom.append(each_theta+np.sum((2.0/n)*jn(n,n*ecc)*np.sin(n*each_theta)))

        eanom=np.array(eanom)

        sin_e=np.sin(eanom)

        cos_e=np.cos(eanom)

        z=asini_d*(sinw*(cos_e-ecc)+sq*cosw*sin_e)

        f = (t-t_in) + z

        df =(sq*cosw*cos_e - sinw*sin_e)*(2.0*pi*asini_d/(p_orb*(1.0-ecc*cos_e)))

        cor=f/(1.0+df)

        t=t-cor



        numiter=numiter+1



    K = 2*np.pi*asini*c/(p_orb*86400)/np.sqrt(1-ecc**2)



    vr = K*(cos_e+ecc*cosw)

    if return_v:

        zz = lambda x: bincor(pars,[x],return_z=True)[0]*c

        # iz = scipy.interpolate.PchipInterpolator(t,bincor(pars,t,return_z=True)*c)

        # dz = iz.derivative()(t)

        dz = np.array([scipy.misc.derivative(zz,x,dx=0.1/86400) for x in t])

        return dz/c

    elif return_z:

        return z

    elif return_correction:

        return t_in-t

    else:

        return t
''';

In [3]:
orbdatafile = '/home/jacob/Desktop/HerX-1/herx1.fits'
orbdataopen = astropy.io.fits.open(orbdatafile)
orbdata1 = orbdataopen[1].data
orbdata2 = orbdataopen[2].data
orbitdata=orbdata1
spindata = orbdata2
orbdataopen.close()

RA = orbdata1['RA']
DEC = orbdata1['DEC']
BINARY = orbdata1['BINARY']
PBINARY = orbdata1['PBINARY']
PBDOT = orbdata1['PBDOT']
BINARYEPOCH = orbdata1['BINARYEPOCH']
AXSINI = orbdata1['AXSINI']
ECCENTRICITY = orbdata1['ECCENTRICITY']
EGRESS = orbdata1['EGRESS']
INGRESS = orbdata1['INGRESS']
OMEGA = orbdata1['PERIAPSE']

par1 = par2 =par3= [PBINARY,AXSINI,ECCENTRICITY,OMEGA,BINARYEPOCH-2400000.5]

def bincor(Time,hdu):  
    SOURCENAME = 'herx1'
    #os.system('[ -f %s.fits.gz ] || curl https://gammaray.nsstc.nasa.gov/gbm/science/pulsars/lightcurves/%s.fits.gz --output %s.fits.gz' % (SOURCENAME,SOURCENAME,SOURCENAME))
    #    with fits.open('%s.fits.gz' % SOURCENAME)  as hdu:
    #        orbitdata=hdu['Orbital Elements'].data
    #        spindata=hdu['PSR_HIST'].data
    day=3600*24
        
    MJDREFI=hdu['MJDREFI']
    MJDREFF=hdu['MJDREFF']
        
        
    TPIOVER2O=orbitdata['BINARYEPOCH']-2400000.5
    PORB=orbitdata['PBINARY']*day
    PORBDOT=orbitdata['PBDOT']
    ECCENTRICITY=orbitdata['ECCENTRICITY']
    OMEGA=orbitdata['PERIAPSE']
    ASINIOVERC=orbitdata['AXSINI']
    NUORB=1/PORB
    NUORBDOT=-1/PORB/PORB*PORBDOT
    met0=Time.min()##evtlist['TIME'].min()
    MJDSTART=MJDREFF+MJDREFI+met0/day
    NU=np.interp(MJDSTART,spindata['BARYTIME'],spindata['FREQUENCY'])
    NUDOT=np.interp(MJDSTART,spindata['BARYTIME'],spindata['FDOT'])
    TREFNU=MJDSTART
    torb=((MJDREFF+MJDREFI)+Time/day-TPIOVER2O)*day##((MJDREFF+MJDREFI)+evtlist['TIME']/day-TPIOVER2O)*day
    phaseorb=(0.25+torb*(NUORB+torb*NUORBDOT/2))                    
    orbtime=ASINIOVERC*((np.sin(2*np.pi*phaseorb)-np.sin(np.pi/2))-
                    0.5*ECCENTRICITY*(np.sin(4*np.pi*phaseorb-np.radians(OMEGA))-
                                        np.sin(np.pi/2-np.radians(OMEGA))))
    deltat=Time-orbtime-met0#evtlist['TIME']-orbtime-met0
    freqtst=NU
    if NUDOT==0:
        _ii=spindata['BARYTIME']>59000
        nufunk=InterpolatedUnivariateSpline(spindata['BARYTIME'][_ii],spindata['FREQUENCY'][_ii],k=2)
        nudd=nufunk.derivatives(MJDSTART)
        NU=nudd[0]
        NUDOT=nudd[1]/day
        NUDDOT=nudd[2]/day/day
    else:
        NUDDOT=0

    #else:
    #    day=3600*24
    #    PEPOCH=60022.5116087025740000
    #    met0=Time.min()
    #    MJDSTART=MJDREFF+MJDREFI+met0/day
    #    deltat=Time-Time.min()
        NU=spindata['FREQUENCY'].mean()#0.807911292#0.14328211701342042 # guess for PSR frequency
        NU-=0.91e-3
    phdata=np.mod(deltat*(NU+0.5*deltat*(NUDOT*deltat*NUDDOT*0.3333333)),1)
    return phdata

In [4]:
def CHCount(Det,Detnumb,obs):
    CH = []
    Count = []
    I = []
    u = []
    q = []
    Ch = []
    i =[]
    u_std=[]
    q_std=[]
    i_std=[]
    #df = pd.DataFrame(Det,columns = ['T', 'P', 'U', 'Q', 'CH','X', 'Y','R'])
    df = Det
    r = 1/2#0.0083333
    #delT = df['T'].max()-df['T'].min() + (59687.7- 59689.5)*86400
    #delT =270025-2166-3088-df['T'].min()
    Ch = df['CH']
    ch = df['CH'].max()
    #OnT(observation#)(Detector#)
    if Detnumb==1:
        if obs == 1:
            delT=OnT1
        elif obs == 2:
            delT = OnT12
        elif obs == 3:
            delT = OnT13
    elif Detnumb==2:
        if obs == 1:
            delT=OnT2
        elif obs == 2:
            delT = OnT22
        elif obs == 3:
            delT = OnT23
    elif Detnumb==3:
        if obs == 1:
            delT=OnT3
        elif obs == 2:
            delT = OnT32
        elif obs == 3:
            delT = OnT33
    for i in np.arange(0, 374,1):
        df_i = df.loc[(df['CH']==i)]
        C = (len(df_i))
        CH.append(i)
        Count.append(C)
        u_ = df_i['U'].sum()
        q_ = df_i['Q'].sum()
        i_ = len(df_i)
        
        u_st= np.sqrt(abs(u_))#df_i['U'].std()
        q_st= np.sqrt(abs(q_))#df_i['Q'].std()
        
        if math.isnan(u_st) == True:
            u_st == 0
            q_st == 0
        #i_st= df_i['I'].std
        q.append(q_)
        u.append(u_)
        I.append(i_)
        q_std.append(q_st)
        u_std.append(u_st)
        #i_std.append(i_st)
    CountT = Count/delT
    ChR = np.array([CH, Count,CountT,u,q,I,u_std,q_std]).T
    ChR = pd.DataFrame(ChR)
    ChR = ChR.fillna(0)
    ChR = ChR.to_numpy()
    
    return ChR 

In [5]:
def sinusoid(x,amp,xshift,vshift):
    return amp*np.sin(2*np.pi*x+xshift)+vshift
#8.964
def c_curvefit(x,amp,hshift,amp2,period2,hshift2 ):
    return amp*np.sin((2*np.pi/(par1[0]*86400))*x+hshift)+ amp2*np.sin((2*np.pi/(period2*86400))*x+hshift2)

def gaussian(x,amp, mean,stddev):
    return  (amp)*np.exp(-(0.5)*((x-mean)/(stddev))**2)

def bimodal(x,amp1,mean1,stddev1,amp2,mean2,stddev2):
    return gaussian(x,amp1,mean1,stddev1) + gaussian(x,amp2,mean2,stddev2)

def PulsePhaseModel(t, Phi0,v,vdot,*args, **kwargs):
    vdotdot = kwargs.get('vdotdot',0)
    t0 = 0
    return Phi0 + v*(t-t0) + 0.5*vdot*((t-t0)**2)+(1/6)*vdotdot*((t-t0)**3)

In [6]:
data1path = '/home/jacob/Desktop/HerX-1/02003801/event_l2/ixpe02003801_det1_evt2_v01_barycorr.fits'
hdul1 = astropy.io.fits.open(data1path)
header1 = hdul1[0].header
data1 = hdul1[1].data
data1_2 = hdul1[2].data
hdul1.close()

data2path = '/home/jacob/Desktop/HerX-1/02003801/event_l2/ixpe02003801_det2_evt2_v01_barycorr.fits'
hdul2 = astropy.io.fits.open(data2path)
header2 = hdul2[0].header
data2 = hdul2[1].data
data2_2 = hdul2[2].data
hdul2.close()

data3path = '/home/jacob/Desktop/HerX-1/02003801/event_l2/ixpe02003801_det3_evt2_v01_barycorr.fits'
hdul3 = astropy.io.fits.open(data3path)
header3 = hdul3[0].header
data3 = hdul3[1].data
data3_2 = hdul3[2].data
hdul3.close()

data21path = '/home/jacob/Desktop/HerX-1/02004001/event_l2/ixpe02004001_det1_evt2_v01_barycorr.fits'
hdul21 = astropy.io.fits.open(data21path)
header21 = hdul21[0].header
data21 = hdul21[1].data
data21_2 = hdul21[2].data
hdul21.close()

data22path = '/home/jacob/Desktop/HerX-1/02004001/event_l2/ixpe02004001_det2_evt2_v01_barycorr.fits'
hdul22 = astropy.io.fits.open(data22path)
header22 = hdul22[0].header
data22 = hdul22[1].data
data22_2 = hdul22[2].data
hdul22.close()

data23path = '/home/jacob/Desktop/HerX-1/02004001/event_l2/ixpe02004001_det3_evt2_v01_barycorr.fits'
hdul23 = astropy.io.fits.open(data23path)
header23 = hdul23[0].header
data23 = hdul23[1].data
data23_2 = hdul23[2].data
hdul23.close()

data31path = '/home/jacob/Desktop/HerX-1/01001899/event_l2/ixpe01001899_det1_evt2_v02_barycorr.fits'
hdul31 = astropy.io.fits.open(data31path)
header31 = hdul31[0].header
data31 = hdul31[1].data
data31_2 = hdul31[2].data
hdul31.close()

data32path = '/home/jacob/Desktop/HerX-1/01001899/event_l2/ixpe01001899_det2_evt2_v02_barycorr.fits'
hdul32 = astropy.io.fits.open(data32path)
header32 = hdul32[0].header
data32 = hdul32[1].data
data32_2 = hdul32[2].data
hdul32.close()

data33path = '/home/jacob/Desktop/HerX-1/01001899/event_l2/ixpe01001899_det3_evt2_v02_barycorr.fits'
hdul33 = astropy.io.fits.open(data33path)
header33 = hdul33[0].header
data33 = hdul33[1].data
data33_2 = hdul33[2].data
hdul33.close()

RMFDet1path = '/home/jacob/Desktop/HerX-1/RFiles/ixpe_d1_obssim_v012.rmf'
hdulRMFDet1 = astropy.io.fits.open(RMFDet1path)
RMFDet1 = hdulRMFDet1[2].data
hdulRMFDet1.close()

ARFDet1path = '/home/jacob/Desktop/HerX-1/RFiles/ixpe_d1_obssim_v012.arf'
hdulARFDet1 = astropy.io.fits.open(ARFDet1path)
ARFDet1 = hdulARFDet1[1].data
hdulARFDet1.close()

RMFDet2path = '/home/jacob/Desktop/HerX-1/RFiles/ixpe_d2_obssim_v012.rmf'
hdulRMFDet2 = astropy.io.fits.open(RMFDet2path)
RMFDet2 = hdulRMFDet2[2].data
hdulRMFDet2.close()

ARFDet2path = '/home/jacob/Desktop/HerX-1/RFiles/ixpe_d2_obssim_v012.arf'
hdulARFDet2 = astropy.io.fits.open(ARFDet2path)
ARFDet2 = hdulARFDet2[1].data
hdulARFDet2.close()

RMFDet3path = '/home/jacob/Desktop/HerX-1/RFiles/ixpe_d3_obssim_v012.rmf'
hdulRMFDet3 = astropy.io.fits.open(RMFDet3path)
RMFDet3 = hdulRMFDet3[2].data
hdulRMFDet3.close()

ARFDet3path = '/home/jacob/Desktop/HerX-1/RFiles/ixpe_d3_obssim_v012.arf'
hdulARFDet3 = astropy.io.fits.open(ARFDet3path)
ARFDet3 = hdulARFDet3[1].data
hdulARFDet3.close()

#xselect extract spectrum, grppha chkey AFR and response, used as a comparison for Spec data
'''SPEC1path = '/home/jacob/Desktop/DATASET1/event_l2/DET1SpecI.pha'
hdulSPEC1 = astropy.io.fits.open(SPEC1path)
SPEC1 = hdulSPEC1[1].data
hdulSPEC1.close()

SPEC2path = '/home/jacob/Desktop/DATASET1/event_l2/DET2SpecI.pha'
hdulSPEC2 = astropy.io.fits.open(SPEC2path)
SPEC2 = hdulSPEC2[1].data
hdulSPEC2.close()

SPEC3path = '/home/jacob/Desktop/DATASET1/event_l2/DET3SpecI.pha'
hdulSPEC3 = astropy.io.fits.open(SPEC3path)
SPEC3 = hdulSPEC3[1].data
hdulSPEC3.close()'''

RE1_low= RMFDet1['E_MIN']
RE1_max = RMFDet1['E_MAX']
Channel1 = RMFDet1['CHANNEL']
RE2_low= RMFDet2['E_MIN']
RE2_max = RMFDet2['E_MAX']
Channel2 = RMFDet2['CHANNEL']
RE3_low= RMFDet3['E_MIN']
RE3_max = RMFDet3['E_MAX']
Channel3 = RMFDet3['CHANNEL']

RE = np.array([RE1_low,RE1_max,Channel1,
               RE2_low,RE2_max,Channel2,
               RE3_low,RE3_max,Channel3]).T

AE1_low= ARFDet1['ENERG_LO']
AE1_max = ARFDet1['ENERG_HI']
Resp1 = ARFDet1['SPECRESP']
AE2_low= ARFDet2['ENERG_LO']
AE2_max = ARFDet2['ENERG_HI']
Resp2 = ARFDet2['SPECRESP']
AE3_low= ARFDet3['ENERG_LO']
AE3_max = ARFDet3['ENERG_HI']
Resp3 = ARFDet3['SPECRESP']

AE = np.array([AE1_low,AE1_max,Resp1,
               AE2_low,AE2_max,Resp2,
               AE3_low,AE3_max,Resp3]).T
dfRE = pd.DataFrame(RE,columns = ['RE1_low','RE1_max','CH1'
                                       ,'RE2_low','RE2_max','CH2'
                                       ,'RE3_low','RE3_max','CH3'])

dfRE['E1_AVG'] = dfRE[['RE1_low', 'RE1_max']].mean(axis=1)
dfRE['E2_AVG'] = dfRE[['RE2_low', 'RE2_max']].mean(axis=1)
dfRE['E3_AVG'] = dfRE[['RE3_low', 'RE3_max']].mean(axis=1)

#since I am not rebinning, as of now, delE is the E range for each bin
dfRE['delE1'] = (dfRE.RE1_max - dfRE.RE1_low)
dfRE['delE2'] = (dfRE.RE2_max - dfRE.RE2_low)
dfRE['delE3'] = (dfRE.RE3_max - dfRE.RE3_low)


dfRE = dfRE.drop(columns=['RE1_low', 'RE1_max', 'RE2_low', 'RE2_max', 'RE3_low', 'RE3_max'], axis=1)

dfAE = pd.DataFrame(AE,columns = ['AE1_low','AE1_max','Resp1'
                                       ,'AE2_low','AE2_max','Resp2'
                                       ,'AE3_low','AE3_max','Resp3'])

dfAE['E1_AVG'] = dfAE[['AE1_low', 'AE1_max']].mean(axis=1)
dfAE['E2_AVG'] = dfAE[['AE2_low', 'AE2_max']].mean(axis=1)
dfAE['E3_AVG'] = dfAE[['AE3_low', 'AE3_max']].mean(axis=1)


dfAE = dfAE.drop(columns=['AE1_low', 'AE1_max', 'AE2_low', 'AE2_max', 'AE3_low', 'AE3_max'], axis=1)

dfRESPAE = pd.merge(dfRE, dfAE, on=['E1_AVG'])
dfRESPAE['E2_AVG'] = dfRESPAE[['E2_AVG_x', 'E2_AVG_y']].mean(axis=1)
dfRESPAE['E3_AVG'] = dfRESPAE[['E3_AVG_x', 'E3_AVG_y']].mean(axis=1)
dfRESPAE = dfRESPAE.drop(columns = ['E2_AVG_x', 'E2_AVG_y','E3_AVG_x', 'E3_AVG_y'])


In [7]:
def ARFRMF(CH1,Ra1,CH2,Ra2,CH3,Ra3):
    RA = np.array([CH1,Ra1,CH2,Ra2,CH3,Ra3]).T
    
    

    dfRA = pd.DataFrame(RA,columns = ['CH1','Ra1','CH2','Ra2','CH3','Ra3'])
    dfRA = dfRA.fillna(0)

    dfRERA1 = pd.merge(dfRE, dfRA, on=['CH1'])
    dfRERAAe1 = dfRERA1#pd.merge(dfRERA1, dfAE, on=['E1_AVG'])
    
    dfRERAAe1 = dfRERAAe1.drop(columns=['CH2_x', 'CH2_y','CH3_x', 'CH3_y'
                                    , 'Ra2', 'Ra3','delE2', 'delE3'])
    
    
    dfRERAAe1['RESP*Ra1'] =  dfRERAAe1[['Ra1']]
    dfRERAAe1['RESP*Ra1/delE1'] = dfRERAAe1['RESP*Ra1']/(dfRERAAe1['delE1'])

    dfRERA2 = pd.merge(dfRE, dfRA, on=['CH2'])
    dfRERAAe2 = dfRERA2#pd.merge(dfRERA2, dfAE, on=['E2_AVG'])
    dfRERAAe2 = dfRERAAe2.drop(columns=['CH1_x', 'CH1_y','CH3_x', 'CH3_y'
                                    , 'Ra1', 'Ra3', 'delE1', 'delE3'])

    dfRERAAe2['RESP*Ra2'] = dfRERAAe2[['Ra2']]
    dfRERAAe2['RESP*Ra2/delE2'] = dfRERAAe2['RESP*Ra2']/(dfRERAAe2['delE2'])

    dfRERA3 = pd.merge(dfRE, dfRA, on=['CH3'])
    dfRERAAe3 = dfRERA3#pd.merge(dfRERA3, dfAE, on=['E3_AVG'])
    dfRERAAe3 = dfRERAAe3.drop(columns=['CH1_x', 'CH1_y','CH2_x', 'CH2_y'
                                    , 'Ra1', 'Ra2', 'delE1', 'delE2'])

    dfRERAAe3['RESP*Ra3'] = dfRERAAe3[['Ra3']]
    dfRERAAe3['RESP*Ra3/delE3'] = dfRERAAe3['RESP*Ra3']/(dfRERAAe3['delE3'])


    dfSPEC = pd.concat([dfRERAAe1, dfRERAAe2, dfRERAAe3], axis=1)
    dfSPEC['RESP*RATE'] = dfSPEC[['RESP*Ra1', 'RESP*Ra2', 'RESP*Ra3']].mean(axis=1)
    dfSPEC['AVG_E'] = dfSPEC[['E1_AVG', 'E2_AVG', 'E3_AVG']].mean(axis=1)
    
    
    dfSPEC['RESP*Ra/delE'] = dfSPEC['RESP*RATE']/(dfSPEC['delE1'])
    
    
    
    return dfSPEC

In [8]:
def LightCurve(Det,Detnumb,obs,E_low,E_high,OnT):
    Count = []
    T = []
    CountRate = []

    if Detnumb == 1:
        Det = pd.merge(Det,dfRESPAE, left_on='CH', right_on='CH1')
        Det.drop(columns =['CH2', 'CH3','E2_AVG', 'E3_AVG', 'Resp2', 'Resp3'])
        Det['E'] = Det['E1_AVG']

    elif Detnumb == 2:
        Det = pd.merge(Det,dfRESPAE, left_on='CH', right_on='CH2')
        Det.drop(columns = ['CH1', 'CH3','E1_AVG', 'E3_AVG', 'Resp1', 'Resp3'])
        Det['E'] = Det['E2_AVG']
        
    elif Detnumb == 3:
        Det = pd.merge(Det,dfRESPAE, left_on='CH', right_on='CH3')
        Det.drop(columns = ['CH1', 'CH2','E1_AVG', 'E2_AVG', 'Resp1', 'Resp2'])
        Det['E'] = Det['E3_AVG']

    
    
    Det = Det.loc[(Det['E']>E_low)&(Det['E']<E_high)]
    binsize =LCbin
    
    #binsize =500
    for i in np.arange(0,Det['T'].max()-(binsize/2),binsize):
        Ont = OnT.loc[(OnT['OnT_Start']<(i-(binsize/2)))&((OnT['OnT_End'] > (i+(binsize/2))))]
        if len(Ont) == 0:
            Ont = OnT.loc[(OnT['OnT_Start'].between((i-(binsize/2)),(i+(binsize/2))))]
            delt = (i+(binsize/2))-Ont['OnT_Start'].mean()
            if len(Ont) ==0:
                Ont = OnT.loc[(OnT['OnT_End'].between((i-(binsize/2)),(i+(binsize/2))))]
                delt = Ont['OnT_End'].mean()-(i-(binsize/2))
            
        elif len(Ont) != 0:
            delt = binsize
            
        if delt < 0.2*binsize:
            continue
        
        Det_i = Det.loc[(Det['T']<i+(binsize/2)) &(Det['T']>i-(binsize/2))]
        T.append(i+(binsize/2))
        C = len(Det_i)
        #delt=250
        CountRate.append(C/delt)
    LC = np.array([T,CountRate]).T
    LCurve = pd.DataFrame(LC, columns = ['T', 'CountRate'])
    LCurve['T'] = (LCurve['T']/86400)
    
    return LCurve

In [9]:
def normalize_Stokes(Det,Detnumb, E_low,E_high, *args, **kwargs):
    PStart = float(kwargs.get('PS',0))
    PEnd = float(kwargs.get('PE',1))
    periodnum = kwargs.get('PNum',0)
    df1 = Det
    
    if Detnumb == 1:
        df1 = pd.merge(df1,dfRESPAE, left_on='CH', right_on='CH1')
        df1['E'] = df1['E1_AVG']
        df1.drop(columns =['CH2', 'CH3','E1_AVG','E2_AVG'
                           , 'E3_AVG','Resp1', 'Resp2', 'Resp3'])
        
        
    elif Detnumb == 2:
        df1 = pd.merge(df1,dfRESPAE, left_on='CH', right_on='CH2')
        df1['E'] = df1['E2_AVG']
        df1.drop(columns = ['CH1', 'CH3','E1_AVG','E2_AVG', 
                            'E3_AVG', 'Resp1','Resp2', 'Resp3'])

        
        
        
    elif Detnumb == 3:
        df1 = pd.merge(df1,dfRESPAE, left_on='CH', right_on='CH3')
        df1['E'] = df1['E3_AVG']
        df1.drop(columns = ['CH1', 'CH2','E1_AVG', 'E2_AVG', 
                            'E3_AVG','Resp1','Resp3', 'Resp2'])

        
    
    df1 = df1.loc[(df1['E']>E_low)&(df1['E']<E_high)]
    
    lend = len(df1)
    df1 = df1.loc[(df1['P']>PStart)&(df1['P']<PEnd)]
    lend1 = len(df1)
    Q = df1['Q'].sum()
    U = df1['U'].sum()
    I = float(len(df1))
    
    Qmean = df1['Q'].mean()
    Qstd = df1['Q'].std()
    
    Umean = df1['U'].mean()
    Ustd = df1['U'].std()
    
    df1['q'] = (df1['Q']/I)*100
    df1['u'] = (df1['U']/I)*100
    q =np.float64(df1['q'].sum())
    u =np.float64(df1['u'].sum()) 
    qstd = np.float64(np.sqrt(I*(df1['q'].std()**2)))#/(lend/lend1))
    
    ustd = np.float64(np.sqrt(I*(df1['u'].std()**2)))#/(lend/lend1))
    
    
    PD = float(np.sqrt([(q**2)+(u**2)]))
    PDstd = np.abs(float(np.sqrt([(qstd**2)+(ustd**2)])))
    PA = float((0.5)*np.arctan(u/ q)*180/np.pi)
    PAstd = np.abs(float((0.5)*((qstd+ustd)/(1+((u/q)**2)))*180/np.pi))
    
    Stokes = np.array([I,q,u,PD,PA,qstd,ustd,PDstd,PAstd,E_low,E_high]).T
    return Stokes

In [10]:
def ErrorCont(Stokes, sigma,prec):
    Error = np.sqrt((Stokes[6])**2+(Stokes[5])**2)
    UE = []
    QE = []

    for i in np.arange(0,2+(2/prec),2/prec):
        UError = np.float64(Stokes[2]+sigma*np.cos(i*np.pi)*Error)
        QError = np.float64(Stokes[1]+sigma*np.sin(i*np.pi)*Error)
        UE.append(UError)
        QE.append(QError)
    
    Cont = np.array([UE, QE]).T

    Cont =pd.DataFrame(Cont, columns=['U','Q'])
    
    
    
    Cont['PD'] = np.sqrt((Cont['U']**2)+(Cont['Q']**2))
    Cont['PA'] = ((0.5)*np.arctan2(Cont['U'],Cont['Q']))

    Cont = pd.concat([Cont,Cont.loc[Cont.index[0]]])
    
    
    return Cont

In [11]:
def  PULSEPROFILE(Det,Detnumb,obs, E_low,E_high,periodnum, *args, **kwargs):
#Det,Detnumb, E_low,E_high = Det1S,1,2,8
#for f in range(0,1):
    TStart = kwargs.get('Tstart',-1)
    P = []
    Count = []
    CountT = []
    df1 = Det
    r = 1/120 #1/120 120bins in the data
    
    if TStart >-1:
        df1 = df1.loc[(df1['T']>float(TStart))&(df1['T']<float(TStart+periodnum*Period1))]
        
    if Detnumb == 1:
        df1 = pd.merge(df1,dfRESPAE, left_on='CH', right_on='CH1')
        df1.drop(columns =['CH2', 'CH3','E2_AVG', 'E3_AVG', 'Resp2', 'Resp3'])
        df1['E'] = df1['E1_AVG']
        df1['RESP'] = df1['Resp1']
        if TStart >-1 and periodnum >0:
            if obs ==1:
                ONT1 = Ont1.loc[(Ont1['OnT_Start']>TStart)&(Ont1['OnT_Start']<float(TStart+periodnum*Period1))]
                delT=ONT1['OnT'].sum()
            if obs == 2:
                ONT1 = Ont12.loc[(Ont12['OnT_Start']>TStart)&(Ont12['OnT_Start']<float(TStart+periodnum*Period2))]
                delT=ONT1['OnT'].sum()
            if obs ==3:
                ONT3 = Ont13.loc[(Ont13['OnT_Start']>TStart)&(Ont13['OnT_Start']<float(TStart+periodnum*Period3))]
                delT=ONT3['OnT'].sum()
                
        else:
            if obs == 1:
                delT=OnT1#*2*r
            if obs == 2:
                delT = OnT12#*2*r
            if obs ==3:
                delT = OnT13
    elif Detnumb == 2:
        df1 = pd.merge(df1,dfRESPAE, left_on='CH', right_on='CH2')
        df1.drop(columns = ['CH1', 'CH3','E1_AVG', 'E3_AVG', 'Resp1', 'Resp3'])
        df1['E'] = df1['E2_AVG']
        df1['RESP'] = df1['Resp2']
        if TStart >-1:
            if obs ==1:
                ONT2 = Ont2.loc[(Ont2['OnT_Start']>TStart)&(Ont2['OnT_Start']<float(TStart+periodnum*Period1))]
                delT=ONT2['OnT'].sum()
            if obs ==2:
                ONT2 = Ont22.loc[(Ont22['OnT_Start']>TStart)&(Ont22['OnT_Start']<float(TStart+periodnum*Period2))]
                delT=ONT2['OnT'].sum()
            if obs ==3:
                ONT3 = Ont23.loc[(Ont23['OnT_Start']>TStart)&(Ont23['OnT_Start']<float(TStart+periodnum*Period3))]
                delT=ONT3['OnT'].sum()
        else:
            if obs == 1:
                delT=OnT2#*2*r
            if obs == 2:
                delT = OnT22#*2*r
            if obs ==3:
                delT = OnT23
    elif Detnumb == 3:
        df1 = pd.merge(df1,dfRESPAE, left_on='CH', right_on='CH3')
        df1.drop(columns = ['CH1', 'CH2','E1_AVG', 'E2_AVG', 'Resp1', 'Resp2'])
        df1['E'] = df1['E3_AVG']
        df1['RESP'] = df1['Resp3']
        if TStart >-1:
            if obs ==1:
                ONT3 = Ont3.loc[(Ont3['OnT_Start']>TStart)&(Ont3['OnT_Start']<float(TStart+periodnum*Period1))]
                delT=ONT3['OnT'].sum()
            if obs ==2:
                ONT3 = Ont32.loc[(Ont32['OnT_Start']>TStart)&(Ont32['OnT_Start']<float(TStart+periodnum*Period2))]
                delT=ONT3['OnT'].sum()
            if obs ==3:
                ONT3 = Ont33.loc[(Ont33['OnT_Start']>TStart)&(Ont33['OnT_Start']<float(TStart+periodnum*Period3))]
                delT=ONT3['OnT'].sum()
        else:
            if obs == 1:
                delT=OnT3#*2*r
            if obs == 2:
                delT = OnT32#*2*r
            if obs ==3:
                delT = OnT33
                
    df1 = df1.loc[(df1['E']>E_low)&(df1['E']<E_high)]
    
    
    for i in np.arange(r, 1,r*2):
        df1_i = df1.loc[(df1['P']>=i-r)&(df1['P']<i+r)]
        C = (len(df1_i))
        P.append(i)
        Count.append(C)

    PC = pd.DataFrame(P, columns =['P'])
    Count = pd.DataFrame(Count, columns = ['Count'])
    PC['Count'] = Count
    PC['SigmaC'] = np.sqrt(Count)
    
    PC['CountT'] = PC['Count']/delT

    PC['SigmaCT'] = PC['SigmaC']/delT
    return PC




In [12]:
def PulseprofileBCFit(Det1S,Det1B,Det2S,Det2B,Det3S,Det3B,obs,EL,EH,*args, **kwargs):
    periodnum = kwargs.get('PNum',0)
    TStart = kwargs.get('Tstart',-1)
    EH = 4
    EL=2
    PC1S = PULSEPROFILE(Det1S, 1,obs, EL,EH,periodnum,Tstart = TStart)#DET MATRIX,DETNUMBER,LOW AND HIGH ELevels
    PC1B = PULSEPROFILE(Det1B, 1,obs, EL,EH,periodnum,Tstart = TStart)
    PC2S = PULSEPROFILE(Det2S, 2,obs, EL,EH,periodnum,Tstart = TStart)
    PC2B = PULSEPROFILE(Det2B, 2,obs, EL,EH,periodnum,Tstart = TStart)
    PC3S = PULSEPROFILE(Det3S, 3,obs, EL,EH,periodnum,Tstart = TStart)
    PC3B = PULSEPROFILE(Det3B, 3,obs, EL,EH,periodnum,Tstart = TStart)

    PC1 = (PC1S/(3.14159*((26.923)**2)) - PC1B/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923)**2))
    PC1['SigmaC'] = np.sqrt(((PC1S['SigmaC']/(3.14159*((26.923)**2)))*(3.14159*((26.923)**2))**2) + (((PC1B['SigmaC']/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923**2)))**2)))
    PC1['SigmaCT'] = np.sqrt(((PC1S['SigmaCT']/(3.14159*((26.923)**2)))*(3.14159*((26.923)**2))**2) + (((PC1B['SigmaCT']/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923**2)))**2)))
   
    PC1['P'] = PC1S['P']
    
    PC2 = (PC2S/(3.14159*((26.923)**2)) - PC2B/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923)**2))
    PC2['SigmaC'] = np.sqrt(((PC2S['SigmaC']/(3.14159*((26.923)**2)))*(3.14159*((26.923)**2))**2) + (((PC2B['SigmaC']/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923**2)))**2)))
    PC2['SigmaCT'] = np.sqrt(((PC2S['SigmaCT']/(3.14159*((26.923)**2)))*(3.14159*((26.923)**2))**2) + (((PC2B['SigmaCT']/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923**2)))**2)))
    PC2['P'] = PC2S['P']
    
    PC3 = (PC3S/(3.14159*((26.923)**2)) - PC3B/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923)**2))
    PC3['SigmaC'] = np.sqrt(((PC3S['SigmaC']/(3.14159*((26.923)**2)))*(3.14159*((26.923)**2))**2) + (((PC3B['SigmaC']/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923**2)))**2)))
    PC3['SigmaCT'] = np.sqrt(((PC3S['SigmaCT']/(3.14159*((26.923)**2)))*(3.14159*((26.923)**2))**2) + (((PC3B['SigmaCT']/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923**2)))**2)))
   
    PC3['P'] = PC3S['P']

    PC = pd.merge(PC1,PC2, on=['P'])
    PC = pd.merge(PC,PC3, on= ['P'])
    PC['COUNT'] = PC[['Count_x','Count_y', 'Count']].mean(axis=1)
    PC['COUNTT'] = PC[['CountT_x','CountT_y', 'CountT']].sum(axis=1)
    PC = PC.drop(columns = ['CountT_x','Count_x','CountT_y','Count_y','Count','CountT'])
    PC = PC.loc[(PC['COUNTT'] > 0)]

    PC['COUNTT'] = PC['COUNTT']/PC['COUNTT'].max()

    PCP = PC.copy()
    PCP['P'] = PCP['P']+1
    PC = pd.concat([PC,PCP])
    
    PCP = PC.loc[(PC['P'].between(0,1))]
    amp = PCP['COUNTT'].max()-PCP['COUNT'].min()
    vshift = PCP['COUNTT'].mean()
    
    params, params_covariance = opt.curve_fit(sinusoid,PCP['P'].T,PCP['COUNTT'].T,maxfev=100000,p0 =[amp,0,vshift], absolute_sigma = True)
    
    PCP = PC.loc[(PC['P']).between(0.25,1.25)&(PC['COUNTT'] == PC['COUNTT'].max())]
    mean = float(PCP['P'].mean())
    
    Smean = PC.loc[(PC['P'].between(mean-0.25,mean+0.25)&(~PC['P'].between(mean-0.05,mean+0.05)))]
    Smean = Smean.loc[(Smean['COUNTT'] == Smean['COUNTT'].max())]


    if PCP['P'].mean()< Smean['P'].mean():
        mean = float(Smean['P'].mean())
    
    PCP = PC.loc[PC['P'].between(mean-0.1,mean+0.1)]

    
    gparams,gparam_covariance = opt.curve_fit(gaussian,PCP['P'].T,PCP['COUNTT'].T,maxfev=100000,p0 = [1,mean,0.1], absolute_sigma = True)

    #PCP = PC.loc[PC['P'].between(mean-0.25,mean+0.25)]
    Smean = PC.loc[(PC['P'].between(mean-0.25,mean+0.25)&(~PC['P'].between(mean-0.075,mean+0.075)))]
    Smean = Smean.loc[(Smean['COUNTT'] == Smean['COUNTT'].max())]
    Smean = float(Smean['P'].mean())

    PCP = PC.loc[PC['P'].between(((Smean+mean)/2)-0.2,((Smean+mean)/2)+0.2)]

    bparams,bparam_covariance = opt.curve_fit(bimodal,PCP['P'].T,PCP['COUNTT'].T,maxfev=100000,p0 = [0.9,Smean,0.05,1,mean,0.05], absolute_sigma = True)

    PCS = pd.merge(PC1S,PC2S, on=['P'])
    PCS = pd.merge(PCS,PC3S, on= ['P'])
    PCS['COUNT'] = PCS[['Count_x','Count_y', 'Count']].mean(axis=1)
    PCS['COUNTT'] = PCS[['CountT_x','CountT_y', 'CountT']].sum(axis=1)
    
    
    return PC, params,params_covariance,gparams,gparam_covariance,bparams,bparam_covariance

In [13]:
def PulseprofileBCFit2(Det1S,Det1B,Det2S,Det2B,Det3S,Det3B,obs,EL,EH,*args, **kwargs):
    periodnum = kwargs.get('PNum',0)
    TStart = kwargs.get('Tstart',-1)
    EH = 4
    EL=2
    PC1S = PULSEPROFILE(Det1S, 1,obs, EL,EH,periodnum,Tstart = TStart)#DET MATRIX,DETNUMBER,LOW AND HIGH ELevels
    PC1B = PULSEPROFILE(Det1B, 1,obs, EL,EH,periodnum,Tstart = TStart)
    PC2S = PULSEPROFILE(Det2S, 2,obs, EL,EH,periodnum,Tstart = TStart)
    PC2B = PULSEPROFILE(Det2B, 2,obs, EL,EH,periodnum,Tstart = TStart)
    PC3S = PULSEPROFILE(Det3S, 3,obs, EL,EH,periodnum,Tstart = TStart)
    PC3B = PULSEPROFILE(Det3B, 3,obs, EL,EH,periodnum,Tstart = TStart)


    PC1 = (PC1S/(3.14159*((26.923)**2)) - PC1B/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923)**2))
    PC1['SigmaC'] = np.sqrt(((PC1S['SigmaC']/(3.14159*((26.923)**2)))*(3.14159*((26.923)**2))**2) + (((PC1B['SigmaC']/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923**2)))**2)))
    PC1['SigmaCT'] = np.sqrt(((PC1S['SigmaCT']/(3.14159*((26.923)**2)))*(3.14159*((26.923)**2))**2) + (((PC1B['SigmaCT']/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923**2)))**2)))
   
    PC1['P'] = PC1S['P']
    
    PC2 = (PC2S/(3.14159*((26.923)**2)) - PC2B/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923)**2))
    PC2['SigmaC'] = np.sqrt(((PC2S['SigmaC']/(3.14159*((26.923)**2)))*(3.14159*((26.923)**2))**2) + (((PC2B['SigmaC']/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923**2)))**2)))
    PC2['SigmaCT'] = np.sqrt(((PC2S['SigmaCT']/(3.14159*((26.923)**2)))*(3.14159*((26.923)**2))**2) + (((PC2B['SigmaCT']/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923**2)))**2)))
    PC2['P'] = PC2S['P']
    
    PC3 = (PC3S/(3.14159*((26.923)**2)) - PC3B/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923)**2))
    PC3['SigmaC'] = np.sqrt(((PC3S['SigmaC']/(3.14159*((26.923)**2)))*(3.14159*((26.923)**2))**2) + (((PC3B['SigmaC']/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923**2)))**2)))
    PC3['SigmaCT'] = np.sqrt(((PC3S['SigmaCT']/(3.14159*((26.923)**2)))*(3.14159*((26.923)**2))**2) + (((PC3B['SigmaCT']/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923**2)))**2)))
   
    PC3['P'] = PC3S['P']

    PC = pd.merge(PC1,PC2, on=['P'])
    PC = pd.merge(PC,PC3, on= ['P'])
    PC['COUNT'] = PC[['Count_x','Count_y', 'Count']].mean(axis=1)
    PC['COUNTT'] = PC[['CountT_x','CountT_y', 'CountT']].sum(axis=1)
    PC = PC.drop(columns = ['CountT_x','Count_x','CountT_y','Count_y','Count','CountT'])
    PC = PC.loc[(PC['COUNTT'] > 0)]
    
    PC['COUNTT'] = PC['COUNTT']/PC['COUNTT'].max()

    PCP = PC.copy()
    PCP['P'] = PCP['P']+1
    PC = pd.concat([PC,PCP])
    
    PCP = PC.loc[(PC['P'].between(0,1))]
    amp = PCP['COUNTT'].max()-PCP['COUNT'].min()
    vshift = PCP['COUNTT'].mean()
    
    params, params_covariance = opt.curve_fit(sinusoid,PCP['P'].T,PCP['COUNTT'].T,maxfev=100000,p0 =[amp,0,vshift], absolute_sigma = True)
    
    PCP = PC.loc[(PC['P']).between(0.25,1.25)&(PC['COUNTT'] == PC['COUNTT'].max())]
    mean = float(PCP['P'].mean())
    
    Smean = PC.loc[(PC['P'].between(mean-0.25,mean+0.25)&(~PC['P'].between(mean-0.05,mean+0.05)))]
    Smean = Smean.loc[(Smean['COUNTT'] == Smean['COUNTT'].max())]


    if PCP['P'].mean()< Smean['P'].mean():
        mean = float(Smean['P'].mean())
    
    PCP = PC.loc[PC['P'].between(mean-0.1,mean+0.1)]

    
    gparams,gparam_covariance = opt.curve_fit(gaussian,PCP['P'].T,PCP['COUNTT'].T,maxfev=100000,p0 = [1,mean,0.1], absolute_sigma = True)

    #PCP = PC.loc[PC['P'].between(mean-0.25,mean+0.25)]
    Smean = PC.loc[(PC['P'].between(mean-0.25,mean+0.25)&(~PC['P'].between(mean-0.075,mean+0.075)))]
    Smean = Smean.loc[(Smean['COUNTT'] == Smean['COUNTT'].max())]
    Smean = float(Smean['P'].mean())

    PCP = PC.loc[PC['P'].between(((Smean+mean)/2)-0.2,((Smean+mean)/2)+0.2)]

    bparams,bparam_covariance = opt.curve_fit(bimodal,PCP['P'].T,PCP['COUNTT'].T,maxfev=100000,p0 = [0.9,Smean,gparams[2],1,mean,gparams[2]], absolute_sigma = True)

    PCS = pd.merge(PC1S,PC2S, on=['P'])
    PCS = pd.merge(PCS,PC3S, on= ['P'])
    PCS['COUNT'] = PCS[['Count_x','Count_y', 'Count']].mean(axis=1)
    PCS['COUNTT'] = PCS[['CountT_x','CountT_y', 'CountT']].sum(axis=1)
    
    
    return PC, params,params_covariance,gparams,gparam_covariance,bparams,bparam_covariance

In [14]:
def DataSetup(data, data_2 ,par, EclipseStart , EclipseEnd, Period,vdot,*args, **kwargs):
    
    TZero_ = kwargs.get('TZero',0)
    CenterX = kwargs.get('CenterX',-1)
    CenterY = kwargs.get('CenterY',-1)
    obsid = kwargs.get('obsid', -1)
    Detnumb = kwargs.get('Detnumb', -1)
    hdu = kwargs.get('hdu',0)
    
    
    
    if obsid == -1:
        obsid = hdu['OBS_ID']
        Detnumb = hdu['DETNAM']
    if UseLoadedData == True and os.path.isfile(f'/home/jacob/Desktop/HerX-1/{obsid}/event_l2/DataFrameforDet{Detnumb}.xlsx') == True:
        DET1 = pd.read_excel(f'/home/jacob/Desktop/HerX-1/{obsid}/event_l2/DataFrameforDet{Detnumb}.xlsx')
    
    
    else:
        #TIME1 = data['TIME']
        #TIME1 = bincor(TIME1,hdu)

        TIME1 = (data['TIME']/86400)+ MJDRef
        #TIME1 = bincor(par,TIME1)

        TIME1 = TIME1*86400



        TZero12 = np.min(TIME1)

        #Time relative to start obs2
        TIME1 = TIME1 - TZero12
        TIME1 = TIME1 - (TZero_*86400)



        #PHASE1 = (PulsePhaseModel(TIME1,Phi0,1/Period,vdot)%(1))
        PHASE1 = bincor(TIME1,hdu)
        u1 = data['U']
        q1 = data['Q']
        Ch1 = data['PI']


        #center the coordinates on the pulsar as found by ds9
        y = data['Y']
        x = data['X']
        if CenterX <0 and CenterY<0:
            heatmap,xedges,yedges = np.histogram2d(x,y,bins=(int(600),int(600)))
            extent = [0,int(600),0,int(600)]
            center_location = np.unravel_index(heatmap.argmax(),heatmap.shape)
            print("Center Location: "+str(center_location))
            print("Center Location X: " +str(center_location[0]))
            print("Center Location Y: " +str(center_location[1]))

            fig,ax = plt.subplots()
            im =ax.imshow(heatmap.T,extent=extent,origin='lower')
            circle1 = plt.Circle(center_location,Radius,fill=False,color='green')
            circle2 = plt.Circle(center_location,2*Radius,fill=False,color='red')
            circle4 = plt.Circle(center_location,4*Radius,fill=False,color='red')
            ax.add_patch(circle1)
            ax.add_patch(circle2)
            ax.add_patch(circle4)
            fig.colorbar(im,ax=ax,label='Colorbar')
            plt.show()
        else:
            center_location = [CenterX,CenterY]


        x1 = data['X']-center_location[0]
        y1 = data['Y']-center_location[1]


        r1 = ((x1**2)+(y1**2))**(1/2)
        Det1 = np.array([TIME1, PHASE1, u1, q1, Ch1, x1,y1,r1]).T
        DET1 = pd.DataFrame(Det1, columns =['T', 'P', 'U','Q','CH','X','Y','R'])
        TIME1 = TIME1 - (TZero_*86400)
        DET1 = DET1.loc[DET1['T']>0]
        print('LenData:'+str(len(DET1)))

        for i in np.arange(0,len(EclipseStart),1):

            DET1i = DET1.iloc[(DET1['T']> float(EclipseStart[i]- TStart)*86400).values & (DET1['T'] < float(EclipseEnd[i]-TStart)*86400).values]
            DET1 = DET1.drop(index = DET1i.index)
        DET1['T'] = DET1['T'] - (TZero_*86400)
        DET1 = DET1.loc[DET1['T']>0]
        
        if OverwriteOldData == True:    
            ifsheetexists = "replace"
            with pd.ExcelWriter(f'/home/jacob/Desktop/HerX-1/{obsid}/event_l2/DataFrameforDet{Detnumb}.xlsx') as writer:
                DET1.to_excel(writer)

        

        


    DetS = DET1.loc[(DET1['R']<Radius)]
    DetB = DET1.loc[(DET1['R']>2*Radius)& DET1['R']<4*Radius]
    
    OnT_Start = data_2['START']
    OnT_End = data_2['STOP']
    OnT_End = OnT_End - np.min(OnT_Start)
    OnT_Start = OnT_Start - np.min(OnT_Start)
    OnT = np.array([OnT_Start,OnT_End]).T
    Ont12 = pd.DataFrame(OnT, columns = ['OnT_Start', 'OnT_End'])
    Ont12['OnT'] = Ont12['OnT_End'] - Ont12['OnT_Start']


    for i in np.arange(0,len(EclipseStart),1):
        Ont12i = Ont12.iloc[(Ont12['OnT_End']>(float(EclipseStart[i]-TStart)*86400)).values & (Ont12['OnT_Start']<float(EclipseEnd[i]-TStart)*86400).values]
        Ont12 = Ont12.drop(index=Ont12i.index)

    OnT12 = Ont12['OnT'].sum()
    print('Counts: ' + str(len(DET1)))
    print('On Time: '+str(OnT12))


    print("Done with Det")

    return DetS,DetB,Ont12,OnT12

In [15]:
EclipseStart1 = [0]
EclipseEnd1 = [0]

MJDRef = 57754
TStart = 0

print('Observation 1')

Det1S,Det1B,Ont1,OnT1 = DataSetup(data1, data1_2 ,par1, EclipseStart1 ,EclipseEnd1,Period= Period1, vdot=0,CenterX=304,CenterY=287,hdu = header1)
for i in range(0,1):
    print(' ')
Det2S,Det2B,Ont2,OnT2 = DataSetup(data2, data2_2 ,par1, EclipseStart1 ,EclipseEnd1,Period= Period1, vdot=0,CenterX=304,CenterY=287,hdu = header2)
for i in range(0,1):
    print(' ')
Det3S,Det3B,Ont3,OnT3 = DataSetup(data3, data3_2 ,par1, EclipseStart1 ,EclipseEnd1,Period= Period1, vdot=0,CenterX=304,CenterY=287,hdu = header3)

for i in range(0,5):
    print(' ')

EclipseStart2 = [0.6,0.6+par2[0],0.6+2*par2[0]]
EclipseEnd2 = [1.2,1.2+par2[0],1.2+2*par2[0]]
#EclipseStart2 = [0]
#EclipseEnd2 = [0]

print('Observation 2')

Det12S,Det12B,Ont12,OnT12 = DataSetup(data21, data21_2 ,par2, EclipseStart2 ,EclipseEnd2,Period= Period2, vdot=0,CenterX=302,CenterY=286,hdu = header21)
for i in range(0,1):
    print(' ')
Det22S,Det22B,Ont22,OnT22 = DataSetup(data22, data22_2 ,par2, EclipseStart2 ,EclipseEnd2,Period= Period2, vdot=0,CenterX=302,CenterY=286,hdu = header22)
for i in range(0,1):
    print(' ')
Det32S,Det32B,Ont32,OnT32 = DataSetup(data23, data23_2 ,par2, EclipseStart2 ,EclipseEnd2,Period= Period2, vdot=0,CenterX=302,CenterY=286,hdu = header23)


for i in range(0,5):
    print(' ')
print('Observation 3')

#FirstEclipse = [1.8,2.25]
#EclipseStart3 = [FirstEclipse[0],FirstEclipse[0]+par2[0],4.1]
#EclipseEnd3 = [FirstEclipse[1],FirstEclipse[1]+par2[0],5.6]


FirstEclipse = [1.8,2.25]
EclipseStart3 = [FirstEclipse[0],FirstEclipse[0]+par2[0],4.1,FirstEclipse[0]+2.9*par2[0]]
EclipseEnd3 = [FirstEclipse[1],FirstEclipse[1]+par2[0],5.6,FirstEclipse[1]+3*par2[0]]
#EclipseStart3 = [0]
#EclipseEnd3 = [0]

Det13S,Det13B,Ont13,OnT13 = DataSetup(data31, data31_2 ,par3, EclipseStart3 ,EclipseEnd3,Period= Period3, vdot=0,CenterX = 304,CenterY=284,hdu = header31)
for i in range(0,1):
    print(' ')
Det23S,Det23B,Ont23,OnT23 = DataSetup(data32, data32_2 ,par3, EclipseStart3 ,EclipseEnd3,Period= Period3, vdot=0,CenterX = 304,CenterY=284,hdu = header32)
for i in range(0,1):
    print(' ')
Det33S,Det33B,Ont33,OnT33 = DataSetup(data33, data33_2 ,par3, EclipseStart3 ,EclipseEnd3,Period= Period3, vdot=0,CenterX = 304,CenterY=284,hdu = header33 )



In [ ]:
def PulseprofileBC(Det1S,Det1B,Det2S,Det2B,Det3S,Det3B,obs,EL,EH,*args, **kwargs):
    periodnum = kwargs.get('PNum',0)
    TStart = kwargs.get('Tstart',-1)
    PC1S = PULSEPROFILE(Det1S, 1,obs, EL,EH,periodnum,Tstart = TStart)#DET MATRIX,DETNUMBER,LOW AND HIGH ELevels
    PC1B = PULSEPROFILE(Det1B, 1,obs, EL,EH,periodnum,Tstart = TStart)
    PC2S = PULSEPROFILE(Det2S, 2,obs, EL,EH,periodnum,Tstart = TStart)
    PC2B = PULSEPROFILE(Det2B, 2,obs, EL,EH,periodnum,Tstart = TStart)
    PC3S = PULSEPROFILE(Det3S, 3,obs, EL,EH,periodnum,Tstart = TStart)
    PC3B = PULSEPROFILE(Det3B, 3,obs, EL,EH,periodnum,Tstart = TStart)

    PC1 = (PC1S/(3.14159*((26.923)**2)) - PC1B/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923)**2))
    PC1['SigmaC'] = np.sqrt(((PC1S['SigmaC']/(3.14159*((26.923)**2)))*(3.14159*((26.923)**2))**2) + (((PC1B['SigmaC']/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923**2)))**2)))
    PC1['SigmaCT'] = np.sqrt(((PC1S['SigmaCT']/(3.14159*((26.923)**2)))*(3.14159*((26.923)**2))**2) + (((PC1B['SigmaCT']/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923**2)))**2)))
   
    PC1['P'] = PC1S['P']
    
    PC2 = (PC2S/(3.14159*((26.923)**2)) - PC2B/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923)**2))
    PC2['SigmaC'] = np.sqrt(((PC2S['SigmaC']/(3.14159*((26.923)**2)))*(3.14159*((26.923)**2))**2) + (((PC2B['SigmaC']/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923**2)))**2)))
    PC2['SigmaCT'] = np.sqrt(((PC2S['SigmaCT']/(3.14159*((26.923)**2)))*(3.14159*((26.923)**2))**2) + (((PC2B['SigmaCT']/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923**2)))**2)))
    PC2['P'] = PC2S['P']
    
    PC3 = (PC3S/(3.14159*((26.923)**2)) - PC3B/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923)**2))
    PC3['SigmaC'] = np.sqrt(((PC3S['SigmaC']/(3.14159*((26.923)**2)))*(3.14159*((26.923)**2))**2) + (((PC3B['SigmaC']/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923**2)))**2)))
    PC3['SigmaCT'] = np.sqrt(((PC3S['SigmaCT']/(3.14159*((26.923)**2)))*(3.14159*((26.923)**2))**2) + (((PC3B['SigmaCT']/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923**2)))**2)))
   
    PC3['P'] = PC3S['P']
                             
    

    PC = pd.merge(PC1,PC2, on=['P'])
    PC = pd.merge(PC,PC3, on= ['P'])
    PC['COUNT'] = PC[['Count_x','Count_y', 'Count']].mean(axis=1)
    PC['COUNTT'] = PC[['CountT_x','CountT_y', 'CountT']].sum(axis=1)
    PC = PC.drop(columns = ['CountT_x','Count_x','CountT_y','Count_y','Count','CountT'])
    PC = PC.loc[(PC['COUNTT'] > 0)]

    
    PCP = PC.copy()
    PCP['P'] = PCP['P']+1
    PC = pd.concat([PC,PCP])
    
    
    return PC

In [ ]:
def SolarActivitySub(Det1S,Det1B,Det2S,Det2B,Det3S,Det3B,obs,Elow,EHigh,Ont1,Ont2,Ont3):
    #Generate a Lightcurve for Det1 obs 1, this is done for background subtraction, 
    #removing any data where the count rate is 3sigma above the background rate
    #LightCurve(Data,Det number, obs number, E_low,E_High)
    LCurve1S = LightCurve(Det1S,1,obs,Elow,EHigh,Ont1)
    LCurve1B = LightCurve(Det1B,1,obs,Elow,EHigh,Ont1)

    LC1mean= LCurve1B['CountRate'].mean()
    LC1std = (LCurve1B['CountRate'].std())

    LCurve1B = LCurve1B.loc[(LCurve1B['CountRate'] < (LC1mean + 3*LC1std))]
    LCurve1S = LCurve1S.loc[(LCurve1S['T'].isin(LCurve1B['T']) == True)]
    LCurve1S = LCurve1S.loc[(LCurve1S['T'].isin(LCurve1B['T']) == True)& LCurve1S['CountRate'] > (LCurve1S['CountRate'].mean() - 3*LCurve1S['CountRate'].std())]
    LCurve1B = LCurve1B.loc[(LCurve1B['T'].isin(LCurve1S['T']) == True)]

    LCurve1 = (LCurve1S/(np.pi*((Radius)**2)) - LCurve1B/(np.pi*(((4*Radius)**2)-(2*Radius)**2)))*(np.pi*((Radius)**2))
    LCurve1['T'] = LCurve1S['T']




    #Repeat for detector 2 and 3

    LCurve2S = LightCurve(Det2S,2,obs,Elow,EHigh,Ont2)
    LCurve2B = LightCurve(Det2B,2,obs,Elow,EHigh,Ont2)
    LC2mean= LCurve2B['CountRate'].mean()
    LC2std = (LCurve2B['CountRate'].std())


    LCurve2B = LCurve2B.loc[(LCurve2B['CountRate'] < (LC2mean + 3*LC2std))]
    LCurve2S = LCurve2S.loc[(LCurve2S['T'].isin(LCurve2B['T']) == True)]
    LCurve2S = LCurve2S.loc[(LCurve2S['T'].isin(LCurve2B['T']) == True)& LCurve2S['CountRate'] > (LCurve2S['CountRate'].mean() - 3*LCurve2S['CountRate'].std())]
    LCurve2B = LCurve2B.loc[(LCurve2B['T'].isin(LCurve2S['T']) == True)]

    LCurve2 = (LCurve2S/(np.pi*((Radius)**2)) - LCurve2B/(np.pi*(((4*Radius)**2)-(2*Radius)**2)))*(np.pi*((Radius)**2))
    LCurve2['T'] = LCurve2S['T']





    LCurve3S = LightCurve(Det3S,3,obs,Elow,EHigh,Ont3)
    LCurve3B = LightCurve(Det3B,3,obs,Elow,EHigh,Ont3)
    LC3mean= LCurve3B['CountRate'].mean()
    LC3std = (LCurve3B['CountRate'].std())
    LCurve3B = LCurve3B.loc[(LCurve3B['CountRate'] < (LC3mean + 3*LC3std))]
    LCurve3S = LCurve3S.loc[(LCurve3S['T'].isin(LCurve3B['T']) == True)]
    LCurve3S = LCurve3S.loc[(LCurve3S['T'].isin(LCurve3B['T']) == True)& LCurve3S['CountRate'] > (LCurve3S['CountRate'].mean() - 3*LCurve3S['CountRate'].std())]
    LCurve3B = LCurve3B.loc[(LCurve3B['T'].isin(LCurve3S['T']) == True)]

    LCurve3 = (LCurve3S/(np.pi*((Radius)**2)) - LCurve3B/(np.pi*(((4*Radius)**2)-(2*Radius)**2)))*(np.pi*((Radius)**2))
    LCurve3['T'] = LCurve3S['T']




    #Generate a lightcurve summing the 3 detectors
    LCurve = pd.DataFrame()
    LCurve['T'] = LCurve1['T']
    LCurve['CountRate'] = LCurve1['CountRate']+LCurve2['CountRate']+LCurve3['CountRate']

    LCurve1B['T'] = (LCurve1B['T'])*86400
    LCurve2B['T'] = (LCurve2B['T'])*86400
    LCurve3B['T'] = (LCurve3B['T'])*86400

    #if the data has been emitted from being above 3sigma, drop from det.
    #The lightCurves are binned while det is not
    for v in LCurve1B['T']:
        Det1SS = Det1S[(Det1S['T']<(v+(LCbin/2)))&(Det1S['T']<(v+(LCbin/2)))]
        Det1BB = Det1B[(Det1B['T']<(v+(LCbin/2)))&(Det1B['T']<(v+(LCbin/2)))]
    for v in LCurve2B['T']:
        Det2SS = Det2S[(Det2S['T']<(v+(LCbin/2)))&(Det2S['T']<(v+(LCbin/2)))]
        Det2BB = Det2B[(Det2B['T']<(v+(LCbin/2)))&(Det2B['T']<(v+(LCbin/2)))]
    for v in LCurve3B['T']:
        Det3SS = Det3S[(Det3S['T']<(v+(LCbin/2)))&(Det3S['T']<(v+(LCbin/2)))]
        Det3BB = Det3B[(Det3B['T']<(v+(LCbin/2)))&(Det3B['T']<(v+(LCbin/2)))]
    # We just reset the names here, if done before it was messing up the loops.
    Det1S = Det1SS
    Det1B = Det1BB
    Det2S = Det2SS
    Det2B = Det2BB
    Det3S = Det3SS
    Det3B = Det3BB

    return Det1S,Det1B,Det2S,Det2B,Det3S,Det3B

In [ ]:
Elow = 2
EHigh = 8
def LCurve_eachDetector(Det1S,Det1B,Detnumb,obs,Elow,EHigh,Ont1):
    LCurve1S = LightCurve(Det1S,Detnumb,obs,Elow,EHigh,Ont1)
    LCurve1B = LightCurve(Det1B,Detnumb,obs,Elow,EHigh,Ont1)
    LCurve1 = (LCurve1S/(np.pi*((Radius)**2)) - LCurve1B/(np.pi*(((4*Radius)**2)-(2*Radius)**2)))*(np.pi*((Radius)**2))
    LCurve1['T'] = LCurve1S['T']
    print(f'Observation {obs}, Detector {Detnumb}, Average Count Rate:' + str(LCurve1['CountRate'].mean()))
    plt.plot(LCurve1['T'],LCurve1['CountRate'],label=f'Det number: {Detnumb}')
obs =1   
plt.figure()
plt.title('Light Curves for each detector, observation:' + str(obs1date))
LCurve_eachDetector(Det1S,Det1B,1,obs,Elow,EHigh,Ont1)
LCurve_eachDetector(Det2S,Det2B,2,obs,Elow,EHigh,Ont2)
LCurve_eachDetector(Det3S,Det3B,3,obs,Elow,EHigh,Ont3)
plt.ylabel('RATE [cst/s]')
plt.xlabel('Time [days]')
plt.ylim(0.1,15)
plt.yscale('log')
plt.legend()
plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/DetectorLightCurves{obs}.jpeg')
plt.show()
plt.close()


obs =2 
plt.figure()
plt.title('Light Curves for each detector, observation:' + str(obs1date))
LCurve_eachDetector(Det12S,Det12B,1,obs,Elow,EHigh,Ont12)
LCurve_eachDetector(Det22S,Det22B,2,obs,Elow,EHigh,Ont22)
LCurve_eachDetector(Det32S,Det32B,3,obs,Elow,EHigh,Ont32)
plt.ylabel('RATE [cst/s]')
plt.xlabel('Time [days]')
plt.ylim(0.1,15)
plt.yscale('log')
plt.legend()
plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/DetectorLightCurves{obs}.jpeg')
plt.show()
plt.close()


obs = 3
plt.figure()
plt.title('Light Curves for each detector, observation dates:' + str(obs1date))
LCurve_eachDetector(Det13S,Det13B,1,obs,Elow,EHigh,Ont13)
LCurve_eachDetector(Det23S,Det23B,2,obs,Elow,EHigh,Ont23)
LCurve_eachDetector(Det33S,Det33B,3,obs,Elow,EHigh,Ont33)
plt.ylabel('RATE [cst/s]')
plt.xlabel('Time [days]')
plt.ylim(0.1,5)
plt.yscale('log')
plt.legend()
plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/DetectorLightCurves{obs}.jpeg')
plt.show()
plt.close()


In [ ]:
E_Low = 2
E_High =8 
obs = 1
LCurve1S = LightCurve(Det1S,1,1,E_Low,E_High,Ont1)
LCurve1B = LightCurve(Det1B,1,1,E_Low,E_High,Ont1)
LCurve1 = (LCurve1S/(3.14159*((26.923)**2)) - LCurve1B/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923)**2))
LCurve1['T'] = LCurve1S['T']

LCurve2S = LightCurve(Det2S,2,1,E_Low,E_High,Ont2)
LCurve2B = LightCurve(Det2B,2,1,E_Low,E_High,Ont2)
LCurve2 = (LCurve2S/(3.14159*((26.923)**2)) - LCurve2B/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923)**2))
LCurve2['T'] = LCurve2S['T']

LCurve3S = LightCurve(Det3S,3,1,E_Low,E_High,Ont3)
LCurve3B = LightCurve(Det3B,3,1,E_Low,E_High,Ont3)
LCurve3 = (LCurve3S/(3.14159*((26.923)**2)) - LCurve3B/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923)**2))
LCurve3['T'] = LCurve3S['T']

LCurve = pd.DataFrame()
LCurve['T'] = LCurve1['T']
LCurve['CountRate'] = LCurve1['CountRate']+LCurve2['CountRate']+LCurve3['CountRate']
LCurve1 = LCurve.copy()

plt.figure()
plt.scatter(LCurve['T'],LCurve['CountRate'], s=1)
plt.suptitle('Total Light curve Obs 1')
plt.title('01/ 18-21 /2023')
plt.ylabel('RATE [cst/s]')
plt.xlabel('Time [days]')
plt.ylim(0.01,15)
plt.yscale('log')
plt.axvspan(1.55,2.0, color = 'purple',alpha=0.25,label='eclipse')
#plt.legend()
plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/LightCurve{obs}.jpeg')
plt.show()
plt.close()


In [ ]:
E_Low = 2
E_High =8 
obs = 2
LCurve1S = LightCurve(Det12S,1,2,E_Low,E_High,Ont12)
LCurve1B = LightCurve(Det12B,1,2,E_Low,E_High,Ont12)
LCurve1 = (LCurve1S/(3.14159*((26.923)**2)) - LCurve1B/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923)**2))
LCurve1['T'] = LCurve1S['T']

LCurve2S = LightCurve(Det22S,2,2,E_Low,E_High,Ont22)
LCurve2B = LightCurve(Det22B,2,2,E_Low,E_High,Ont22)
LCurve2 = (LCurve2S/(3.14159*((26.923)**2)) - LCurve2B/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923)**2))
LCurve2['T'] = LCurve2S['T']

LCurve3S = LightCurve(Det32S,3,2,E_Low,E_High,Ont32)
LCurve3B = LightCurve(Det32B,3,2,E_Low,E_High,Ont32)
LCurve3 = (LCurve3S/(3.14159*((26.923)**2)) - LCurve3B/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923)**2))
LCurve3['T'] = LCurve3S['T']

LCurve = pd.DataFrame()
LCurve['T'] = LCurve1['T']
LCurve['CountRate'] = LCurve1['CountRate']+LCurve2['CountRate']+LCurve3['CountRate']
LCurve2 = LCurve.copy()


plt.figure()
plt.scatter(LCurve['T'],LCurve['CountRate'], s=1)
for i in np.arange(0,len(EclipseStart2)):
    plt.axvspan(EclipseStart2[i],EclipseEnd2[i], color = 'purple',alpha=0.25,label='eclipse')
plt.suptitle('Total Light curve Obs2')
plt.title('2/ 3-8 /2023')
plt.ylabel('RATE [cst/s]')
plt.xlabel('Time [days]')
plt.ylim(0.1,20)
plt.yscale('log')
#plt.legend()
plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/LightCurve{obs}.jpeg')
plt.show()
plt.close()

In [ ]:
E_Low = 2
E_High =8 
obs = 3
LCurve1S = LightCurve(Det13S,1,3,E_Low,E_High,Ont13)
LCurve1B = LightCurve(Det13B,1,3,E_Low,E_High,Ont13)
LCurve1 = (LCurve1S/(3.14159*((26.923)**2)) - LCurve1B/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923)**2))
LCurve1['T'] = LCurve1S['T']

LCurve2S = LightCurve(Det23S,2,3,E_Low,E_High,Ont23)
LCurve2B = LightCurve(Det23B,2,3,E_Low,E_High,Ont23)
LCurve2 = (LCurve2S/(3.14159*((26.923)**2)) - LCurve2B/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923)**2))
LCurve2['T'] = LCurve2S['T']

LCurve3S = LightCurve(Det33S,3,3,E_Low,E_High,Ont33)
LCurve3B = LightCurve(Det33B,3,3,E_Low,E_High,Ont33)
LCurve3 = (LCurve3S/(3.14159*((26.923)**2)) - LCurve3B/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923)**2))
LCurve3['T'] = LCurve3S['T']

LCurve = pd.DataFrame()
LCurve['T'] = LCurve1['T']
LCurve['CountRate'] = LCurve1['CountRate']+LCurve2['CountRate']+LCurve3['CountRate']
LCurve2 = LCurve.copy()

#FirstEclipse = [1.8,2.25]
#EclipseStart3 = [FirstEclipse[0],FirstEclipse[0]+par2[0],4.1]
#EclipseEnd3 = [FirstEclipse[1],FirstEclipse[1]+par2[0],5.6]


plt.figure()
plt.scatter(LCurve['T'],LCurve['CountRate'], s=1)
for i in np.arange(0,len(EclipseStart3)):
    plt.axvspan(EclipseStart3[i],EclipseEnd3[i], color = 'purple',alpha=0.25,label='eclipse')

plt.suptitle('Total Light curve Obs3')
plt.title('2/ 17-24 /2022')
plt.ylabel('RATE [cst/s]')
plt.xlabel('Time [days]')
plt.ylim(0.01,15)
plt.yscale('log')
#plt.legend()
plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/LightCurve{obs}.jpeg')
plt.show()
plt.close()

In [ ]:
'''f, Pxx_den = signal.periodogram(LCurve1['T'].T,0.003533)
plt.semilogy(f,Pxx_den)
plt.xlabel('frequency [Hz]')
plt.xlim(0,0.00001)
plt.ylabel('Power Spectral Density [V**2/Hz]')
plt.show()''';

In [ ]:
toton = plt.hist(Det12S['P'],bins=100)
print((toton[0].max()-toton[0].min())/(toton[0].max()+toton[0].min()))

In [ ]:
def FourierExpansion1(Phi,a,b,h):
    return a*np.cos(2*np.pi*h*Phi) + b*np.sin(2*np.pi*h*Phi)
def Fourier6thorder(Phi,a1,b1,a2,b2,a3,b3,a4,b4,a5,b5,a6,b6,Vshift):
    return FourierExpansion1(Phi,a1,b1,1) + FourierExpansion1(Phi,a2,b2,2) + FourierExpansion1(Phi,a3,b3,3) + FourierExpansion1(Phi,a4,b4,4)+ FourierExpansion1(Phi,a5,b5,5) + FourierExpansion1(Phi,a6,b6,6)+ Vshift

In [ ]:
def PulseprofileBC(Det1S,Det1B,Det2S,Det2B,Det3S,Det3B,EL,EH,*args, **kwargs):
    
    periodnum = kwargs.get('PNum',0)
    obs = kwargs.get('Obs',1)
    TStart = kwargs.get('Tstart',-1)
    TStart = float(TStart)
    EH = 4
    EL=2
    PC1S = PULSEPROFILE(Det1S, 1,obs, EL,EH,periodnum,Tstart = TStart)#DET MATRIX,DETNUMBER,LOW AND HIGH ELevels
    PC1B = PULSEPROFILE(Det1B, 1,obs, EL,EH,periodnum,Tstart = TStart)
    PC2S = PULSEPROFILE(Det2S, 2,obs, EL,EH,periodnum,Tstart = TStart)
    PC2B = PULSEPROFILE(Det2B, 2,obs, EL,EH,periodnum,Tstart = TStart)
    PC3S = PULSEPROFILE(Det3S, 3,obs, EL,EH,periodnum,Tstart = TStart)
    PC3B = PULSEPROFILE(Det3B, 3,obs, EL,EH,periodnum,Tstart = TStart)
    
    
        
    #PC1S = pd.DataFrame(PC1S, columns=['P','Count','CountT'])
    #PC2S = pd.DataFrame(PC2S, columns=['P','Count','CountT'])
    #PC3S = pd.DataFrame(PC3S, columns=['P','Count','CountT'])
    #PC1B = pd.DataFrame(PC1B, columns=['P','Count','CountT'])
    #PC2B = pd.DataFrame(PC2B, columns=['P','Count','CountT'])
       #PC3B = pd.DataFrame(PC3B, columns=['P','Count','CountT'])
    '''
    PC1 = (PC1S/(3.14159*((26.923)**2)) - PC1B/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923)**2))
    PC1['SigmaC'] = np.sqrt(((PC1S['SigmaC']/(3.14159*((26.923)**2)))*(3.14159*((26.923)**2))**2) + (((PC1B['SigmaC']/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923**2)))**2)))
    PC1['SigmaCT'] = np.sqrt(((PC1S['SigmaCT']/(3.14159*((26.923)**2)))*(3.14159*((26.923)**2))**2) + (((PC1B['SigmaCT']/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923**2)))**2)))
   
    PC1['P'] = PC1S['P']
    
    PC2 = (PC2S/(3.14159*((26.923)**2)) - PC2B/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923)**2))
    PC2['SigmaC'] = np.sqrt(((PC2S['SigmaC']/(3.14159*((26.923)**2)))*(3.14159*((26.923)**2))**2) + (((PC2B['SigmaC']/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923**2)))**2)))
    PC2['SigmaCT'] = np.sqrt(((PC2S['SigmaCT']/(3.14159*((26.923)**2)))*(3.14159*((26.923)**2))**2) + (((PC2B['SigmaCT']/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923**2)))**2)))
    PC2['P'] = PC2S['P']
    
    PC3 = (PC3S/(3.14159*((26.923)**2)) - PC3B/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923)**2))
    PC3['SigmaC'] = np.sqrt(((PC3S['SigmaC']/(3.14159*((26.923)**2)))*(3.14159*((26.923)**2))**2) + (((PC3B['SigmaC']/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923**2)))**2)))
    PC3['SigmaCT'] = np.sqrt(((PC3S['SigmaCT']/(3.14159*((26.923)**2)))*(3.14159*((26.923)**2))**2) + (((PC3B['SigmaCT']/(3.14159*(((107.69)**2)-(53.846)**2)))*(3.14159*((26.923**2)))**2)))
   
    PC3['P'] = PC3S['P']
    '''

    PC1 = PC1S
    PC2 = PC2S
    PC3 = PC3S
    
    PC = pd.merge(PC1,PC2, on=['P'])
    PC = pd.merge(PC,PC3, on= ['P'])
    PC['COUNT'] = PC[['Count_x','Count_y', 'Count']].mean(axis=1)
    PC['COUNTT'] = PC[['CountT_x','CountT_y', 'CountT']].sum(axis=1)
    PC['SigmaC'] = np.sqrt((PC1['SigmaC']**2)+(PC2['SigmaC']**2)+(PC3['SigmaC']**2))
    PC['SigmaCT'] = np.sqrt((PC1['SigmaCT']**2)+(PC2['SigmaCT']**2)+(PC3['SigmaCT']**2))
    
    PC = PC.drop(columns = ['CountT_x','Count_x','CountT_y','Count_y','Count','CountT','SigmaC_x','SigmaC_y','SigmaCT_x','SigmaCT_y'])
    PC = PC.loc[(PC['COUNTT'] > 0)]
    if PC['COUNTT'].empty ==True:
        PC['COUNTT'] = float('Nan')
        return PC,np.array([float('Nan')]),np.array([float('Nan')]),np.array([float('Nan')]),np.array([float('Nan')]),np.array([float('Nan')]),np.array([float('Nan')])

    PC['COUNTT'] = PC['COUNTT']/PC['COUNTT'].max()
    PC['SigmaCT'] = np.sqrt(((PC['SigmaCT']/PC['COUNTT'])**2)+(PC['COUNTT'].max()/PC['SigmaCT'].loc[PC['COUNTT'] == PC['COUNTT'].max()].mean())**2)
    PC['COUNTT'] = PC['COUNTT'].fillna(0)
    #PCP = PC.loc[(PC['COUNTT'] == 1)]
    #PC['P'] = PC['P']-PCP['P'].min()
    #PCP = PC.loc[(PC['P']<0)]
    #PCP['P'] = PCP['P'] + 1 

    #PC = PC.loc[(PC['P']>0)]
    #PC = pd.concat([PC,PCP])
    PCP = PC.copy()
    PCP['P'] = PCP['P']+1
    PC = pd.concat([PC,PCP])
    
    
    PCP = PC.loc[(PC['P'].between(0,1))]
    amp = abs(PCP['COUNTT'].max()-PCP['COUNTT'].min())
    vshift = PCP['COUNTT'].mean()
    
    params, params_covariance = opt.curve_fit(sinusoid,PC['P'].T,PC['COUNTT'].T,maxfev=100000,p0 =[amp,1,vshift], absolute_sigma = True,bounds=([0,0,0],[np.inf,2*np.pi,2*np.pi]))
    
    
    PCP = PC.loc[(PC['P']).between(0.1,1.1)&(PC['COUNTT'] == PC['COUNTT'].max())]
    mean = float(PCP['P'].mean())
    
    Smean = PC.loc[(PC['P'].between(mean-0.25,mean+0.25)&(~PC['P'].between(mean-0.05,mean+0.05)))]
    Smean = Smean.loc[(Smean['COUNTT'] == Smean['COUNTT'].max())]


    if PCP['P'].mean()< Smean['P'].mean():
        mean = float(Smean['P'].mean())
    
    PCP = PC.loc[PC['P'].between(mean-0.1,mean+0.1)]

    
    gparams,gparam_covariance = opt.curve_fit(gaussian,PCP['P'].T,PCP['COUNTT'].T,maxfev=100000,p0 = [1,mean,0.1], absolute_sigma = True,bounds=([0,0,0],[1,2,0.1]))

    #PCP = PC.loc[PC['P'].between(mean-0.25,mean+0.25)]
    Smean = PC.loc[(PC['P'].between(mean-0.25,mean+0.25)&(~PC['P'].between(mean-0.075,mean+0.075)))]
    Smean = Smean.loc[(Smean['COUNTT'] == Smean['COUNTT'].max())]
    Smean = float(Smean['P'].mean())

    PCP = PC.loc[PC['P'].between(((Smean+mean)/2)-0.2,((Smean+mean)/2)+0.2)]
    '''
    bparams,bparam_covariance = opt.curve_fit(bimodal,PCP['P'].T,PCP['COUNTT'].T,maxfev=100000,p0 = [0.9,Smean,0.05,1,mean,0.05], absolute_sigma = True)
    '''
    PCS = pd.merge(PC1S,PC2S, on=['P'])
    PCS = pd.merge(PCS,PC3S, on= ['P'])
    PCS['COUNT'] = PCS[['Count_x','Count_y', 'Count']].mean(axis=1)
    PCS['COUNTT'] = PCS[['CountT_x','CountT_y', 'CountT']].sum(axis=1)
    
    if gparams[0] > 0:
        gparams = gparams
        gparam_covariance = gparam_covariance
    else:
        gparams,gparam_covariance = np.array([float('Nan')]),np.array([float('Nan')])
    '''
    if bparams[0] > 0:
        bparams = bparams
        bparam_covariance = bparam_covariance
    else:
        bparams,bparam_covariance = np.array([float('Nan')]),np.array([float('Nan')])
    '''
    
    bparams = bparam_covariance = np.array([float('Nan')])
    return PC, params,params_covariance,gparams,gparam_covariance,bparams,bparam_covariance

In [ ]:
'''#plt.plot(LCurve2['T'],Fourier6thorder(LCurve2['CountRate'],params[0],params[1],params[2],params[3],params[4],params[5],params[6],params[7],params[8],params[9],params[10],params[11],params[12]),label = 'Fourier Obs1')
fs = np.linspace(0.001,2*np.pi,10000)
display(LCurve2)
f = signal.lombscargle(LCurve2['T'],LCurve2['CountRate'],fs)
plt.figure()
plt.plot(fs,f)
plt.show()
display(f)''';

PC,params,params_covariance,gparams,gparam_covariance,bparams,bparam_covariance = PulseprofileBC(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,2,7,Tstart = -1,PNum =0,Obs =2)
plt.figure()
plt.plot(PC['P'],PC['COUNTT'],color='blue',label = 'obs1')
plt.ylabel('Normalized Count Rate [cts/s]')
plt.xlabel('Phase [pi radians]')

plt.legend()
plt.show()
plt.close()

F_max = PC['COUNTT'].max()
F_min = PC['COUNTT'].min()
PF = 100*(F_max-F_min)/(F_max+F_min)

print(PF)

In [ ]:
para = pd.DataFrame(columns = ['amp','shift','d','t'])
Cerror = pd.DataFrame(columns = ['Sigma'])
GShift = pd.DataFrame(columns = ['Shift'])
BShift = pd.DataFrame(columns = ['Shift'])
gpara = pd.DataFrame(columns = ['amp', 'mean', 'stddev','t'])
bpara = pd.DataFrame(columns = ['amp1', 'mean1', 'stddev1','amp2', 'mean2', 'stddev2','t'])
pnum = 1000
Period = Period2
obs = 2

P = np.arange(0,EclipseStart2[0]*86400-pnum*Period,pnum*Period)
for k in range (len(EclipseEnd2)-1):
    C = np.arange(EclipseEnd2[k]*86400,EclipseStart2[k+1]*86400-pnum*Period,pnum*Period)
    P = np.append(P,C)

C = np.arange((EclipseEnd2[-1]*86400),Det12S['T'].max()-pnum*Period,pnum*Period)
P = np.append(P,C)
#for i in np.arange(0,Det12S['T'].max()-pnum*Period,pnum*Period):
#for i in np.arange(0,EclipseStart[0]*86400-5*pnum*Period,pnum*Period):
p = 0
for i in range(len(P)-1):

    TStart = 0
    
    k=0
    PC,params,params_covariance,gparams,gparams_covariance,bparams,bparams_covariance = PulseprofileBC(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,2,4,Tstart = P[i],PNum =pnum,Obs =2)
    if PC['COUNTT'].max() == 0:
        continue
    F_max = PC['COUNTT'].max()
    F_min = PC['COUNTT'].min()
    PF = 100*(F_max-F_min)/(F_max+F_min)
    '''
    
    if i + pnum*Period < (EclipseStart[k]- TStart)*86400 or i > (EclipseEnd[k]-TStart)*86400:
        PC,params,params_covariance,gparams,gparams_covariance,bparams,bparams_covariance = PulseprofileBC(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,2,4,Tstart = i,PNum =pnum,Obs =2)
    elif i + pnum*Period > (EclipseStart[k]- TStart)*86400 and i < (EclipseStart[k]-TStart)*86400:
        Pnum = ((EclipseStart[k]-TStart)*86400-i)/Period
        PC,params,params_covariance,gparams,gparams_covariance,bparams,bparams_covariance = PulseprofileBC(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,2,4,Tstart = i,PNum =Pnum,Obs=2)
    elif i + pnum*Period > (EclipseEnd[k]- TStart)*86400 and i < (EclipseEnd[k]-TStart)*86400:
        Tstart = (EclipseEnd[k]-TStart)*86400
        Pnum = ((i+pnum*Period)-Tstart)/Period
        PC,params,params_covariance,gparams,gparams_covariance,bparams,bparams_covariance = PulseprofileBC(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,2,4,Tstart = Tstart,PNum =Pnum,Obs=2)
    else:
        continue
    '''
    #PC1 = PulseprofileBC(Det1S,Det1B,Det2S,Det2B,Det3S,Det3B,2,4,300000,PNum =pnum)
    
    
    sigma = 1
    #if math.isnan(params[0]) == False and (2*params[0]/params[2])> 0.8:
    if math.isnan(params[0]) == False:#print(((params[0]-sigma*np.sqrt(params_covariance[0,0]))-(params[2]+sigma*np.sqrt(params_covariance[2,2])))> 0)
        #print(((params[2]-sigma*np.sqrt(params_covariance[2,2]))-(params[0]+sigma*np.sqrt(params_covariance[0,0])))> 0)
        p = p+1
        if (p%10) ==0:
            plt.figure()
            plt.plot(PC['P'], PC['COUNTT'])
            plt.plot(PC['P'],sinusoid(PC['P'],params[0],params[1],params[2]))
        
        
            plt.plot(PC['P'],gaussian(PC['P'],gparams[0],gparams[1],gparams[2]))
        #plt.plot(PC['P'],bimodal(PC['P'],bparams[0],bparams[1],bparams[2],bparams[3],bparams[4],bparams[5]))
        
        
            plt.title('Count Rate vs Phase, Time='+str(P[i])+'-'+str(P[i]+pnum*Period))
            plt.ylabel('Normalized Counts/delT [cts/s]')
            plt.xlabel('Phase [pi rad]')
            plt.xlim(0,2)
            plt.ylim(0,1)
        F_max = PC['COUNTT'].max()
        F_min = PC['COUNTT'].min()
        PF = 100*(F_max-F_min)/(F_max+F_min)
        
        if((((params[0]-sigma*np.sqrt(params_covariance[0,0]))>(params[2]+sigma*np.sqrt(params_covariance[2,2])))) or (((params[2]-sigma*np.sqrt(params_covariance[2,2]))>(params[0]+sigma*np.sqrt(params_covariance[0,0]))))):
            params = np.array([params])
            params = pd.DataFrame(params, columns = ['amp','shift','d'])
            params['t'] = P[i]+(pnum*Period/2)
            para = pd.concat([para,params])
            c_error = np.sqrt(params_covariance[1,1])
            c_error = np.array([c_error])
            c_error = pd.DataFrame(c_error, columns=['Sigma'])
            Cerror = pd.concat([Cerror,c_error])
            
        if PF>40:    
            gparams = np.array([gparams])
            #bparams = np.array([bparams])

            gparams = pd.DataFrame(gparams, columns = ['amp', 'mean', 'stddev'])
            #bparams = pd.DataFrame(bparams, columns = ['amp1', 'mean1', 'stddev1','amp2', 'mean2', 'stddev2'])

            gparams['t']= P[i]+(pnum*Period/2)
            #bparams['t']= i+(pnum*Period/2)


            gpara = pd.concat([gpara,gparams])
            #bpara = pd.concat([bpara,bparams])


            #print(PF)



            gshift = np.sqrt(gparams_covariance[1,1])
            g_shift = np.array([gshift*2*np.pi])
            g_shift = pd.DataFrame(g_shift, columns=['Sigma'])
            GShift = pd.concat([GShift,g_shift])

        #bshift = np.sqrt(bparams_covariance[1,1])
        #b_shift = np.array([bshift*2*np.pi])
        #b_shift = pd.DataFrame(b_shift, columns=['Sigma'])
        #BShift = pd.concat([BShift,b_shift])
    #plt.axvspan(59687.7,59689.5,label = 'Eclipse', color = 'green', alpha =0.25)
    #plt.yscale('log')
    #plt.legend()
plt.show()
plt.close()

para['shift'] = para['shift']*2*np.pi
params = para.loc[(para['shift']<0)]
para = para.loc[(para['shift']>0)]

params['shift'] = params['shift']+2*np.pi

para = pd.concat([para,params])

para = para.sort_values(by =['t'])



gpara['shift'] = gpara['mean']*2*np.pi
gparams = gpara.loc[(gpara['shift']>2*np.pi)]
gpara = gpara.loc[(gpara['shift']<2*np.pi)]
gparams['shift'] = gparams['shift']-2*np.pi



gpara = pd.concat([gpara,gparams])
gpara = gpara.sort_values(by=['t'])
'''
bpara['shift'] = ((bpara['mean1']+bpara['mean2'])/2)*2*np.pi
bparams = bpara.loc[(bpara['shift']>2*np.pi)]
bpara = bpara.loc[(bpara['shift']<2*np.pi)]
bparams['shift'] = bparams['shift']-2*np.pi
bpara = pd.concat([bpara,bparams])'''



para = para.reset_index()
Cerror = Cerror.reset_index()
gpara = gpara.reset_index()
GShift = GShift.reset_index()
#bpara = bpara.reset_index()
#BShift = BShift.reset_index()

parai = para[((para['shift']>(2*np.pi)) | (para['shift']<0))].index
gparai = gpara[((gpara['shift']>(2*np.pi)) | (gpara['shift']<0))].index
#bparai = bpara[((bpara['shift']>(2*np.pi)) | (bpara['shift']<0))].index


if parai.empty == False:
    para = para.drop(index = parai)
    Cerror = Cerror.drop(index = parai)
if gparai.empty == False:
    gpara = gpara.drop(index = gparai)
    GShift = GShift.drop(index = gparai)
'''
if bparai.empty == False:    
    bpara = bpara.drop(index = bparai)
    BShift = BShift.drop(index = bparai)
'''


#,bounds=([0,-np.pi,0,34.5,-np.pi],[np.inf,np.pi,np.inf,35,np.pi])
#parafit, parafit_covariance = opt.curve_fit(c_curvefit,para['t'].T,para['shift'].T,maxfev=10000,p0 =[1,1,1,34.84,0.1],sigma = Cerror['Sigma'],bounds=([0,-np.pi,0,34.5,-np.pi],[np.inf,np.pi,np.inf,35,np.pi]))
gparafit, gparafit_covariance = opt.curve_fit(c_curvefit,gpara['t'].T,gpara['shift'].T,maxfev=10000,p0 =[1,0,0,34.84,0])
#bparafit, bparafit_covariance = opt.curve_fit(c_curvefit,bpara['t'].T,bpara['shift'].T,maxfev=10000,p0 =[1,0,1])

#F_max = PC['COUNTT'].max()
#F_min = PC['COUNTT'].min()

#PF = 100*(F_max-F_min)/(F_max+F_min)
#print(PF)

T = np.linspace(0,Det12S['T'].max()+1000,100)

para = para.reset_index()
#para = para.drop([3])



#Error bars denote 68.3% confidence (1 sigma)
plt.figure()
plt.errorbar(para['t'],para['shift'],yerr = Cerror['Sigma'],xerr = pnum*Period/2,fmt='o',markersize=4,color = 'orange')
#plt.plot(T,c_curvefit(T,parafit[0],parafit[1],parafit[2],parafit[3],parafit[4]),color = 'orange')

#plt.errorbar(bpara['t'],bpara['shift'],xerr = 0.683*pnum*Period/2,yerr = BShift['Sigma'],fmt='o',markersize=4,color = 'red')
#plt.plot(T,c_curvefit(T,bparafit[0],bparafit[1],bparafit[2]),color = 'red')
plt.xlabel('Time [s]')
plt.ylabel('Phase Shift [radians]')
plt.ylim(0,2*np.pi)
#plt.ylim(0,2*np.pi)
plt.show()
plt.close()

#display(bparafit)
#display(parafit_covariance)

plt.figure()

plt.errorbar(gpara['t'],gpara['shift'],yerr = GShift['Sigma'],xerr = pnum*Period/2,fmt='o',markersize=3,color = 'green')
plt.plot(T,c_curvefit(T,gparafit[0],gparafit[1],gparafit[2],gparafit[3],gparafit[4]),color = 'green')
#plt.plot(T,c_curvefit(T,3*np.pi,40000,0.5,34.84,0),color = 'green')
#plt.errorbar(bpara['t'],bpara['shift'],xerr = 0.683*pnum*Period/2,yerr = BShift['Sigma'],fmt='o',markersize=4,color = 'red')
#plt.plot(T,c_curvefit(T,bparafit[0],bparafit[1],bparafit[2]),color = 'red')
plt.xlabel('Time [s]')
plt.ylabel('Phase Shift [radians]')
plt.ylim(0,2*np.pi)
#plt.ylim(0,2*np.pi)
plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/PulseProfileShifts{obs}.jpeg')
plt.show()
plt.close()

In [ ]:
def PeriodCorrection(x,m,b):
    return (m*x+b)

In [ ]:
def Sawtooth(x,m,b):
    y = m*x+b
    if y > (2*np.pi):
        y = y-(2*np.pi)
    if y < 0:
        y = y+(2*np.pi)
    return y
def PhaseJumps(df):
    df['prevphase'] = df['shift'].shift(1)
    df['nextphase'] = df['shift'].shift(-1)
    df['prevtime'] = df['t'].shift(1)
    df['nexttime'] = df['t'].shift(-1)

    df = df.fillna(value={'prevphase':-10})
    
    df['startofphase'] = (df['shift']>(df['prevphase']+2))
    
    df['endofphase'] = (df['shift']<(df['nextphase']-2))
    
    #df['timejump'] = (df['t'] == 2*df['t']) #creating a column of False
    df['endtimejump'] = (df['t'] > (df['prevtime']+ 7000))
    df['timejump'] = (df['nexttime'] > (df['t']+ 7000))
    
    df_ = df.loc[((df['startofphase'] == True)&(df['shift']>2))|(df['timejump']==True)|(df['endtimejump']==True)|(df['endofphase']==True)]
    df_ = df_.loc[~(df_['t'].between(251193,251194)) & ~(df_['t'].between(342787,342788))& ~(df_['t'].between(104298,106800)) ]
    #df_ = df_[df_['startofphase'] != df_['timejump']] 
    #df_ = df_[df_['endofphase'] != df_['timejump']]
    
    df_['nextindex'] = df_['level_0'].shift(-1)
    
    #display(df_.iloc[(df_['level_0'] = (df_['nextindex']))].values)
    #df_ = df_[(df_['level_0'] != (df_['nextindex']-1))&(df_['endtimejump'] == True)]
    
    
    return df_

In [ ]:
gpara['Sigma'] = GShift['Sigma']

PeriodC = pd.DataFrame(columns = ['slope', 'intercept','time_start','time_end'])
def FindSlopes(gpara,T):
    gparaP = gpara.loc[gpara['t'].between(T.min(),T.max()) == True]
    PeriodCorr, PeriodCorr_covariance = opt.curve_fit(PeriodCorrection,gparaP['t'].T,gparaP['shift'].T,sigma =gparaP['Sigma'],maxfev=10000,p0 =[-5,7])
    PeriodCorr = np.array([PeriodCorr])
    PeriodCorr = pd.DataFrame(PeriodCorr,columns=['slope', 'intercept'])
    PeriodCorr['time_start'] = T.min()
    PeriodCorr['time_end'] = T.max()
    PeriodCorr = PeriodCorr.astype('float64')
    return PeriodCorr


#gpara = gpara.drop(columns = ['level_0'])
gpara = gpara.reset_index()

jumps = PhaseJumps(gpara)

'''
T = np.linspace(0,28000,100)

PeriodCorr = FindSlopes(gpara,T)
display(PeriodCorr)

T2 = np.linspace(137000,165000,100)
PeriodCorr2 = FindSlopes(gpara,T2)

display(PeriodCorr2)

T3 = np.linspace(275000,300000,100)
PeriodCorr3 = FindSlopes(gpara,T3)

display(PeriodCorr3)
'''

for i in np.arange(0,(len(jumps)-1),2):
    T = np.linspace(jumps['t'].iloc[i],jumps['t'].iloc[i+1],100)
    PeriodCorr = FindSlopes(gpara,T)
    PeriodC = pd.concat([PeriodC,PeriodCorr])
plt.figure()

plt.errorbar(gpara['t'],gpara['shift'],yerr = GShift['Sigma'],xerr = pnum*Period/2,fmt='o',markersize=3,color = 'green')
for i in range((len(PeriodC))):
    T = np.linspace(PeriodC['time_start'].iloc[i],PeriodC['time_end'].iloc[i],100)
    plt.plot(T,PeriodCorrection(T,PeriodC['slope'].iloc[i],PeriodC['intercept'].iloc[i]))
plt.scatter(jumps['t'],jumps['shift'],color='red')
plt.xlabel('Time [s]')
plt.ylabel('Phase Shift [radians]')
plt.ylim(0,2*np.pi)
#plt.ylim(0,2*np.pi)
plt.show()
plt.close()

display(PeriodC)

In [ ]:
para = pd.DataFrame(columns = ['amp','shift','d','t'])
Cerror = pd.DataFrame(columns = ['Sigma'])
GShift = pd.DataFrame(columns = ['Shift'])
BShift = pd.DataFrame(columns = ['Shift'])
gpara = pd.DataFrame(columns = ['amp', 'mean', 'stddev','t'])
bpara = pd.DataFrame(columns = ['amp1', 'mean1', 'stddev1','amp2', 'mean2', 'stddev2','t'])
pnum = 1000
Period = Period3

obs = 3

P = np.arange(1.6*86400,EclipseStart3[0]*86400-pnum*Period,pnum*Period)
for k in range (len(EclipseEnd3)-1):
    C = np.arange(EclipseEnd3[k]*86400,EclipseStart3[k+1]*86400-pnum*Period,pnum*Period)
    P = np.append(P,C)

C = np.arange((EclipseEnd3[-1]*86400),Det13S['T'].max()-pnum*Period,pnum*Period)
P = np.append(P,C)



#for i in np.arange(0,Det12S['T'].max()-pnum*Period,pnum*Period):
#for i in np.arange(0,EclipseStart[0]*86400-5*pnum*Period,pnum*Period):
p = 0
for i in range(len(P)-1):

    TStart = 0
    
    k=0
    PC,params,params_covariance,gparams,gparams_covariance,bparams,bparams_covariance = PulseprofileBC(Det13S,Det13B,Det23S,Det23B,Det33S,Det33B,2,4,Tstart = P[i],PNum =pnum,Obs =3)
    if PC['COUNTT'].max() == 0:
        continue
    F_max = PC['COUNTT'].max()
    F_min = PC['COUNTT'].min()
    PF = 100*(F_max-F_min)/(F_max+F_min)
    '''
    
    if i + pnum*Period < (EclipseStart[k]- TStart)*86400 or i > (EclipseEnd[k]-TStart)*86400:
        PC,params,params_covariance,gparams,gparams_covariance,bparams,bparams_covariance = PulseprofileBC(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,2,4,Tstart = i,PNum =pnum,Obs =2)
    elif i + pnum*Period > (EclipseStart[k]- TStart)*86400 and i < (EclipseStart[k]-TStart)*86400:
        Pnum = ((EclipseStart[k]-TStart)*86400-i)/Period
        PC,params,params_covariance,gparams,gparams_covariance,bparams,bparams_covariance = PulseprofileBC(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,2,4,Tstart = i,PNum =Pnum,Obs=2)
    elif i + pnum*Period > (EclipseEnd[k]- TStart)*86400 and i < (EclipseEnd[k]-TStart)*86400:
        Tstart = (EclipseEnd[k]-TStart)*86400
        Pnum = ((i+pnum*Period)-Tstart)/Period
        PC,params,params_covariance,gparams,gparams_covariance,bparams,bparams_covariance = PulseprofileBC(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,2,4,Tstart = Tstart,PNum =Pnum,Obs=2)
    else:
        continue
    '''
    #PC1 = PulseprofileBC(Det1S,Det1B,Det2S,Det2B,Det3S,Det3B,2,4,300000,PNum =pnum)
    
    
    sigma = 1
    #if math.isnan(params[0]) == False and (2*params[0]/params[2])> 0.8:
    if math.isnan(params[0]) == False:#print(((params[0]-sigma*np.sqrt(params_covariance[0,0]))-(params[2]+sigma*np.sqrt(params_covariance[2,2])))> 0)
        #print(((params[2]-sigma*np.sqrt(params_covariance[2,2]))-(params[0]+sigma*np.sqrt(params_covariance[0,0])))> 0)
        p = p+1
        if (p%10) ==0:
            plt.figure()
            plt.plot(PC['P'], PC['COUNTT'])
            plt.plot(PC['P'],sinusoid(PC['P'],params[0],params[1],params[2]))
        
        
            plt.plot(PC['P'],gaussian(PC['P'],gparams[0],gparams[1],gparams[2]))
        #plt.plot(PC['P'],bimodal(PC['P'],bparams[0],bparams[1],bparams[2],bparams[3],bparams[4],bparams[5]))
        
        
            plt.title('Count Rate vs Phase, Time='+str(P[i])+'-'+str(P[i]+pnum*Period))
            plt.ylabel('Normalized Counts/delT [cts/s]')
            plt.xlabel('Phase [pi rad]')
            plt.xlim(0,2)
            plt.ylim(0,1)
        F_max = PC['COUNTT'].max()
        F_min = PC['COUNTT'].min()
        PF = 100*(F_max-F_min)/(F_max+F_min)
        
        if((((params[0]-sigma*np.sqrt(params_covariance[0,0]))>(params[2]+sigma*np.sqrt(params_covariance[2,2])))) or (((params[2]-sigma*np.sqrt(params_covariance[2,2]))>(params[0]+sigma*np.sqrt(params_covariance[0,0]))))):
            params = np.array([params])
            params = pd.DataFrame(params, columns = ['amp','shift','d'])
            params['t'] = P[i]+(pnum*Period/2)
            para = pd.concat([para,params])
            c_error = np.sqrt(params_covariance[1,1])
            c_error = np.array([c_error])
            c_error = pd.DataFrame(c_error, columns=['Sigma'])
            Cerror = pd.concat([Cerror,c_error])
            
        if PF>40:    
            gparams = np.array([gparams])
            #bparams = np.array([bparams])

            gparams = pd.DataFrame(gparams, columns = ['amp', 'mean', 'stddev'])
            #bparams = pd.DataFrame(bparams, columns = ['amp1', 'mean1', 'stddev1','amp2', 'mean2', 'stddev2'])

            gparams['t']= P[i]+(pnum*Period/2)
            #bparams['t']= i+(pnum*Period/2)


            gpara = pd.concat([gpara,gparams])
            #bpara = pd.concat([bpara,bparams])


            #print(PF)



            gshift = np.sqrt(gparams_covariance[1,1])
            g_shift = np.array([gshift*2*np.pi])
            g_shift = pd.DataFrame(g_shift, columns=['Sigma'])
            GShift = pd.concat([GShift,g_shift])

        #bshift = np.sqrt(bparams_covariance[1,1])
        #b_shift = np.array([bshift*2*np.pi])
        #b_shift = pd.DataFrame(b_shift, columns=['Sigma'])
        #BShift = pd.concat([BShift,b_shift])
    #plt.axvspan(59687.7,59689.5,label = 'Eclipse', color = 'green', alpha =0.25)
    #plt.yscale('log')
    #plt.legend()
plt.show()
plt.close()

para['shift'] = para['shift']*2*np.pi
params = para.loc[(para['shift']<0)]
para = para.loc[(para['shift']>0)]

params['shift'] = params['shift']+2*np.pi

para = pd.concat([para,params])

para = para.sort_values(by =['t'])



gpara['shift'] = gpara['mean']*2*np.pi
gparams = gpara.loc[(gpara['shift']>2*np.pi)]
gpara = gpara.loc[(gpara['shift']<2*np.pi)]
gparams['shift'] = gparams['shift']-2*np.pi



gpara = pd.concat([gpara,gparams])
gpara = gpara.sort_values(by=['t'])
'''
bpara['shift'] = ((bpara['mean1']+bpara['mean2'])/2)*2*np.pi
bparams = bpara.loc[(bpara['shift']>2*np.pi)]
bpara = bpara.loc[(bpara['shift']<2*np.pi)]
bparams['shift'] = bparams['shift']-2*np.pi
bpara = pd.concat([bpara,bparams])'''



para = para.reset_index()
Cerror = Cerror.reset_index()
gpara = gpara.reset_index()
GShift = GShift.reset_index()
#bpara = bpara.reset_index()
#BShift = BShift.reset_index()

parai = para[((para['shift']>(2*np.pi)) | (para['shift']<0))].index
gparai = gpara[((gpara['shift']>(2*np.pi)) | (gpara['shift']<0))].index
#bparai = bpara[((bpara['shift']>(2*np.pi)) | (bpara['shift']<0))].index


if parai.empty == False:
    para = para.drop(index = parai)
    Cerror = Cerror.drop(index = parai)
if gparai.empty == False:
    gpara = gpara.drop(index = gparai)
    GShift = GShift.drop(index = gparai)
'''
if bparai.empty == False:    
    bpara = bpara.drop(index = bparai)
    BShift = BShift.drop(index = bparai)
'''


#,bounds=([0,-np.pi,0,34.5,-np.pi],[np.inf,np.pi,np.inf,35,np.pi])
#parafit, parafit_covariance = opt.curve_fit(c_curvefit,para['t'].T,para['shift'].T,maxfev=10000,p0 =[1,1,1,34.84,0.1],sigma = Cerror['Sigma'],bounds=([0,-np.pi,0,34.5,-np.pi],[np.inf,np.pi,np.inf,35,np.pi]))
gparafit, gparafit_covariance = opt.curve_fit(c_curvefit,gpara['t'].T,gpara['shift'].T,maxfev=10000,p0 =[1,0,0,34.84,0])
#bparafit, bparafit_covariance = opt.curve_fit(c_curvefit,bpara['t'].T,bpara['shift'].T,maxfev=10000,p0 =[1,0,1])

#F_max = PC['COUNTT'].max()
#F_min = PC['COUNTT'].min()

#PF = 100*(F_max-F_min)/(F_max+F_min)
#print(PF)

T = np.linspace(0,Det13S['T'].max()+1000,100)

para = para.reset_index()
#para = para.drop([3])



#Error bars denote 68.3% confidence (1 sigma)
plt.figure()
plt.errorbar(para['t'],para['shift'],yerr = Cerror['Sigma'],xerr = pnum*Period/2,fmt='o',markersize=4,color = 'orange')
#plt.plot(T,c_curvefit(T,parafit[0],parafit[1],parafit[2],parafit[3],parafit[4]),color = 'orange')

#plt.errorbar(bpara['t'],bpara['shift'],xerr = 0.683*pnum*Period/2,yerr = BShift['Sigma'],fmt='o',markersize=4,color = 'red')
#plt.plot(T,c_curvefit(T,bparafit[0],bparafit[1],bparafit[2]),color = 'red')
plt.xlabel('Time [s]')
plt.ylabel('Phase Shift [radians]')
plt.ylim(0,2*np.pi)
#plt.ylim(0,2*np.pi)
plt.show()
plt.close()

#display(bparafit)
#display(parafit_covariance)

plt.figure()
plt.title(f'PhaseShifts Obs:{obs}')
plt.errorbar(gpara['t'],gpara['shift'],yerr = GShift['Sigma'],xerr = pnum*Period/2,fmt='o',markersize=3,color = 'green')
plt.plot(T,c_curvefit(T,gparafit[0],gparafit[1],gparafit[2],gparafit[3],gparafit[4]),color = 'green')
#plt.plot(T,c_curvefit(T,3*np.pi,40000,0.5,34.84,0),color = 'green')
#plt.errorbar(bpara['t'],bpara['shift'],xerr = 0.683*pnum*Period/2,yerr = BShift['Sigma'],fmt='o',markersize=4,color = 'red')
#plt.plot(T,c_curvefit(T,bparafit[0],bparafit[1],bparafit[2]),color = 'red')
plt.xlabel('Time [s]')
plt.ylabel('Phase Shift [radians]')
plt.ylim(0,2*np.pi)
#plt.ylim(0,2*np.pi)
plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/PulseProfileShifts{obs}.jpeg')
plt.show()
plt.close()

In [ ]:
PeriodC['correction'] = PeriodC['slope']/(2*np.pi)
#display(PeriodC)

def PhaseCorrection(Det,correction,Period,vdot):
    correction = (correction/(2*np.pi))
    
    Period_ = Period + correction
    Det['P'] = (PulsePhaseModel(Det['T'],Phi0,1/Period_,vdot)%(1))
    return Det
obs = 3
Period = Period3
EclipseStart,EclipseEnd = EclipseStart3,EclipseEnd3



if obs == 3:
    TZero = 1.6*86400
else:
    TZero = 0


P = np.array([TZero,EclipseStart[0]*86400])

#C = np.array([EclipseEnd[0]*86400,EclipseStart[1]*86400])
#P = np.append(P,C)
for i in range(len(EclipseStart)-2):
    C = np.array([EclipseEnd[i+1]*86400,EclipseStart[i+2]*86400])
    P = np.append(P,C)
P = np.append(P,Det13S['T'].max()-150)
print(P)




Det1G,Det1G_,Det2G,Det2G_,Det3G,Det3G_ = Det13S.copy(),Det13B.copy(),Det23S.copy(),Det23B.copy(),Det33S.copy(),Det33B.copy()
#Det1G,Det1G_,Det2G,Det2G_,Det3G,Det3G_ = Det12S.copy(),Det12B.copy(),Det22S.copy(),Det22B.copy(),Det32S.copy(),Det32B.copy()

vdot = 0
#correction =PeriodC['slope'].mean() #-2.811e-4
def Period_correction(correction):
    #vdot = correction[1]
    correction = correction[0]
    Det1SG,Det1BG,Det2SG,Det2BG,Det3SG,Det3BG =  Det1G.copy(),Det1G_.copy(),Det2G.copy(),Det2G_.copy(),Det3G.copy(),Det3G_.copy()
    Det1SG = PhaseCorrection(Det1SG,correction,Period,vdot)
    Det1BG = PhaseCorrection(Det1BG,correction,Period,vdot)
    Det2SG = PhaseCorrection(Det2SG,correction,Period,vdot)
    Det2BG = PhaseCorrection(Det2BG,correction,Period,vdot)
    Det3SG = PhaseCorrection(Det3SG,correction,Period,vdot)
    Det3BG = PhaseCorrection(Det3BG,correction,Period,vdot)

    PC,_,_,_,_,_,_ = PulseprofileBC(Det1SG,Det1BG,Det2SG,Det2BG,Det3SG,Det3BG,2,4,Tstart = TZero,PNum =(P[-1]-P[0])/Period,Obs =obs)
    F_max = PC['COUNTT'].max()
    F_min = PC['COUNTT'].min()
    PF = 100*(F_max-F_min)/(F_max+F_min)
    return 1/PF
#plt.figure()
#plt.plot(PC['P'], PC['COUNTT'])
            
#plt.title('Count Rate vs Phase, Time')
#plt.ylabel('Normalized Counts/delT [cts/s]')
#plt.xlabel('Phase [rev]')
#plt.xlim(0,2)
#plt.ylim(0,1)
#plt.show()
#plt.close()


bnds = [(-1e-3,1.5e-4)]#,(-6000e-12,1000e-12)]

Det1G,Det1G_,Det2G,Det2G_,Det3G,Det3G_ = Det13S.copy(),Det13B.copy(),Det23S.copy(),Det23B.copy(),Det33S.copy(),Det33B.copy()
#Det1G,Det1G_,Det2G,Det2G_,Det3G,Det3G_ = Det12S.copy(),Det12B.copy(),Det22S.copy(),Det22B.copy(),Det32S.copy(),Det32B.copy()

res = opt.brute(Period_correction,ranges = bnds)
print(res)


Det1G,Det1G_,Det2G,Det2G_,Det3G,Det3G_ = Det13S.copy(),Det13B.copy(),Det23S.copy(),Det23B.copy(),Det33S.copy(),Det33B.copy()
#Det1G,Det1G_,Det2G,Det2G_,Det3G,Det3G_ = Det12S.copy(),Det12B.copy(),Det22S.copy(),Det22B.copy(),Det32S.copy(),Det32B.copy()

correction = res[0]
#vdot = res[1]
Det1G = PhaseCorrection(Det1G,correction,Period,vdot)
Det1G_ = PhaseCorrection(Det1G_,correction,Period,vdot)
Det2G = PhaseCorrection(Det2G,correction,Period,vdot)
Det2G_ = PhaseCorrection(Det2G_,correction,Period,vdot)
Det3G = PhaseCorrection(Det3G,correction,Period,vdot)
Det3G_ = PhaseCorrection(Det3G_,correction,Period,vdot)
plt.figure()



for i in range(len(P)-1):
#for i in range(0,1):
    PC,_,_,_,_,_,_ = PulseprofileBC( Det1G,Det1G_,Det2G,Det2G_,Det3G,Det3G_,2,4,Tstart = P[i],PNum =(P[i+1]-P[i])/Period,Obs =obs)
    F_max = PC['COUNTT'].max()
    F_min = PC['COUNTT'].min()
    PF = 100*(F_max-F_min)/(F_max+F_min)
    print(PF)

    plt.plot(PC['P'], PC['COUNTT'],label =f'Time: {P[i]} - {P[i+1]} ') 

    plt.title('Count Rate vs Phase, Time')
    plt.ylabel('Normalized Counts/delT [cts/s]')
    plt.xlabel('Phase [revs]')
    plt.xlim(0,2)
    plt.ylim(0,1)
    plt.legend()
    plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/ManyPulseProfileobs{obs}.jpeg')
plt.show()
plt.close()

plt.figure()
PC,_,_,_,_,_,_ = PulseprofileBC(Det1G,Det1G_,Det2G,Det2G_,Det3G,Det3G_,2,4,Tstart = TZero,PNum =(P[-1]-P[0])/Period,Obs =obs)
F_max = PC['COUNTT'].max()
F_min = PC['COUNTT'].min()
PF = 100*(F_max-F_min)/(F_max+F_min)
print(PF)

plt.plot(PC['P'], PC['COUNTT']) 

plt.title('NO Vdot Obs3 Count Rate vs Phase, Time')
plt.ylabel('Normalized Counts/delT [cts/s]')
plt.xlabel('Phase [revs]')
plt.xlim(0,2)
plt.ylim(0,1)
#plt.legend()
plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/PulseProfileVdotobs{obs}.jpeg')
plt.show()
plt.close()

Det13S,Det13B,Det23S,Det23B,Det33S,Det33B=Det1G,Det1G_,Det2G,Det2G_,Det3G,Det3G_

In [ ]:
PeriodC['correction'] = PeriodC['slope']/(2*np.pi)
#display(PeriodC)

def PhaseCorrection(Det,correction,Period,vdot):
    correction = (correction/(2*np.pi))
    
    Period_ = Period + correction
    Det['P'] = (PulsePhaseModel(Det['T'],Phi0,1/Period_,vdot)%(1))
    return Det
obs = 2
Period = Period2
EclipseStart,EclipseEnd = EclipseStart2,EclipseEnd2
#Det1G,Det1G_,Det2G,Det2G_,Det3G,Det3G_ = Det13S.copy(),Det13B.copy(),Det23S.copy(),Det23B.copy(),Det33S.copy(),Det33B.copy()
Det1G,Det1G_,Det2G,Det2G_,Det3G,Det3G_ = Det12S.copy(),Det12B.copy(),Det22S.copy(),Det22B.copy(),Det32S.copy(),Det32B.copy()

vdot = 0
#correction =PeriodC['slope'].mean() #-2.811e-4
def Period_correction(correction):
    #vdot = correction[1]
    correction = correction[0]
    Det1SG,Det1BG,Det2SG,Det2BG,Det3SG,Det3BG =  Det1G.copy(),Det1G_.copy(),Det2G.copy(),Det2G_.copy(),Det3G.copy(),Det3G_.copy()
    Det1SG = PhaseCorrection(Det1SG,correction,Period,vdot)
    Det1BG = PhaseCorrection(Det1BG,correction,Period,vdot)
    Det2SG = PhaseCorrection(Det2SG,correction,Period,vdot)
    Det2BG = PhaseCorrection(Det2BG,correction,Period,vdot)
    Det3SG = PhaseCorrection(Det3SG,correction,Period,vdot)
    Det3BG = PhaseCorrection(Det3BG,correction,Period,vdot)

    PC,_,_,_,_,_,_ = PulseprofileBC(Det1SG,Det1BG,Det2SG,Det2BG,Det3SG,Det3BG,2,4,Tstart = -1,PNum =0,Obs =obs)
    F_max = PC['COUNTT'].max()
    F_min = PC['COUNTT'].min()
    PF = 100*(F_max-F_min)/(F_max+F_min)
    return 1/PF



bnds = [(-1e-3,1.5e-4)]#,(-6000e-12,1000e-12))

#Det1G,Det1G_,Det2G,Det2G_,Det3G,Det3G_ = Det13S.copy(),Det13B.copy(),Det23S.copy(),Det23B.copy(),Det33S.copy(),Det33B.copy()
Det1G,Det1G_,Det2G,Det2G_,Det3G,Det3G_ = Det12S.copy(),Det12B.copy(),Det22S.copy(),Det22B.copy(),Det32S.copy(),Det32B.copy()

res = opt.brute(Period_correction,ranges = bnds)
print(res)


#Det1G,Det1G_,Det2G,Det2G_,Det3G,Det3G_ = Det13S.copy(),Det13B.copy(),Det23S.copy(),Det23B.copy(),Det33S.copy(),Det33B.copy()
Det1G,Det1G_,Det2G,Det2G_,Det3G,Det3G_ = Det12S.copy(),Det12B.copy(),Det22S.copy(),Det22B.copy(),Det32S.copy(),Det32B.copy()

correction = res[0]
#vdot = res[1]
Det1G = PhaseCorrection(Det1G,correction,Period,vdot)
Det1G_ = PhaseCorrection(Det1G_,correction,Period,vdot)
Det2G = PhaseCorrection(Det2G,correction,Period,vdot)
Det2G_ = PhaseCorrection(Det2G_,correction,Period,vdot)
Det3G = PhaseCorrection(Det3G,correction,Period,vdot)
Det3G_ = PhaseCorrection(Det3G_,correction,Period,vdot)
plt.figure()

if obs == 3:
    TZero = 1.6*86400
else:
    TZero = 0
P = np.array([TZero,EclipseStart[0]])
C = np.arange(EclipseEnd[0],EclipseStart[1]-(25000/Period),25000/Period)
P = np.append(P,C)
for i in range(len(EclipseStart)-2):
    C = np.arange(EclipseEnd[i+1],EclipseStart[i+2]-(25000/Period),25000/Period)
    P = np.append(P,C)
P = np.append(P,Det1G['T'].max()-150)
print(P)

for i in range(len(P)-1):
#for i in range(0,1):
    PC,_,_,_,_,_,_ = PulseprofileBC( Det1G,Det1G_,Det2G,Det2G_,Det3G,Det3G_,2,4,Tstart = P[i],PNum =(P[i+1]-P[i])/Period,Obs =obs)
    F_max = PC['COUNTT'].max()
    F_min = PC['COUNTT'].min()
    PF = 100*(F_max-F_min)/(F_max+F_min)
    print(PF)

    plt.plot(PC['P'], PC['COUNTT'],label =f'Time: {P[i]} - {P[i]+(25000/Period)} ') 

    plt.title('Count Rate vs Phase, Time')
    plt.ylabel('Normalized Counts/delT [cts/s]')
    plt.xlabel('Phase [revs]')
    plt.xlim(0,2)
    plt.ylim(0,1)
    plt.legend()
    plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/ManyPulseProfileVdotobs{obs}.jpeg')
plt.show()
plt.close()

plt.figure()
PC,_,_,_,_,_,_ = PulseprofileBC(Det1G,Det1G_,Det2G,Det2G_,Det3G,Det3G_,2,4,Tstart = -1,PNum =0,Obs =obs)
F_max = PC['COUNTT'].max()
F_min = PC['COUNTT'].min()
PF = 100*(F_max-F_min)/(F_max+F_min)
print(PF)

plt.plot(PC['P'], PC['COUNTT']) 

plt.title(f'NO Vdot Obs: {obs} Count Rate vs Phase, Time')
plt.ylabel('Normalized Counts/delT [cts/s]')
plt.xlabel('Phase [revs]')
plt.xlim(0,2)
plt.ylim(0,1)
#plt.legend()
plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/PulseProfileVdotobs{obs}.jpeg')
plt.show()
plt.close()

Det12S,Det12B,Det22S,Det22B,Det32S,Det32B=Det1G,Det1G_,Det2G,Det2G_,Det3G,Det3G_

In [ ]:
PeriodC['correction'] = PeriodC['slope']/(2*np.pi)
#display(PeriodC)

def PhaseCorrection(Det,correction,Period,vdot):
    correction = (correction/(2*np.pi))
    
    Period_ = Period + correction
    Det['P'] = (PulsePhaseModel(Det['T'],Phi0,1/Period_,vdot)%(1))
    return Det
obs = 1
Period = Period1
EclipseStart,EclipseEnd = EclipseStart1,EclipseEnd1
#Det1G,Det1G_,Det2G,Det2G_,Det3G,Det3G_ = Det13S.copy(),Det13B.copy(),Det23S.copy(),Det23B.copy(),Det33S.copy(),Det33B.copy()
Det1G,Det1G_,Det2G,Det2G_,Det3G,Det3G_ = Det1S.copy(),Det1B.copy(),Det2S.copy(),Det2B.copy(),Det3S.copy(),Det3B.copy()

vdot = 0
#correction =PeriodC['slope'].mean() #-2.811e-4
def Period_correction(correction):
    #vdot = correction[1]
    correction = correction[0]
    Det1SG,Det1BG,Det2SG,Det2BG,Det3SG,Det3BG =  Det1G.copy(),Det1G_.copy(),Det2G.copy(),Det2G_.copy(),Det3G.copy(),Det3G_.copy()
    Det1SG = PhaseCorrection(Det1SG,correction,Period,vdot)
    Det1BG = PhaseCorrection(Det1BG,correction,Period,vdot)
    Det2SG = PhaseCorrection(Det2SG,correction,Period,vdot)
    Det2BG = PhaseCorrection(Det2BG,correction,Period,vdot)
    Det3SG = PhaseCorrection(Det3SG,correction,Period,vdot)
    Det3BG = PhaseCorrection(Det3BG,correction,Period,vdot)

    PC,_,_,_,_,_,_ = PulseprofileBC(Det1SG,Det1BG,Det2SG,Det2BG,Det3SG,Det3BG,2,4,Tstart = -1,PNum =0,Obs =obs)
    F_max = PC['COUNTT'].max()
    F_min = PC['COUNTT'].min()
    PF = 100*(F_max-F_min)/(F_max+F_min)
    return 1/PF


bnds = [(-1e-3,1.5e-4)]#,(-6000e-12,1000e-12))

#Det1G,Det1G_,Det2G,Det2G_,Det3G,Det3G_ = Det13S.copy(),Det13B.copy(),Det23S.copy(),Det23B.copy(),Det33S.copy(),Det33B.copy()
Det1G,Det1G_,Det2G,Det2G_,Det3G,Det3G_ = Det1S.copy(),Det1B.copy(),Det2S.copy(),Det2B.copy(),Det3S.copy(),Det3B.copy()

res = opt.brute(Period_correction,ranges = bnds)
print(res)


#Det1G,Det1G_,Det2G,Det2G_,Det3G,Det3G_ = Det13S.copy(),Det13B.copy(),Det23S.copy(),Det23B.copy(),Det33S.copy(),Det33B.copy()
Det1G,Det1G_,Det2G,Det2G_,Det3G,Det3G_ = Det1S.copy(),Det1B.copy(),Det2S.copy(),Det2B.copy(),Det3S.copy(),Det3B.copy()

correction = res[0]
#vdot = res[1]
Det1G = PhaseCorrection(Det1G,correction,Period,vdot)
Det1G_ = PhaseCorrection(Det1G_,correction,Period,vdot)
Det2G = PhaseCorrection(Det2G,correction,Period,vdot)
Det2G_ = PhaseCorrection(Det2G_,correction,Period,vdot)
Det3G = PhaseCorrection(Det3G,correction,Period,vdot)
Det3G_ = PhaseCorrection(Det3G_,correction,Period,vdot)
plt.figure()

if obs == 3:
    TZero = 1.6*86400
else:
    TZero = 0
'''
P = np.array([TZero,EclipseStart[0]])
C = np.arange(EclipseEnd[0],EclipseStart[1]-(25000/Period),25000/Period)
P = np.append(P,C)
for i in range(len(EclipseStart)-2):
    C = np.arange(EclipseEnd[i+1],EclipseStart[i+2]-(25000/Period),25000/Period)
    P = np.append(P,C)
P = np.append(P,Det1G['T'].max()-150)
'''
P = np.array([TZero,Det1G['T'].max()])
print(P)

for i in range(len(P)-1):
#for i in range(0,1):
    PC,_,_,_,_,_,_ = PulseprofileBC( Det1G,Det1G_,Det2G,Det2G_,Det3G,Det3G_,2,4,Tstart = P[i],PNum =(P[i+1]-P[i])/Period,Obs =obs)
    F_max = PC['COUNTT'].max()
    F_min = PC['COUNTT'].min()
    PF = 100*(F_max-F_min)/(F_max+F_min)
    print(PF)

    plt.plot(PC['P'], PC['COUNTT'],label =f'Time: {P[i]} - {P[i]+(25000/Period)} ') 

    plt.title('Count Rate vs Phase, Time')
    plt.ylabel('Normalized Counts/delT [cts/s]')
    plt.xlabel('Phase [revs]')
    plt.xlim(0,2)
    plt.ylim(0,1)
    plt.legend()
    plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/ManyPulseProfileVdotobs{obs}.jpeg')
plt.show()
plt.close()

plt.figure()
PC,_,_,_,_,_,_ = PulseprofileBC(Det1G,Det1G_,Det2G,Det2G_,Det3G,Det3G_,2,4,Tstart = -1,PNum =0,Obs =2)
F_max = PC['COUNTT'].max()
F_min = PC['COUNTT'].min()
PF = 100*(F_max-F_min)/(F_max+F_min)
print(PF)

plt.plot(PC['P'], PC['COUNTT']) 

plt.title(f'NO Vdot Obs{obs} Count Rate vs Phase, Time')
plt.ylabel('Normalized Counts/delT [cts/s]')
plt.xlabel('Phase [revs]')
plt.xlim(0,2)
plt.ylim(0,1)
#plt.legend()
plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/PulseProfileVdotobs{obs}.jpeg')
plt.show()
plt.close()

Det1S,Det1B,Det2S,Det2B,Det3S,Det3B=Det1G,Det1G_,Det2G,Det2G_,Det3G,Det3G_

In [ ]:
para = pd.DataFrame(columns = ['amp','shift','d','t'])
Cerror = pd.DataFrame(columns = ['Sigma'])
GShift = pd.DataFrame(columns = ['Shift'])
BShift = pd.DataFrame(columns = ['Shift'])
gpara = pd.DataFrame(columns = ['amp', 'mean', 'stddev','t'])
bpara = pd.DataFrame(columns = ['amp1', 'mean1', 'stddev1','amp2', 'mean2', 'stddev2','t'])
pnum = 1000
Period = Period2
obs = 2

P = np.arange(0,EclipseStart2[0]*86400-pnum*Period,pnum*Period)
for k in range (len(EclipseEnd2)-1):
    C = np.arange(EclipseEnd2[k]*86400,EclipseStart2[k+1]*86400-pnum*Period,pnum*Period)
    P = np.append(P,C)

C = np.arange((EclipseEnd2[-1]*86400),Det12S['T'].max()-pnum*Period,pnum*Period)
P = np.append(P,C)
#for i in np.arange(0,Det12S['T'].max()-pnum*Period,pnum*Period):
#for i in np.arange(0,EclipseStart[0]*86400-5*pnum*Period,pnum*Period):
p = 0
for i in range(len(P)-1):

    TStart = 0
    
    k=0
    PC,params,params_covariance,gparams,gparams_covariance,bparams,bparams_covariance = PulseprofileBC(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,2,4,Tstart = P[i],PNum =pnum,Obs =2)
    if PC['COUNTT'].max() == 0:
        continue
    F_max = PC['COUNTT'].max()
    F_min = PC['COUNTT'].min()
    PF = 100*(F_max-F_min)/(F_max+F_min)
    '''
    
    if i + pnum*Period < (EclipseStart[k]- TStart)*86400 or i > (EclipseEnd[k]-TStart)*86400:
        PC,params,params_covariance,gparams,gparams_covariance,bparams,bparams_covariance = PulseprofileBC(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,2,4,Tstart = i,PNum =pnum,Obs =2)
    elif i + pnum*Period > (EclipseStart[k]- TStart)*86400 and i < (EclipseStart[k]-TStart)*86400:
        Pnum = ((EclipseStart[k]-TStart)*86400-i)/Period
        PC,params,params_covariance,gparams,gparams_covariance,bparams,bparams_covariance = PulseprofileBC(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,2,4,Tstart = i,PNum =Pnum,Obs=2)
    elif i + pnum*Period > (EclipseEnd[k]- TStart)*86400 and i < (EclipseEnd[k]-TStart)*86400:
        Tstart = (EclipseEnd[k]-TStart)*86400
        Pnum = ((i+pnum*Period)-Tstart)/Period
        PC,params,params_covariance,gparams,gparams_covariance,bparams,bparams_covariance = PulseprofileBC(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,2,4,Tstart = Tstart,PNum =Pnum,Obs=2)
    else:
        continue
    '''
    #PC1 = PulseprofileBC(Det1S,Det1B,Det2S,Det2B,Det3S,Det3B,2,4,300000,PNum =pnum)
    
    
    sigma = 1
    #if math.isnan(params[0]) == False and (2*params[0]/params[2])> 0.8:
    if math.isnan(params[0]) == False:#print(((params[0]-sigma*np.sqrt(params_covariance[0,0]))-(params[2]+sigma*np.sqrt(params_covariance[2,2])))> 0)
        #print(((params[2]-sigma*np.sqrt(params_covariance[2,2]))-(params[0]+sigma*np.sqrt(params_covariance[0,0])))> 0)
        p = p+1
        if (p%10) ==0:
            plt.figure()
            plt.plot(PC['P'], PC['COUNTT'])
            plt.plot(PC['P'],sinusoid(PC['P'],params[0],params[1],params[2]))
        
        
            plt.plot(PC['P'],gaussian(PC['P'],gparams[0],gparams[1],gparams[2]))
        #plt.plot(PC['P'],bimodal(PC['P'],bparams[0],bparams[1],bparams[2],bparams[3],bparams[4],bparams[5]))
        
        
            plt.title('Count Rate vs Phase, Time='+str(P[i])+'-'+str(P[i]+pnum*Period))
            plt.ylabel('Normalized Counts/delT [cts/s]')
            plt.xlabel('Phase [pi rad]')
            plt.xlim(0,2)
            plt.ylim(0,1)
        F_max = PC['COUNTT'].max()
        F_min = PC['COUNTT'].min()
        PF = 100*(F_max-F_min)/(F_max+F_min)
        
        if((((params[0]-sigma*np.sqrt(params_covariance[0,0]))>(params[2]+sigma*np.sqrt(params_covariance[2,2])))) or (((params[2]-sigma*np.sqrt(params_covariance[2,2]))>(params[0]+sigma*np.sqrt(params_covariance[0,0]))))):
            params = np.array([params])
            params = pd.DataFrame(params, columns = ['amp','shift','d'])
            params['t'] = P[i]+(pnum*Period/2)
            para = pd.concat([para,params])
            c_error = np.sqrt(params_covariance[1,1])
            c_error = np.array([c_error])
            c_error = pd.DataFrame(c_error, columns=['Sigma'])
            Cerror = pd.concat([Cerror,c_error])
            
        if PF>40:    
            gparams = np.array([gparams])
            #bparams = np.array([bparams])

            gparams = pd.DataFrame(gparams, columns = ['amp', 'mean', 'stddev'])
            #bparams = pd.DataFrame(bparams, columns = ['amp1', 'mean1', 'stddev1','amp2', 'mean2', 'stddev2'])

            gparams['t']= P[i]+(pnum*Period/2)
            #bparams['t']= i+(pnum*Period/2)


            gpara = pd.concat([gpara,gparams])
            #bpara = pd.concat([bpara,bparams])


            #print(PF)



            gshift = np.sqrt(gparams_covariance[1,1])
            g_shift = np.array([gshift*2*np.pi])
            g_shift = pd.DataFrame(g_shift, columns=['Sigma'])
            GShift = pd.concat([GShift,g_shift])

        #bshift = np.sqrt(bparams_covariance[1,1])
        #b_shift = np.array([bshift*2*np.pi])
        #b_shift = pd.DataFrame(b_shift, columns=['Sigma'])
        #BShift = pd.concat([BShift,b_shift])
    #plt.axvspan(59687.7,59689.5,label = 'Eclipse', color = 'green', alpha =0.25)
    #plt.yscale('log')
    #plt.legend()
plt.show()
plt.close()

para['shift'] = para['shift']*2*np.pi
params = para.loc[(para['shift']<0)]
para = para.loc[(para['shift']>0)]

params['shift'] = params['shift']+2*np.pi

para = pd.concat([para,params])

para = para.sort_values(by =['t'])



gpara['shift'] = gpara['mean']*2*np.pi
gparams = gpara.loc[(gpara['shift']>np.pi)]
gpara = gpara.loc[(gpara['shift']<np.pi)]
gparams['shift'] = gparams['shift']-2*np.pi



gpara = pd.concat([gpara,gparams])
gpara = gpara.sort_values(by=['t'])
'''
bpara['shift'] = ((bpara['mean1']+bpara['mean2'])/2)*2*np.pi
bparams = bpara.loc[(bpara['shift']>2*np.pi)]
bpara = bpara.loc[(bpara['shift']<2*np.pi)]
bparams['shift'] = bparams['shift']-2*np.pi
bpara = pd.concat([bpara,bparams])'''



para = para.reset_index()
Cerror = Cerror.reset_index()
gpara = gpara.reset_index()
GShift = GShift.reset_index()
#bpara = bpara.reset_index()
#BShift = BShift.reset_index()

parai = para[((para['shift']>(2*np.pi)) | (para['shift']<0))].index
gparai = gpara[((gpara['shift']>(2*np.pi)) | (gpara['shift']<0))].index
#bparai = bpara[((bpara['shift']>(2*np.pi)) | (bpara['shift']<0))].index


if parai.empty == False:
    para = para.drop(index = parai)
    Cerror = Cerror.drop(index = parai)
if gparai.empty == False:
    gpara = gpara.drop(index = gparai)
    GShift = GShift.drop(index = gparai)
'''
if bparai.empty == False:    
    bpara = bpara.drop(index = bparai)
    BShift = BShift.drop(index = bparai)
'''


#,bounds=([0,-np.pi,0,34.5,-np.pi],[np.inf,np.pi,np.inf,35,np.pi])
#parafit, parafit_covariance = opt.curve_fit(c_curvefit,para['t'].T,para['shift'].T,maxfev=10000,p0 =[1,1,1,34.84,0.1],sigma = Cerror['Sigma'],bounds=([0,-np.pi,0,34.5,-np.pi],[np.inf,np.pi,np.inf,35,np.pi]))
gparafit, gparafit_covariance = opt.curve_fit(c_curvefit,gpara['t'].T,gpara['shift'].T,maxfev=10000,p0 =[1,0,0,34.84,0])
#bparafit, bparafit_covariance = opt.curve_fit(c_curvefit,bpara['t'].T,bpara['shift'].T,maxfev=10000,p0 =[1,0,1])

#F_max = PC['COUNTT'].max()
#F_min = PC['COUNTT'].min()

#PF = 100*(F_max-F_min)/(F_max+F_min)
#print(PF)

T = np.linspace(0,Det12S['T'].max()+1000,100)

para = para.reset_index()
#para = para.drop([3])



#Error bars denote 68.3% confidence (1 sigma)
plt.figure()
plt.errorbar(para['t'],para['shift'],yerr = Cerror['Sigma'],xerr = pnum*Period/2,fmt='o',markersize=4,color = 'orange')
#plt.plot(T,c_curvefit(T,parafit[0],parafit[1],parafit[2],parafit[3],parafit[4]),color = 'orange')

#plt.errorbar(bpara['t'],bpara['shift'],xerr = 0.683*pnum*Period/2,yerr = BShift['Sigma'],fmt='o',markersize=4,color = 'red')
#plt.plot(T,c_curvefit(T,bparafit[0],bparafit[1],bparafit[2]),color = 'red')
plt.xlabel('Time [s]')
plt.ylabel('Phase Shift [radians]')
plt.ylim(0,2*np.pi)
#plt.ylim(0,2*np.pi)
plt.show()
plt.close()

#display(bparafit)
#display(parafit_covariance)

plt.figure()

plt.title('PhaseShifts After Period Correction')
plt.errorbar(gpara['t'],gpara['shift'],yerr = GShift['Sigma'],xerr = pnum*Period/2,fmt='o',markersize=3,color = 'green')
plt.plot(T,c_curvefit(T,gparafit[0],gparafit[1],gparafit[2],gparafit[3],gparafit[4]),color = 'green')
#plt.plot(T,c_curvefit(T,3*np.pi,40000,0.5,34.84,0),color = 'green')
#plt.errorbar(bpara['t'],bpara['shift'],xerr = 0.683*pnum*Period/2,yerr = BShift['Sigma'],fmt='o',markersize=4,color = 'red')
#plt.plot(T,c_curvefit(T,bparafit[0],bparafit[1],bparafit[2]),color = 'red')
plt.xlabel('Time [s]')
plt.ylabel('Phase Shift [radians]')
plt.ylim(-np.pi,np.pi)
#plt.ylim(0,2*np.pi)
plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/PulseProfileShiftsAfterPeriodCorrection{obs}.jpeg')
plt.show()
plt.close()

In [ ]:
para = pd.DataFrame(columns = ['amp','shift','d','t'])
Cerror = pd.DataFrame(columns = ['Sigma'])
GShift = pd.DataFrame(columns = ['Shift'])
BShift = pd.DataFrame(columns = ['Shift'])
gpara = pd.DataFrame(columns = ['amp', 'mean', 'stddev','t'])
bpara = pd.DataFrame(columns = ['amp1', 'mean1', 'stddev1','amp2', 'mean2', 'stddev2','t'])
pnum = 1000
Period = Period3

obs = 3

P = np.arange(1.6*86400,EclipseStart3[0]*86400-pnum*Period,pnum*Period)
for k in range (len(EclipseEnd3)-1):
    C = np.arange(EclipseEnd3[k]*86400,EclipseStart3[k+1]*86400-pnum*Period,pnum*Period)
    P = np.append(P,C)

C = np.arange((EclipseEnd3[-1]*86400),Det13S['T'].max()-pnum*Period,pnum*Period)
P = np.append(P,C)



#for i in np.arange(0,Det12S['T'].max()-pnum*Period,pnum*Period):
#for i in np.arange(0,EclipseStart[0]*86400-5*pnum*Period,pnum*Period):
p = 0
for i in range(len(P)-1):

    TStart = 0
    
    k=0
    PC,params,params_covariance,gparams,gparams_covariance,bparams,bparams_covariance = PulseprofileBC(Det13S,Det13B,Det23S,Det23B,Det33S,Det33B,2,4,Tstart = P[i],PNum =pnum,Obs =3)
    if PC['COUNTT'].max() == 0:
        continue
    F_max = PC['COUNTT'].max()
    F_min = PC['COUNTT'].min()
    PF = 100*(F_max-F_min)/(F_max+F_min)
    '''
    
    if i + pnum*Period < (EclipseStart[k]- TStart)*86400 or i > (EclipseEnd[k]-TStart)*86400:
        PC,params,params_covariance,gparams,gparams_covariance,bparams,bparams_covariance = PulseprofileBC(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,2,4,Tstart = i,PNum =pnum,Obs =2)
    elif i + pnum*Period > (EclipseStart[k]- TStart)*86400 and i < (EclipseStart[k]-TStart)*86400:
        Pnum = ((EclipseStart[k]-TStart)*86400-i)/Period
        PC,params,params_covariance,gparams,gparams_covariance,bparams,bparams_covariance = PulseprofileBC(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,2,4,Tstart = i,PNum =Pnum,Obs=2)
    elif i + pnum*Period > (EclipseEnd[k]- TStart)*86400 and i < (EclipseEnd[k]-TStart)*86400:
        Tstart = (EclipseEnd[k]-TStart)*86400
        Pnum = ((i+pnum*Period)-Tstart)/Period
        PC,params,params_covariance,gparams,gparams_covariance,bparams,bparams_covariance = PulseprofileBC(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,2,4,Tstart = Tstart,PNum =Pnum,Obs=2)
    else:
        continue
    '''
    #PC1 = PulseprofileBC(Det1S,Det1B,Det2S,Det2B,Det3S,Det3B,2,4,300000,PNum =pnum)
    
    
    sigma = 1
    #if math.isnan(params[0]) == False and (2*params[0]/params[2])> 0.8:
    if math.isnan(params[0]) == False:#print(((params[0]-sigma*np.sqrt(params_covariance[0,0]))-(params[2]+sigma*np.sqrt(params_covariance[2,2])))> 0)
        #print(((params[2]-sigma*np.sqrt(params_covariance[2,2]))-(params[0]+sigma*np.sqrt(params_covariance[0,0])))> 0)
        p = p+1
        if (p%10) ==0:
            plt.figure()
            plt.plot(PC['P'], PC['COUNTT'])
            plt.plot(PC['P'],sinusoid(PC['P'],params[0],params[1],params[2]))
        
        
            plt.plot(PC['P'],gaussian(PC['P'],gparams[0],gparams[1],gparams[2]))
        #plt.plot(PC['P'],bimodal(PC['P'],bparams[0],bparams[1],bparams[2],bparams[3],bparams[4],bparams[5]))
        
        
            plt.title('Count Rate vs Phase, Time='+str(P[i])+'-'+str(P[i]+pnum*Period))
            plt.ylabel('Normalized Counts/delT [cts/s]')
            plt.xlabel('Phase [pi rad]')
            plt.xlim(0,2)
            plt.ylim(0,1)
        F_max = PC['COUNTT'].max()
        F_min = PC['COUNTT'].min()
        PF = 100*(F_max-F_min)/(F_max+F_min)
        
        if((((params[0]-sigma*np.sqrt(params_covariance[0,0]))>(params[2]+sigma*np.sqrt(params_covariance[2,2])))) or (((params[2]-sigma*np.sqrt(params_covariance[2,2]))>(params[0]+sigma*np.sqrt(params_covariance[0,0]))))):
            params = np.array([params])
            params = pd.DataFrame(params, columns = ['amp','shift','d'])
            params['t'] = P[i]+(pnum*Period/2)
            para = pd.concat([para,params])
            c_error = np.sqrt(params_covariance[1,1])
            c_error = np.array([c_error])
            c_error = pd.DataFrame(c_error, columns=['Sigma'])
            Cerror = pd.concat([Cerror,c_error])
            
        if PF>40:    
            gparams = np.array([gparams])
            #bparams = np.array([bparams])

            gparams = pd.DataFrame(gparams, columns = ['amp', 'mean', 'stddev'])
            #bparams = pd.DataFrame(bparams, columns = ['amp1', 'mean1', 'stddev1','amp2', 'mean2', 'stddev2'])

            gparams['t']= P[i]+(pnum*Period/2)
            #bparams['t']= i+(pnum*Period/2)


            gpara = pd.concat([gpara,gparams])
            #bpara = pd.concat([bpara,bparams])


            #print(PF)



            gshift = np.sqrt(gparams_covariance[1,1])
            g_shift = np.array([gshift*2*np.pi])
            g_shift = pd.DataFrame(g_shift, columns=['Sigma'])
            GShift = pd.concat([GShift,g_shift])

        #bshift = np.sqrt(bparams_covariance[1,1])
        #b_shift = np.array([bshift*2*np.pi])
        #b_shift = pd.DataFrame(b_shift, columns=['Sigma'])
        #BShift = pd.concat([BShift,b_shift])
    #plt.axvspan(59687.7,59689.5,label = 'Eclipse', color = 'green', alpha =0.25)
    #plt.yscale('log')
    #plt.legend()
plt.show()
plt.close()

para['shift'] = para['shift']*2*np.pi
params = para.loc[(para['shift']<0)]
para = para.loc[(para['shift']>0)]

params['shift'] = params['shift']+2*np.pi

para = pd.concat([para,params])

para = para.sort_values(by =['t'])



gpara['shift'] = gpara['mean']*2*np.pi
gparams = gpara.loc[(gpara['shift']>np.pi)]
gpara = gpara.loc[(gpara['shift']<np.pi)]
gparams['shift'] = gparams['shift']-2*np.pi



gpara = pd.concat([gpara,gparams])
gpara = gpara.sort_values(by=['t'])
'''
bpara['shift'] = ((bpara['mean1']+bpara['mean2'])/2)*2*np.pi
bparams = bpara.loc[(bpara['shift']>2*np.pi)]
bpara = bpara.loc[(bpara['shift']<2*np.pi)]
bparams['shift'] = bparams['shift']-2*np.pi
bpara = pd.concat([bpara,bparams])'''



para = para.reset_index()
Cerror = Cerror.reset_index()
gpara = gpara.reset_index()
GShift = GShift.reset_index()
#bpara = bpara.reset_index()
#BShift = BShift.reset_index()

parai = para[((para['shift']>(2*np.pi)) | (para['shift']<0))].index
gparai = gpara[((gpara['shift']>(2*np.pi)) | (gpara['shift']<0))].index
#bparai = bpara[((bpara['shift']>(2*np.pi)) | (bpara['shift']<0))].index


if parai.empty == False:
    para = para.drop(index = parai)
    Cerror = Cerror.drop(index = parai)
if gparai.empty == False:
    gpara = gpara.drop(index = gparai)
    GShift = GShift.drop(index = gparai)
'''
if bparai.empty == False:    
    bpara = bpara.drop(index = bparai)
    BShift = BShift.drop(index = bparai)
'''


#,bounds=([0,-np.pi,0,34.5,-np.pi],[np.inf,np.pi,np.inf,35,np.pi])
#parafit, parafit_covariance = opt.curve_fit(c_curvefit,para['t'].T,para['shift'].T,maxfev=10000,p0 =[1,1,1,34.84,0.1],sigma = Cerror['Sigma'],bounds=([0,-np.pi,0,34.5,-np.pi],[np.inf,np.pi,np.inf,35,np.pi]))
gparafit, gparafit_covariance = opt.curve_fit(c_curvefit,gpara['t'].T,gpara['shift'].T,maxfev=10000,p0 =[1,0,0,34.84,0])
#bparafit, bparafit_covariance = opt.curve_fit(c_curvefit,bpara['t'].T,bpara['shift'].T,maxfev=10000,p0 =[1,0,1])

#F_max = PC['COUNTT'].max()
#F_min = PC['COUNTT'].min()

#PF = 100*(F_max-F_min)/(F_max+F_min)
#print(PF)

T = np.linspace(0,Det13S['T'].max()+1000,100)

para = para.reset_index()
#para = para.drop([3])



#Error bars denote 68.3% confidence (1 sigma)
plt.figure()
plt.errorbar(para['t'],para['shift'],yerr = Cerror['Sigma'],xerr = pnum*Period/2,fmt='o',markersize=4,color = 'orange')
#plt.plot(T,c_curvefit(T,parafit[0],parafit[1],parafit[2],parafit[3],parafit[4]),color = 'orange')

#plt.errorbar(bpara['t'],bpara['shift'],xerr = 0.683*pnum*Period/2,yerr = BShift['Sigma'],fmt='o',markersize=4,color = 'red')
#plt.plot(T,c_curvefit(T,bparafit[0],bparafit[1],bparafit[2]),color = 'red')
plt.xlabel('Time [s]')
plt.ylabel('Phase Shift [radians]')
plt.ylim(0,2*np.pi)
#plt.ylim(0,2*np.pi)
plt.show()
plt.close()

#display(bparafit)
#display(parafit_covariance)

plt.figure()
plt.title(f'PhaseShifts After Correction Obs:{obs}')
plt.errorbar(gpara['t'],gpara['shift'],yerr = GShift['Sigma'],xerr = pnum*Period/2,fmt='o',markersize=3,color = 'green')
plt.plot(T,c_curvefit(T,gparafit[0],gparafit[1],gparafit[2],gparafit[3],gparafit[4]),color = 'green')
#plt.plot(T,c_curvefit(T,3*np.pi,40000,0.5,34.84,0),color = 'green')
#plt.errorbar(bpara['t'],bpara['shift'],xerr = 0.683*pnum*Period/2,yerr = BShift['Sigma'],fmt='o',markersize=4,color = 'red')
#plt.plot(T,c_curvefit(T,bparafit[0],bparafit[1],bparafit[2]),color = 'red')
plt.xlabel('Time [s]')
plt.ylabel('Phase Shift [radians]')
plt.ylim(-np.pi,np.pi)
#plt.ylim(0,2*np.pi)
plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/PulseProfileShiftsAfterCorrection{obs}.jpeg')
plt.show()
plt.close()

In [ ]:
print(gparafit)

In [ ]:
'''
def  PULSEPROF(Det,Detnumb,obs, E_low,E_high,periodnum, *args, **kwargs):
#Det,Detnumb, E_low,E_high = Det1S,1,2,8
#for f in range(0,1):
    TStart = kwargs.get('Tstart',-1)
    P = []
    Count = []
    CountT = []
    df1 = Det
    r = 0.0083333 #1/120 120bins in the data
    
    if TStart >-1:
        df1 = df1.loc[(df1['T']>TStart)&(df1['T']<TStart+periodnum*Period1)]
        
    if Detnumb == 1:
        df1 = pd.merge(df1,dfRESPAE, left_on='CH', right_on='CH1')
        df1.drop(columns =['CH2', 'CH3','E2_AVG', 'E3_AVG', 'Resp2', 'Resp3'])
        df1['E'] = df1['E1_AVG']
        df1['RESP'] = df1['Resp1']
        if TStart >-1 and periodnum >0:
            if obs ==1:
                ONT1 = Ont1.loc[(Ont1['OnT_Start']>TStart)&(Ont1['OnT_Start']<TStart+periodnum*Period1)]
                delT=ONT1['OnT'].sum()
            if obs == 2:
                ONT1 = Ont12.loc[(Ont12['OnT_Start']>TStart)&(Ont12['OnT_Start']<TStart+periodnum*Period2)]
                delT=ONT1['OnT'].sum()
        else:
            if obs == 1:
                delT=OnT1#*2*r
            if obs == 2:
                delT = OnT12#*2*r
    elif Detnumb == 2:
        df1 = pd.merge(df1,dfRESPAE, left_on='CH', right_on='CH2')
        df1.drop(columns = ['CH1', 'CH3','E1_AVG', 'E3_AVG', 'Resp1', 'Resp3'])
        df1['E'] = df1['E2_AVG']
        df1['RESP'] = df1['Resp2']
        if TStart >-1:
            if obs ==1:
                ONT2 = Ont2.loc[(Ont2['OnT_Start']>TStart)&(Ont2['OnT_Start']<TStart+periodnum*Period1)]
                delT=ONT2['OnT'].sum()
            if obs ==2:
                ONT2 = Ont22.loc[(Ont22['OnT_Start']>TStart)&(Ont22['OnT_Start']<TStart+periodnum*Period2)]
                delT=ONT2['OnT'].sum()
        else:
            if obs == 1:
                delT=OnT2#*2*r
            if obs == 2:
                delT = OnT22#*2*r
        
    elif Detnumb == 3:
        df1 = pd.merge(df1,dfRESPAE, left_on='CH', right_on='CH3')
        df1.drop(columns = ['CH1', 'CH2','E1_AVG', 'E2_AVG', 'Resp1', 'Resp2'])
        df1['E'] = df1['E3_AVG']
        df1['RESP'] = df1['Resp3']
        if TStart >-1:
            if obs ==1:
                ONT3 = Ont3.loc[(Ont3['OnT_Start']>TStart)&(Ont3['OnT_Start']<TStart+periodnum*Period1)]
                delT=ONT3['OnT'].sum()
            if obs ==2:
                ONT3 = Ont32.loc[(Ont32['OnT_Start']>TStart)&(Ont32['OnT_Start']<TStart+periodnum*Period2)]
                delT=ONT3['OnT'].sum()
        else:
            if obs == 1:
                delT=OnT3#*2*r
            if obs == 2:
                delT = OnT32#*2*r
                
    df1 = df1.loc[(df1['E']>E_low)&(df1['E']<E_high)]
    
    
    for i in np.arange(r, 2,r*2):
        df1_i = df1.loc[(df1['P']>=i-r)&(df1['P']<i+r)]
        C = (len(df1_i))
        P.append(i)
        Count.append(C)

    PC = pd.DataFrame(P, columns =['P'])
    Count = pd.DataFrame(Count, columns = ['Count'])
    PC['Count'] = Count
    PC['SigmaC'] = np.sqrt(Count)
    
    PC['COUNTT'] = PC['Count']/Ont12['OnT'].sum()

    PC['SigmaCT'] = PC['SigmaC']/Ont12['OnT'].sum()
    return PC
'''
def AngleBounds(Det):
    while Det['P'].max()>1:
        DetS = Det.loc[Det['P']>1]
        Det = Det.loc[Det['P']<1]
        DetS['P'] = DetS['P'] -1
        Det = pd.concat([Det,DetS])
    while Det['P'].min()<0:
        DetS = Det.loc[Det['P']<0]
        Det = Det.loc[Det['P']>0]
        DetS['P'] = DetS['P'] +1
        Det = pd.concat([Det,DetS])
    return Det


def applycorr(Det,parafit):
    Det['P'] = ((Det['P']*2*np.pi - correctionform(Det['T'],parafit))%(2*np.pi))/(2*np.pi)
    #Det = AngleBounds(Det)
    return Det


def correctionform(x,params):
    amp = params[0]
    period = params[1] * 86400
    phaseshift = params[2]
    vshift = params[3]
    
    return amp*np.sin(((2*np.pi/period)*x)+phaseshift) + vshift

def PulseP(parafit):
    
    
    Det1SS,Det1BS,Det2SS,Det2BS,Det3SS,Det3BS = Det12S.copy(),Det12B.copy(),Det22S.copy(),Det22B.copy(),Det32S.copy(),Det32B.copy()

    Det1SS = applycorr(Det1SS,parafit)
    Det1BS = applycorr(Det1BS,parafit)
    Det2SS = applycorr(Det2SS,parafit)
    Det2BS = applycorr(Det2BS,parafit)
    Det3SS = applycorr(Det3SS,parafit)
    Det3BS = applycorr(Det3BS,parafit)
    
    PC1,_,_,_,_,_,_ = PulseprofileBC(Det1SS,Det1BS,Det2SS,Det2BS,Det3SS,Det3BS,2,4)
    #PC1 = PULSEPROFILE(Det1SS, 1,2, 2,4,0,Tstart = -1)
    F_max = PC1['COUNTT'].max()
    F_min = PC1['COUNTT'].min()
    PF = (F_max-F_min)/(F_max+F_min)
    PF_1 = 1/PF
    return PF_1

parafit0 = np.array([2,1.7001,0.5,0.1])
parafit_min = np.array([0,-np.pi,0])
parafit_max = np.array([2*np.pi,np.pi,2*np.pi])

bnds = ((-2*np.pi,2*np.pi),(1.70,1.702),(-np.inf,np.inf),(0,np.inf))

res = opt.minimize(PulseP,parafit0,method='nelder-mead',bounds = bnds)
print(res.x)

Det1SS,Det1BS,Det2SS,Det2BS,Det3SS,Det3BS = Det12S.copy(),Det12B.copy(),Det22S.copy(),Det22B.copy(),Det32S.copy(),Det32B.copy()


Det1SS = applycorr(Det1SS,res.x)
Det1BS = applycorr(Det1BS,res.x)
Det2SS = applycorr(Det2SS,res.x)
Det2BS = applycorr(Det2BS,res.x)
Det3SS = applycorr(Det3SS,res.x)
Det3BS = applycorr(Det3BS,res.x)

def TimeCorrection(Det,res):
    Det['T'] = Det['T'] - Period2*correctionform(Det['T'],res) 
    return Det

Det1SS = TimeCorrection(Det1SS,res.x)
Det1BS = TimeCorrection(Det1BS,res.x)
Det2SS = TimeCorrection(Det2SS,res.x)
Det2BS = TimeCorrection(Det2BS,res.x)
Det3SS = TimeCorrection(Det3SS,res.x)
Det3BS = TimeCorrection(Det3BS,res.x)
    

PC1,_,_,_,_,_,_ = PulseprofileBC(Det1SS,Det1BS,Det2SS,Det2BS,Det3SS,Det3BS,2,4)
#PC1 = PULSEPROFILE(Det1SS, 1,2, 2,4,0,Tstart = -1)
F_max = PC1['COUNTT'].max()
F_min = PC1['COUNTT'].min()
PF = 100*(F_max-F_min)/(F_max+F_min)
print(PF)

plt.figure()
plt.plot(PC1['P'],PC1['COUNTT'],color='blue',label = 'obs2')
plt.ylabel('Normalized Count Rate [cts/s]')
plt.xlabel('Phase [pi radians]')
plt.ylim(0,PC1['COUNTT'].max())

plt.legend()
plt.show()

'''
plt.figure()
plt.plot(Det12S['P'],Det12S['P'],label = 'uncorrected')
plt.plot(Det12S['P'],Det1SS['P'],label = 'Corrected')
plt.xlabel('Uncorrected Phase')
plt.ylabel('New Phase Value')
plt.legend()
plt.show()
'''
plt.figure()
plt.scatter(Det12S['T'],Det12S['T'],label = 'uncorrected')
plt.scatter(Det12S['T'],Det1SS['T'],label = 'Corrected')
plt.xlabel('Uncorrected Time')
plt.ylabel('New Time Value')
#plt.xlim(0,100000)
#plt.ylim(0,100000)
plt.legend()
plt.show()

Det12S,Det12B,Det22S,Det22B,Det32S,Det32B=Det1SS,Det1BS,Det2SS,Det2BS,Det3SS,Det3BS


In [ ]:

Det1SS,Det1BS,Det2SS,Det2BS,Det3SS,Det3BS = Det13S.copy(),Det13B.copy(),Det23S.copy(),Det23B.copy(),Det33S.copy(),Det33B.copy()
TZ = 1.6*86400
def AngleBounds(Det):
    while Det['P'].max()>1:
        DetS = Det.loc[Det['P']>1]
        Det = Det.loc[Det['P']<1]
        DetS['P'] = DetS['P'] -1
        Det = pd.concat([Det,DetS])
    while Det['P'].min()<0:
        DetS = Det.loc[Det['P']<0]
        Det = Det.loc[Det['P']>0]
        DetS['P'] = DetS['P'] +1
        Det = pd.concat([Det,DetS])
    return Det


def applycorr(Det,parafit):
    Det['P'] = ((Det['P']*2*np.pi - correctionform(Det['T'],parafit))%(2*np.pi))/(2*np.pi)
    #Det = AngleBounds(Det)
    return Det


def correctionform(x,params):
    amp = params[0]
    period = params[1] * 86400
    phaseshift = params[2]
    #vshift = params[4]
    amp2 = params[3]
    period2 = params[4] * 86400
    phaseshift2 = params[5]

    
    return amp*np.sin(((2*np.pi/period)*x)+phaseshift) + amp2*np.sin(((2*np.pi/period2)*x)+phaseshift2)

def PulseP(parafit):
    
    
    Det1SS,Det1BS,Det2SS,Det2BS,Det3SS,Det3BS = Det13S.copy(),Det13B.copy(),Det23S.copy(),Det23B.copy(),Det33S.copy(),Det33B.copy()

    Det1SS = applycorr(Det1SS,parafit)
    Det1BS = applycorr(Det1BS,parafit)
    Det2SS = applycorr(Det2SS,parafit)
    Det2BS = applycorr(Det2BS,parafit)
    Det3SS = applycorr(Det3SS,parafit)
    Det3BS = applycorr(Det3BS,parafit)
    
    PC1,_,_,_,_,_,_ = PulseprofileBC(Det1SS,Det1BS,Det2SS,Det2BS,Det3SS,Det3BS,2,4,Tstart = TZ,PNum =(Det1SS['T'].max()-TZ)/Period3,obs=3)
    #PC1 = PULSEPROFILE(Det1SS, 1,2, 2,4,0,Tstart = -1)
    F_max = PC1['COUNTT'].max()
    F_min = PC1['COUNTT'].min()
    PF = (F_max-F_min)/(F_max+F_min)
    PF_1 = 1/PF
    return PF_1

parafit0 = np.array([7.08961281e-01,1.701, 2.48613245e-01 ,1e-2,34.9,0])
parafit_min = np.array([0,-np.pi,0])
parafit_max = np.array([2*np.pi,np.pi,2*np.pi])

bnds = ((-2*np.pi,2*np.pi),(1.70,1.703),(-np.inf,np.inf),(0,2*np.pi),(34.5,35),(0,2*np.pi))

res = opt.minimize(PulseP,parafit0,method='nelder-mead',bounds = bnds)
print(res.x)

Det1SS,Det1BS,Det2SS,Det2BS,Det3SS,Det3BS = Det13S.copy(),Det13B.copy(),Det23S.copy(),Det23B.copy(),Det33S.copy(),Det33B.copy()


Det1SS = applycorr(Det1SS,res.x)
Det1BS = applycorr(Det1BS,res.x)
Det2SS = applycorr(Det2SS,res.x)
Det2BS = applycorr(Det2BS,res.x)
Det3SS = applycorr(Det3SS,res.x)
Det3BS = applycorr(Det3BS,res.x)

def TimeCorrection(Det,res):
    Det['T'] = Det['T'] - Period3*correctionform(Det['T'],res) 
    return Det

Det1SS = TimeCorrection(Det1SS,res.x)
Det1BS = TimeCorrection(Det1BS,res.x)
Det2SS = TimeCorrection(Det2SS,res.x)
Det2BS = TimeCorrection(Det2BS,res.x)
Det3SS = TimeCorrection(Det3SS,res.x)
Det3BS = TimeCorrection(Det3BS,res.x)
    

PC1,_,_,_,_,_,_ = PulseprofileBC(Det1SS,Det1BS,Det2SS,Det2BS,Det3SS,Det3BS,2,4,Tstart = TZ,PNum =(Det1SS['T'].max()-TZ)/Period3,obs=3)
#PC1 = PULSEPROFILE(Det1SS, 1,2, 2,4,0,Tstart = -1)
F_max = PC1['COUNTT'].max()
F_min = PC1['COUNTT'].min()
PF = 100*(F_max-F_min)/(F_max+F_min)
print(PF)

plt.figure()
plt.plot(PC1['P'],PC1['COUNTT'],color='blue',label = 'obs2')
plt.ylabel('Normalized Count Rate [cts/s]')
plt.xlabel('Phase [pi radians]')
plt.ylim(0,PC1['COUNTT'].max())
obs=2
plt.legend()
plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/PulseProfileShiftsAfterCorrection{obs}.jpeg')
plt.show()

'''
plt.figure()
plt.plot(Det12S['P'],Det12S['P'],label = 'uncorrected')
plt.plot(Det12S['P'],Det1SS['P'],label = 'Corrected')
plt.xlabel('Uncorrected Phase')
plt.ylabel('New Phase Value')
plt.legend()
plt.show()
'''
plt.figure()
plt.scatter(Det13S['T'],Det13S['T'],label = 'uncorrected')
plt.scatter(Det13S['T'],Det1SS['T'],label = 'Corrected')
plt.xlabel('Uncorrected Time')
plt.ylabel('New Time Value')
#plt.xlim(0,100000)
#plt.ylim(0,100000)
plt.legend()
plt.show()


In [ ]:
'''PC1 = PulseprofileBC(Det1S,Det1B,Det2S,Det2B,Det3S,Det3B,1,2,4)
PC2 = PulseprofileBC(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,2,2,4)

df = pd.read_csv('/home/jacob/Desktop/ForsbloomPulse2.csv', sep = ',',header = None,dtype = np.float64)

df['Phase'] = df.loc[:,0]
df['Phase'] = df['Phase']- df['Phase'].min()
df['Phase'] = df['Phase']*2/df['Phase'].max()
df['Count'] = df.loc[:,1]

df = pd.DataFrame(df,columns = ['Phase','Count'])

df['Count'] = df['Count']*(PC1['COUNTT'].max())/df['Count'].max()

PC1P = PC1.loc[(PC1['P'].between(0,1))&(PC1['COUNTT']==PC1['COUNTT'].max())]
PC1P = float(PC1P['P'].mean())
dfd = df.loc[(df['Phase'].between(0,1))&(df['Count']==df['Count'].max())]
dfd = float(dfd['Phase'].mean())


shift = 0.92

Det1S['P'] = Det1S['P'] -(PC1P - dfd)-1
Det = Det1S.loc[(Det1S['P']<0)]
Det['P'] = Det['P'] + 1
Det1S = pd.concat([Det1S,Det])

Det1B['P'] = Det1B['P'] -(PC1P - dfd)-1
Det = Det1B.loc[(Det1B['P']<0)]
Det['P'] = Det['P'] + 1
Det1B = pd.concat([Det1B,Det])

Det2S['P'] = Det2S['P'] -(PC1P - dfd)-1
Det = Det2S.loc[(Det2S['P']<0)]
Det['P'] = Det['P'] + 1
Det2S = pd.concat([Det2S,Det])

Det2B['P'] = Det2B['P'] -(PC1P - dfd)-1
Det = Det2B.loc[(Det2B['P']<0)]
Det['P'] = Det['P'] + 1
Det2B = pd.concat([Det2B,Det])


Det3S['P'] = Det3S['P'] -(PC1P - dfd)-1
Det = Det3S.loc[(Det3S['P']<0)]
Det['P'] = Det['P'] + 1
Det3S = pd.concat([Det3S,Det])

Det3B['P'] = Det3B['P'] -(PC1P - dfd)-1
Det = Det3B.loc[(Det3B['P']<0)]
Det['P'] = Det['P'] + 1
Det3B = pd.concat([Det3B,Det])

PC1 = PulseprofileBC(Det1S,Det1B,Det2S,Det2B,Det3S,Det3B,1,2,4)
PC2 = PulseprofileBC(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,2,2,4)

PC1P = PC1.loc[(PC1['P'].between(0,1))&(PC1['COUNTT']==PC1['COUNTT'].max())]
PC1P = float(PC1P['P'].mean())
PC2P = PC2.loc[(PC2['P'].between(0,1))&(PC2['COUNTT']==PC2['COUNTT'].max())]
PC2P = float(PC2P['P'].mean())


Det12S['P'] = Det12S['P'] -(PC2P - PC1P)
Det = Det12S.loc[(Det12S['P']<0)]
Det['P'] = Det['P'] + 1
Det12S = pd.concat([Det12S,Det])

Det12B['P'] = Det12B['P'] -(PC2P - PC1P)
Det = Det12B.loc[(Det12B['P']<0)]
Det['P'] = Det['P'] + 1
Det12B = pd.concat([Det12B,Det])

Det22S['P'] = Det22S['P'] -(PC2P - PC1P)
Det = Det22S.loc[(Det22S['P']<0)]
Det['P'] = Det['P'] + 1
Det22S = pd.concat([Det22S,Det])

Det22B['P'] = Det22B['P'] -(PC2P - PC1P)
Det = Det22B.loc[(Det22B['P']<0)]
Det['P'] = Det['P'] + 1
Det22B = pd.concat([Det22B,Det])


Det32S['P'] = Det32S['P'] -(PC2P - PC1P)
Det = Det32S.loc[(Det32S['P']<0)]
Det['P'] = Det['P'] + 1
Det32S = pd.concat([Det32S,Det])

Det32B['P'] = Det32B['P'] -(PC2P - PC1P)
Det = Det32B.loc[(Det32B['P']<0)]
Det['P'] = Det['P'] + 1
Det32B = pd.concat([Det32B,Det])'''

PC1,_,_,_,_,_,_ = PulseprofileBC(Det1S,Det1B,Det2S,Det2B,Det3S,Det3B,1,2,4)
PC2,_,_,_,_,_,_ = PulseprofileBC(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,2,2,4)



print('Computing Fit Observation1')

'''params, params_covariance = opt.curve_fit(Fourier6thorder,PC1['P'].T,PC1['COUNTT'].T,
                                          maxfev=10000000, 
                                          absolute_sigma = True)
params2, params_covariance2 = opt.curve_fit(Fourier6thorder,PC2['P'].T,PC2['COUNTT'].T,
                                          maxfev=10000000, 
                                          absolute_sigma = True)'''

plt.figure()
plt.plot(PC1['P'],PC1['COUNTT'],color='blue',label = 'obs1')
plt.plot(PC2['P'],PC2['COUNTT'],color='orange',label = 'obs2')
#plt.ylim(3.5,4.5)
plt.ylabel('Count Rate [cts/s]')
plt.xlabel('Phase [pi radians]')
plt.legend()
plt.show()
#print('Computing Fit Observation2')

'''P = [0,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.65,0.7,0.775,0.875,1]
plt.figure()
a = 0.25
plt.plot(df['Phase'],df['Count'],color = 'green', label = 'Forsbloom Obs2')
plt.plot(PC1['P'],Fourier6thorder(PC1['P'],params[0],params[1],params[2],params[3],params[4],params[5],params[6],params[7],params[8],params[9],params[10],params[11],params[12]),label = 'Fourier Obs1')
plt.plot(PC2['P'],Fourier6thorder(PC2['P'],params2[0],params2[1],params2[2],params2[3],params2[4],params2[5],params2[6],params2[7],params2[8],params2[9],params2[10],params2[11],params2[12]),label = 'Fourier Obs2')
plt.axvspan(0,0.1, color = 'purple', alpha =a)
plt.axvspan(0.1,0.15, color = 'pink', alpha =a)
plt.axvspan(0.15,0.2, color = 'red', alpha =a)
plt.axvspan(0.2,0.3, color = 'green', alpha =a)
plt.axvspan(0.3,0.4, color = 'blue', alpha =a)
plt.axvspan(0.4,0.5, color = 'yellow', alpha =a)
plt.axvspan(0.5,0.6, color = 'purple', alpha =a)
plt.axvspan(0.65,0.7, color = 'pink', alpha =a)
plt.axvspan(0.7,0.775, color = 'red', alpha =a)
plt.axvspan(0.775,0.875, color = 'green', alpha =a)
plt.axvspan(0.875,1, color = 'blue', alpha =a)
plt.ylabel('Count Rate [cts/s]')
plt.xlabel('Phase [pi radians]')
plt.legend()
plt.show()

P = np.linspace(0,2,20000)
Fourier = pd.DataFrame(P,columns=['Phase'])
Fourier['CountR1'] = Fourier6thorder(Fourier['Phase'],params[0],params[1],params[2],params[3],params[4],params[5],params[6],params[7],params[8],params[9],params[10],params[11],params[12])
Fourier['CountR2'] = Fourier6thorder(Fourier['Phase'],params2[0],params2[1],params2[2],params2[3],params2[4],params2[5],params2[6],params2[7],params2[8],params2[9],params2[10],params2[11],params2[12])


F_max = Fourier['CountR1'].max()
F_min = Fourier['CountR1'].min()
PF = 100*(F_max-F_min)/(F_max+F_min)
print('Fourier Transform Obs1:',PF)

F_max = Fourier['CountR2'].max()
F_min = Fourier['CountR2'].min()
PF = 100*(F_max-F_min)/(F_max+F_min)
print('Fourier Transform Obs2:',PF)
'''
F_max = PC1['COUNTT'].max()
F_min = PC1['COUNTT'].min()
PF = 100*(F_max-F_min)/(F_max+F_min)
print('Data Set 1:',PF)

F_max = PC2['COUNTT'].max()
F_min = PC2['COUNTT'].min()
PF = 100*(F_max-F_min)/(F_max+F_min)
print('Data Set 2:',PF)
'''

Det1S = pd.concat([Det1S , Det12S])  
Det1B = pd.concat([Det1B , Det12B]) 
Det2S = pd.concat([Det2S , Det22S]) 
Det2B = pd.concat([Det2B , Det22B]) 
Det3S = pd.concat([Det3S , Det32S]) 
Det3B = pd.concat([Det3B , Det32B]) ''';

In [ ]:
Periodnum = 0
det1StokesS = normalize_Stokes(Det12S,1,2,8,PNum = Periodnum)
det2StokesS = normalize_Stokes(Det22S,2,2,8,PNum = Periodnum)
det3StokesS = normalize_Stokes(Det32S,3,2,8,PNum = Periodnum)
det1Stokesbkg = normalize_Stokes(Det12B,1,2,8,PNum = Periodnum)
det2Stokesbkg = normalize_Stokes(Det22B,2,2,8,PNum = Periodnum)
det3Stokesbkg = normalize_Stokes(Det32B,3,2,8,PNum = Periodnum)

det1Stokes= (det1StokesS/(3.14159*((26.923)**2)) - 
                       det1Stokesbkg /(3.14159*(((107.69)**2)-
                                (53.846)**2)))*(3.14159*((26.923)**2))
det2Stokes= (det2StokesS/(3.14159*((26.923)**2)) - 
                       det2Stokesbkg /(3.14159*(((107.69)**2)-
                                (53.846)**2)))*(3.14159*((26.923)**2))
det3Stokes= (det3StokesS/(3.14159*((26.923)**2)) - 
                       det3Stokesbkg /(3.14159*(((107.69)**2)-
                                (53.846)**2)))*(3.14159*((26.923)**2))
det1Stokes = det1StokesS
det2Stokes = det2StokesS
det3Stokes = det3StokesS


Stokes = det1Stokes+det2Stokes+det3Stokes
print(np.arange(5,9,1))
for i in np.arange(5,10,1):
    Stokes[i] = np.sqrt(det1StokesS[i]**2+det2StokesS[i]**2+det3StokesS[i]**2)
Stokes[3] = float(np.sqrt([(Stokes[1]**2)+(Stokes[2]**2)]))
Stokes[4]= float((0.5)*np.arctan2(Stokes[2], Stokes[1]))
    

Cont1 = ErrorCont(Stokes,1,80)
Cont2 = ErrorCont(Stokes,2,80)
Cont3 = ErrorCont(Stokes,3,80)


fig, ax = plt.subplots(subplot_kw={'projection':'polar'})
ax.set_theta_zero_location("N")
ax.scatter(Stokes[4],Stokes[3],s=1)
ax.plot(Cont1['PA'],Cont1['PD'])
ax.plot(Cont2['PA'],Cont2['PD'])
ax.plot(Cont3['PA'],Cont3['PD'])
ax.arrow(0,0,Stokes[4],Stokes[3])
#ax.errorbar(Stokes[4],Stokes[3], yerr = Stokes[7], xerr = Stokes[8], capsize =0)
ax.set_thetamin(-90)
ax.set_thetamax(90)
ax.set_rmax(12.5)
ax.set_rmin(0)
ax.set_rticks([0,2.5,5,7.5,10,12.5])
plt.show()

In [ ]:
def PolarPlot(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,PStart,PEnd,*args, **kwargs):
    periodnum = kwargs.get('PNum',0)
    PStart = float(PStart)
    det1Stokes = normalize_Stokes(Det12S,1,2,8,PS= PStart,PE= PEnd,PNum = periodnum)
    det2Stokes = normalize_Stokes(Det22S,2,2,8,PS= PStart,PE= PEnd,PNum = periodnum)
    det3Stokes = normalize_Stokes(Det32S,3,2,8,PS= PStart,PE= PEnd,PNum = periodnum)
    
    det1Stokesbkg = normalize_Stokes(Det12B,1,2,8,PS= PStart,PE= PEnd,PNum = periodnum)
    det2Stokesbkg = normalize_Stokes(Det22B,2,2,8,PS= PStart,PE= PEnd,PNum = periodnum)
    det3Stokesbkg = normalize_Stokes(Det32B,3,2,8,PS= PStart,PE= PEnd,PNum = periodnum)

    det1Stokes= (det1Stokes/(3.14159*((26.923)**2)) - 
                           det1Stokesbkg /(3.14159*(((107.69)**2)-
                                    (53.846)**2)))*(3.14159*((26.923)**2))
    det2Stokes= (det2Stokes/(3.14159*((26.923)**2)) - 
                           det2Stokesbkg /(3.14159*(((107.69)**2)-
                                    (53.846)**2)))*(3.14159*((26.923)**2))
    det3Stokes= (det3Stokes/(3.14159*((26.923)**2)) - 
                           det3Stokesbkg /(3.14159*(((107.69)**2)-
                                    (53.846)**2)))*(3.14159*((26.923)**2))


    Stokes = det1Stokes+det2Stokes+det3Stokes
    for i in np.arange(5,10,1):
        Stokes[i] = np.sqrt(det1StokesS[i]**2+det2StokesS[i]**2+det3StokesS[i]**2)
    Stokes[3] = float(np.sqrt([(Stokes[1]**2)+(Stokes[2]**2)]))
    Stokes[4]= float((0.5)*np.arctan2(Stokes[2], Stokes[1]))
    
    Cont1 = ErrorCont(Stokes,2,150)
    Cont2 = ErrorCont(Stokes,4,150)
    Cont3 = ErrorCont(Stokes,6,150)
    
    return Stokes, Cont1,Cont2,Cont3


In [ ]:
obs = 2
P = [0,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.775,0.875,0.925,1]
def PhaseResolvedPolarizationVectors(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,obs,B,*arg,**kwargs):
    Rmax = kwargs.get('Rmax',20)
    Color = ['red','green','blue','pink']

    PC2,_,_,_,_,_,_ = PulseprofileBC(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,obs,2,4)
    plt.figure()
    plt.plot(PC2['P'],PC2['COUNTT'],color='blue',label = 'obs2')
    for i in np.arange(0,len(P)-1,1):
        plt.axvspan(P[i],P[i+1],alpha=0.25,color = Color[int(i%4)])
    plt.ylabel('Count Rate [cts/s]')
    plt.xlabel('Phase [pi radians]')
    plt.ylim(0,1)
    plt.legend()
    plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/PolarizationPhaseBinsobs{obs}.jpeg')
    plt.show()


    numofcolumns = 3
    STOKES = []
    fig, axs = plt.subplots(int(np.ceil((len(P)-1)/numofcolumns)),numofcolumns,subplot_kw={'projection':'polar'},figsize=(10,10))
    for i in range (0,len(P)-1):
        S,Cont1,Cont2,Cont3 = PolarPlot(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,B[i],B[i+1])

        axs[int(np.floor(i/numofcolumns)),i%numofcolumns].set_title('Phase='+str(P[i])+' - '+str(P[i+1]))
        axs[int(np.floor(i/numofcolumns)),i%numofcolumns].scatter(S[4],S[3],s=1)
        axs[int(np.floor(i/numofcolumns)),i%numofcolumns].arrow(0,0,S[4],S[3])
        axs[int(np.floor(i/numofcolumns)),i%numofcolumns].set_theta_zero_location("N")
        axs[int(np.floor(i/numofcolumns)),i%numofcolumns].plot(Cont1['PA'],Cont1['PD'])
        axs[int(np.floor(i/numofcolumns)),i%numofcolumns].plot(Cont2['PA'],Cont2['PD'])
        axs[int(np.floor(i/numofcolumns)),i%numofcolumns].plot(Cont3['PA'],Cont3['PD'])
        axs[int(np.floor(i/numofcolumns)),i%numofcolumns].set_thetamin(-90)
        axs[int(np.floor(i/numofcolumns)),i%numofcolumns].set_thetamax(90)
        axs[int(np.floor(i/numofcolumns)),i%numofcolumns].set_rmax(Rmax)
        axs[int(np.floor(i/numofcolumns)),i%numofcolumns].set_rmin(0)
        axs[int(np.floor(i/numofcolumns)),i%numofcolumns].set_rticks([0,10,20])
    plt.tight_layout(pad = 1)
    plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/PolarizationVectorsobs{obs}.jpeg')
    plt.show()
    plt.close()

    
obs=1
P = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
PhaseResolvedPolarizationVectors(Det1S,Det1B,Det2S,Det2B,Det3S,Det3B,obs,P,Rmax = 30)
    
obs=2
P = [0,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.775,0.875,0.925,1]
PhaseResolvedPolarizationVectors(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,obs,P)





In [ ]:
def EnergyDep(EStart,Eend):
    PStart,PEnd = 0,1
    det1StokesS = normalize_Stokes(Det1S,1,EStart,Eend,PS= PStart,PE= PEnd)
    det2StokesS = normalize_Stokes(Det2S,2,EStart,Eend,PS= PStart,PE= PEnd)
    det3StokesS = normalize_Stokes(Det3S,3,EStart,Eend,PS= PStart,PE= PEnd)
    det1Stokesbkg = normalize_Stokes(Det1B,1,EStart,Eend,PS= PStart,PE= PEnd)
    det2Stokesbkg = normalize_Stokes(Det2B,2,EStart,Eend,PS= PStart,PE= PEnd)
    det3Stokesbkg = normalize_Stokes(Det3B,3,EStart,Eend,PS= PStart,PE= PEnd)

    det1Stokes= (det1StokesS/(3.14159*((26.923)**2)) - 
                           det1Stokesbkg /(3.14159*(((107.69)**2)-
                                    (53.846)**2)))*(3.14159*((26.923)**2))
    det2Stokes= (det2StokesS/(3.14159*((26.923)**2)) - 
                           det2Stokesbkg /(3.14159*(((107.69)**2)-
                                    (53.846)**2)))*(3.14159*((26.923)**2))
    det3Stokes= (det3StokesS/(3.14159*((26.923)**2)) - 
                           det3Stokesbkg /(3.14159*(((107.69)**2)-
                                    (53.846)**2)))*(3.14159*((26.923)**2))
    Stokes = det1Stokes+det2Stokes+det3Stokes
    for i in np.arange(5,8,1):
        Stokes[i] = np.sqrt(det1StokesS[i]**2+det2StokesS[i]**2+det3StokesS[i]**2)
    Stokes[9] = det1StokesS[9]
    Stokes[10] = det1StokesS[10]
    Stokes[4] = Stokes[4]*np.pi/180
    
    return Stokes

In [ ]:
P = [0,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.775,0.875,0.925,1]
def QUPlot(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,obs,P):
    STOKES = pd.DataFrame(columns=['CENTERPHASE','I','q','u','PD','PA','qstd','ustd','PDstd','PAstd','E_low','E_high'])

    for i in np.arange(0,len(P)-1,1):
        Det12SP = Det12S.loc[(Det12S['P'].between(P[i],P[i+1]))]
        Det22SP = Det22S.loc[(Det22S['P'].between(P[i],P[i+1]))]
        Det32SP = Det32S.loc[(Det32S['P'].between(P[i],P[i+1]))]
        PH = np.array([(P[i+1]+P[i])/2])
        S1 = normalize_Stokes(Det12SP,1,2,8)
        S2 = normalize_Stokes(Det22SP,2,2,8)
        S3 = normalize_Stokes(Det32SP,3,2,8)
        S = S1+S2+S3
        S = np.append(PH,S)
        S = np.array([S])
        #print(S)
        S = pd.DataFrame(S,columns =['CENTERPHASE','I','q','u','PD','PA','qstd','ustd','PDstd','PAstd','E_low','E_high'] )
        STOKES = pd.concat([STOKES,S])
        #Det12SP['u'] = Det12SP['U']/i
        #Det12SP['q'] = Det12SP['Q']/i
        #u = Det12SP['u'].sum()
        #q = Det12SP['q'].sum()
        #ustd = Det12SP['u'].std()
        #qstd = Det12SP['q'].std()
    STOKES['q'] = STOKES['q']/100
    STOKES['u'] = STOKES['u']/100
    STOKES['ustd']=STOKES['ustd']/100
    STOKES['qstd']=STOKES['qstd']/100
    
    fig,ax = plt.subplots()
    circle = plt.Circle((0,0),0.1,fill = False,color = 'red',label ='10% Polarization')
    ax.add_patch(circle) 
    circle = plt.Circle((0,0),0.2,fill = False,color = 'green',label ='20% Polarization')
    ax.add_patch(circle) 


    t = np.linspace(0,2*np.pi,len(STOKES))
    t = 5*np.pi*t
    ax.errorbar(STOKES['q'],STOKES['u'],yerr = STOKES['ustd'],xerr=STOKES['qstd'],markersize=4,ls='none',zorder=1)
    ax.scatter(STOKES['q'],STOKES['u'],c = t,zorder=2)
    ax.hlines(0,-1,1,color = 'grey')
    ax.vlines(0,-1,1,color='grey')


    ax.set_xlabel('Q/I')
    ax.set_ylabel('U/I')
    ax.set_title(f'Observation {obs} U/I vs Q/I')
    ax.set_xlim(-0.3,0.3)
    ax.set_ylim(-0.3,0.3)
    plt.legend()
    plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/UvsQplot_obs{obs}.jpeg')
    plt.show()
    plt.close()



    STOKES['P'] = STOKES['CENTERPHASE']
    PC2,_,_,_,_,_,_ = PulseprofileBC(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,obs,2,4)
    COUNT = np.empty(0)
    for P in STOKES['P']:
        COUNTT = PC2['COUNTT'].loc[PC2['P'].between(P-0.05,P+0.05)].mean()
        COUNT = np.append(COUNT,COUNTT)
    STOKES['COUNTT'] = COUNT
    plt.figure()
    plt.title(f'Observation {obs} PulseProfile')
    plt.plot(PC2['P'],PC2['COUNTT'],color='blue',label = 'obs2',zorder=1)
    #for i in np.arange(0,len(P)-1,1):
    #    plt.axvspan(P[i],P[i+1],alpha=0.25,color = Color[int(i%4)])
    plt.scatter(STOKES['CENTERPHASE'],STOKES['COUNTT'],c=t,zorder=2)
    plt.ylabel('Count Rate [cts/s]')
    plt.xlabel('Phase [revolutions]')
    plt.ylim(0,1)
    plt.legend()
    plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/PointsPolarizationPhaseBinsobs{obs}.jpeg')
    plt.show()



    
P = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
obs=1
QUPlot(Det1S,Det1B,Det2S,Det2B,Det3S,Det3B,obs,P)


P = [0,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.775,0.875,0.925,1]
obs=2
QUPlot(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,obs,P)

In [ ]:
def ErrorSumation(Errorvalues):
    Errorvalues = Errorvalues**2
    Errorvalue = Errorvalues.sum()
    return np.sqrt(Errorvalue)

def PhaseRestrictions(Det,Pmin,Pmax):
    Det = Det.loc[Det['P'].between(Pmin,Pmax)]
    return Det

In [ ]:
def SpectraAnalysis(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,obs,*arg,**kwargs):
    Pmax = kwargs.get('Pmax',1)
    Pmin = kwargs.get('Pmin',0)
    
    Det12S = PhaseRestrictions(Det12S,Pmin,Pmax)
    Det12B = PhaseRestrictions(Det12B,Pmin,Pmax)
    Det22S = PhaseRestrictions(Det22S,Pmin,Pmax)
    Det22B = PhaseRestrictions(Det22B,Pmin,Pmax)
    Det32S = PhaseRestrictions(Det32S,Pmin,Pmax)
    Det32B = PhaseRestrictions(Det32B,Pmin,Pmax)
    
    
    ChR1S = CHCount(Det12S,1,obs) 
    ChR1bkg = CHCount(Det12B,1,obs)

    ChR2S = CHCount(Det22S,2,obs) 
    ChR2bkg = CHCount(Det22B,2,obs)

    ChR3S = CHCount(Det32S,3,obs) 
    ChR3bkg = CHCount(Det32B,3,obs)


    CH1S = ChR1S[:,0]
    Ra1IS = ChR1S[:,2]
    CH2S = ChR2S[:,0]
    Ra2IS = ChR2S[:,2]
    CH3S = ChR3S[:,0]
    Ra3IS = ChR3S[:,2]

    SPECSI = ARFRMF(CH1S,Ra1IS,CH2S,Ra2IS,CH3S,Ra3IS)




    SPECSI = SPECSI.loc[(SPECSI['AVG_E']>2) & (SPECSI['AVG_E']<7)]
    CH1 = ChR1bkg[:,0]
    Ra1I = ChR1bkg[:,2]
    CH2 = ChR2bkg[:,0]
    Ra2I = ChR2bkg[:,2]
    CH3 = ChR3bkg[:,0]
    Ra3I = ChR3bkg[:,2]

    SPECSbkg = ARFRMF(CH1,Ra1I,CH2,Ra2I,CH3,Ra3I)

    #SPECSI = SPECBackgroundSubtraction(SPECSI,SPECSbkg,26.923,53.846,107.693)


    #ChR[:,3] is the u, non-normalized stokes parameter = 2sin(2*psi)
    Ra1U = ChR1S[:,2]*ChR1S[:,3]/ChR1S[:,5]
    Ra2U = ChR2S[:,2]*ChR2S[:,3]/ChR2S[:,5]
    Ra3U = ChR3S[:,2]*ChR3S[:,3]/ChR3S[:,5]

    SPECSU = ARFRMF(CH1S,Ra1U,CH2S,Ra2U,CH3S,Ra3U)

    SPECSU['U1STD'] = ChR1S[:,2]*ChR1S[:,6]/(ChR1S[:,5]*0.04)
    SPECSU['U2STD'] = ChR2S[:,2]*ChR2S[:,6]/(ChR2S[:,5]*0.04)
    SPECSU['U3STD'] = ChR3S[:,2]*ChR3S[:,6]/(ChR3S[:,5]*0.04)


    Ra1U = ChR1bkg[:,2]*ChR1bkg[:,3]/(ChR1S[:,5])
    Ra2U = ChR2bkg[:,2]*ChR2bkg[:,3]/(ChR1S[:,5])
    Ra3U = ChR3bkg[:,2]*ChR3bkg[:,3]/(ChR1S[:,5])

    SPECSUbkg = ARFRMF(CH1,Ra1U,CH2,Ra2U,CH3,Ra3U)

    #SPECSU = SPECBackgroundSubtraction(SPECSU,SPECSUbkg,26.923,53.846,107.693)

    SPECSU = SPECSU.loc[(SPECSU['AVG_E']>2) & (SPECSU['AVG_E']<7)]

    bincount = 9
    binsize = len(SPECSU)

    SPECSU1 = SPECSU.loc[:,SPECSU.columns.intersection(['U1STD','U2STD','U3STD'])]
    SPECSU = SPECSU.drop(columns=['U1STD','U2STD','U3STD'])

    SPECSU = SPECSU.groupby(np.arange(binsize)//bincount).mean()
    SPECSU['U1STD'] = SPECSU1['U1STD'].groupby(np.arange(binsize)//bincount).agg(ErrorSumation)
    SPECSU['U2STD'] = SPECSU1['U2STD'].groupby(np.arange(binsize)//bincount).agg(ErrorSumation)
    SPECSU['U3STD'] = SPECSU1['U3STD'].groupby(np.arange(binsize)//bincount).agg(ErrorSumation)


    #ChR[:,4] is the q, non-normalized stokes parameter = 2cos(2*psi)
    Ra1Q = ChR1S[:,2]*ChR1S[:,4]/ChR1S[:,5]
    Ra2Q = ChR2S[:,2]*ChR2S[:,4]/ChR2S[:,5]
    Ra3Q = ChR3S[:,2]*ChR3S[:,4]/ChR3S[:,5]

    SPECSQ = ARFRMF(CH1S,Ra1Q,CH2S,Ra2Q,CH3S,Ra3Q)
    SPECSQ['Q1STD'] = ChR1S[:,2]*ChR1S[:,7]/(ChR1S[:,5]*0.04)
    SPECSQ['Q2STD'] = ChR2S[:,2]*ChR2S[:,7]/(ChR2S[:,5]*0.04)
    SPECSQ['Q3STD'] = ChR3S[:,2]*ChR3S[:,7]/(ChR3S[:,5]*0.04)

    Ra1Q = ChR1bkg[:,2]*ChR1bkg[:,4]
    Ra2Q = ChR2bkg[:,2]*ChR2bkg[:,4]
    Ra3Q = ChR3bkg[:,2]*ChR3bkg[:,4]

    SPECSQbkg = ARFRMF(CH1,Ra1Q,CH2,Ra2Q,CH3,Ra3Q)

    #SPECSQ = SPECBackgroundSubtraction(SPECSQ,SPECSQbkg,26.923,53.846,107.693)
    SPECSQ= SPECSQ.loc[(SPECSQ['AVG_E']>2) & (SPECSQ['AVG_E']<7)]


    #bincount = 9
    binsize = len(SPECSQ)

    SPECSQ1 = SPECSQ.loc[:,SPECSQ.columns.intersection(['Q1STD','Q2STD','Q3STD'])]
    SPECSQ = SPECSQ.drop(columns=['Q1STD','Q2STD','Q3STD'])

    SPECSQ = SPECSQ.groupby(np.arange(binsize)//bincount).mean()
    SPECSQ['Q1STD'] = SPECSQ1['Q1STD'].groupby(np.arange(binsize)//bincount).agg(ErrorSumation)
    SPECSQ['Q2STD'] = SPECSQ1['Q2STD'].groupby(np.arange(binsize)//bincount).agg(ErrorSumation)
    SPECSQ['Q3STD'] = SPECSQ1['Q3STD'].groupby(np.arange(binsize)//bincount).agg(ErrorSumation)


    SPECSQstd = SPECSQ['RESP*Ra/delE'].std()
    
    return SPECSI,SPECSU,SPECSQ

SPECSI1,SPECSU1,SPECSQ1 = SpectraAnalysis(Det1S,Det1B,Det2S,Det2B,Det3S,Det3B,1)
SPECSI2,SPECSU2,SPECSQ2 = SpectraAnalysis(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,2)
SPECSI3,SPECSU3,SPECSQ3 = SpectraAnalysis(Det13S,Det13B,Det23S,Det23B,Det33S,Det33B,3)


In [ ]:
def SpectraPlots(SPECSI,SPECSU,SPECSQ,obs):
    plt.figure()
    #plt.plot(SPECSI['AVG_E'],SPECSI['RESP*Ra/delE'], label = 'Summed Spectrum', zorder=4)
    #SPECSI.plot(x='AVG_E',y=['RESP*Ra1/delE1',
     #                   'RESP*Ra2/delE2','RESP*Ra3/delE3'] ,
      #          label = ['Det1', 'Det2','Det3'], zorder=4)
    plt.plot(SPECSI['E1_AVG'],SPECSI['RESP*Ra1/delE1'], label = 'Det1', color = 'red')
    plt.plot(SPECSI['E2_AVG'],SPECSI['RESP*Ra2/delE2'], label = 'Det2', color = 'green')
    plt.plot(SPECSI['E3_AVG'],SPECSI['RESP*Ra3/delE3'], label = 'Det3', color = 'orange')
    plt.xlim(2,7)
    plt.ylim(0.005,3)
    plt.yscale('log')
    plt.axvspan(2,4,label = '2-4EV', color = 'purple', alpha =0.25)
    plt.axvspan(4,7,label = '4-8EV', color = 'pink', alpha =0.5)
    plt.title('Spectrum Python, Stokes Parameter I Observation ' + str(obs))
    plt.xlabel('Energy [KeV]')
    plt.ylabel('Rate [cst/s/KeV]')
    plt.text(3.4,0.01, '2-4KeV', verticalalignment = 'bottom', horizontalalignment='left')
    plt.text(4.1,0.01, '4-7KeV', verticalalignment = 'bottom', horizontalalignment='left')
    plt.tick_params('both',labelright = True,right=True,direction ='in',which = 'minor')
    plt.legend()
    plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/SPECSIObs{obs}.jpeg')
    plt.show()
    plt.close()

    plt.figure()
    s=4 #size of the scatter plot points
    plt.errorbar(SPECSU['AVG_E'],SPECSU['RESP*Ra1/delE1'],yerr = SPECSU['U1STD'], xerr=0,fmt='o',markersize = s,color = 'red',label = 'Det1')
    plt.errorbar(SPECSU['AVG_E'],SPECSU['RESP*Ra2/delE2'],yerr = SPECSU['U2STD'], xerr=0,fmt='o',markersize = s,color ='green',label = 'Det2')
    plt.errorbar(SPECSU['AVG_E'],SPECSU['RESP*Ra3/delE3'],yerr = SPECSU['U3STD'], xerr=0,fmt='o',markersize = s,color = 'orange',label = 'Det3')
    #plt.axvspan(2,4,label = '2-4EV', color = 'purple', alpha =0.25)
    #plt.axvspan(4,8,label = '4-8EV', color = 'pink', alpha =0.5)
    #SPECSU.plot(x='AVG_E',y=['RESP*Ra/delE'] ,
     #           label = ['Summed Spectrum'], zorder=4)

    plt.xlim(2,7)
    #plt.ylim(-0.02,0.02)
    plt.title('Spectrum Python, Stokes Parameter U Observation ' + str(obs))
    plt.xlabel('Energy [KeV]')
    plt.ylabel('Rate [cst/s/KeV]')
    plt.legend()
    plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/SPECSUObs{obs}.jpeg')
    plt.show()
    plt.close()
    #mupltiply by equations 3a,3b to get u and q
    #ds9 for region files to link x and y to radius for backround subtraction


    plt.figure()
    s=4 #size of the scatter plot points
    plt.errorbar(SPECSQ['AVG_E'],SPECSQ['RESP*Ra1/delE1'],yerr = SPECSQ['Q1STD'], xerr=0,fmt='o',markersize = s,color = 'red',label = 'Det1')
    plt.errorbar(SPECSQ['AVG_E'],SPECSQ['RESP*Ra2/delE2'],yerr = SPECSQ['Q2STD'], xerr=0,fmt='o',markersize = s,color ='green',label = 'Det2')
    plt.errorbar(SPECSQ['AVG_E'],SPECSQ['RESP*Ra3/delE3'],yerr = SPECSQ['Q3STD'], xerr=0,fmt='o',markersize = s,color = 'orange',label = 'Det3')
    #SPECSQ.plot(x='AVG_E',y=['RESP*Ra/delE'] ,
               # label = ['Summed Spectrum'], zorder=4)
    plt.xlim(2,7)
    #plt.ylim(-0.015,0.015)
    #plt.axvspan(2,4,label = '2-4EV', color = 'purple', alpha =0.25)
    #plt.axvspan(4,8,label = '4-8EV', color = 'pink', alpha =0.5)
    plt.title('Spectrum Python, Stokes Parameter Q Observation ' + str(obs))
    plt.xlabel('Energy [KeV]')
    plt.ylabel('Rate [cst/s/KeV]')
    plt.legend()
    plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/SPECSQObs{obs}.jpeg')
    plt.show()
    plt.close()
SpectraPlots(SPECSI1,SPECSU1,SPECSQ1,1)
SpectraPlots(SPECSI2,SPECSU2,SPECSQ2,2)
SpectraPlots(SPECSI3,SPECSU3,SPECSQ3,3)

In [ ]:
def PhaseResolvedSpectraPlots(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,obs,P,*arg,**kwargs):
    numofcolumns = 3  
    
    pd.options.mode.chained_assignment = None
    fig, axs = plt.subplots(int(np.ceil(((3*len(P)+1))/numofcolumns)-2),numofcolumns,figsize=(10,20))
    plt.suptitle(f"I,Q,U Phase Resolved Spectra for Observation: {obs}",y=0.99)
    s=4
    for i in range(len(P)-1):
        SPECSI,SPECSU,SPECSQ = SpectraAnalysis(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,obs,Pmax = P[i+1],Pmin=P[i])
        #plt.figure()
        #plt.plot(SPECSI['E1_AVG'],SPECSI['RESP*Ra1/delE1'], label = 'Det1', color = 'red')
        #plt.show()
        
        #with pd.option_context('display.max_rows',None):
        #    print(SPECSU)
        
        axs[int(np.floor(i)),0].set_title('I Spectra Plot, Phase='+str(P[i])+' - '+str(P[i+1]))
        axs[int(np.floor(i)),0].plot(SPECSI['AVG_E'],SPECSI['RESP*Ra1/delE1'], label = 'Det1', color = 'red')
        axs[int(np.floor(i)),0].plot(SPECSI['AVG_E'],SPECSI['RESP*Ra2/delE2'], label = 'Det2', color = 'green')
        axs[int(np.floor(i)),0].plot(SPECSI['AVG_E'],SPECSI['RESP*Ra3/delE3'], label = 'Det3', color = 'orange')
        axs[int(np.floor(i)),0].set_ylim([0.0005,0.5])
        axs[int(np.floor(i)),0].set_xlim([2,7])
        axs[int(np.floor(i)),0].set_yscale('log')
        #axs[int(np.floor(i/numofcolumns)),i%numofcolumns].scatter(SPECSI['E1_AVG'],SPECSI['RESP*Ra1/delE1'], label = 'Det1', color = 'red')
        
        axs[int((np.floor(i))),1].set_title('Q SpectraPlot, Phase='+str(P[i])+' - '+str(P[i+1]))
        axs[int((np.floor(i))),1].errorbar(SPECSQ['AVG_E'],SPECSQ['RESP*Ra1/delE1'],yerr = SPECSQ['Q1STD'],label = 'Det1', color = 'red',fmt='o',markersize=s)#, label = 'Det1', color = 'red')
        axs[int((np.floor(i))),1].errorbar(SPECSQ['AVG_E'],SPECSQ['RESP*Ra2/delE2'],yerr = SPECSQ['Q2STD'],label = 'Det2', color = 'green',fmt='o',markersize=s)
        axs[int((np.floor(i))),1].errorbar(SPECSQ['AVG_E'],SPECSQ['RESP*Ra3/delE3'],yerr = SPECSQ['Q3STD'],label = 'Det3', color = 'orange',fmt='o',markersize=s)#
        axs[int((np.floor(i))),1].set_xlim([2,7])
        axs[int(np.floor(i)),1].set_ylim([-0.01,0.01])
        
    
        axs[int(np.floor(i)),2].set_title('U SpectraPlot, Phase='+str(P[i])+' - '+str(P[i+1]))
        axs[int(np.floor(i)),2].errorbar(SPECSU['AVG_E'],SPECSU['RESP*Ra1/delE1'],yerr = SPECSU['U1STD'], label = 'Det1', color = 'red',fmt='o',markersize=s)
        axs[int(np.floor(i)),2].errorbar(SPECSU['AVG_E'],SPECSU['RESP*Ra2/delE2'],yerr = SPECSU['U2STD'], label = 'Det2', color = 'green',fmt='o',markersize=s)
        axs[int(np.floor(i)),2].errorbar(SPECSU['AVG_E'],SPECSU['RESP*Ra3/delE3'],yerr = SPECSU['U3STD'], label = 'Det3', color = 'orange',fmt='o',markersize=s)
        axs[int(np.floor(i)),2].set_xlim([2,7])
        axs[int(np.floor(i)),2].set_ylim([-0.01,0.01])
    plt.tight_layout(pad = 1)
    plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/PhaseResolavedSpectraObs{obs}.jpeg')
    plt.show()
    pd.options.mode.chained_assignment = 'warn'
    
    
obs =1
P = [0,0.1,0.2,0.5,0.75,0.875,1]
PhaseResolvedSpectraPlots(Det1S,Det1B,Det2S,Det2B,Det3S,Det3B,obs,P)

for i in range(3):
    print("")

obs = 2
#P = [0,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.775,0.875,0.925,1]

PhaseResolvedSpectraPlots(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,obs,P)

        
        

In [ ]:






def PolarizationGraphs(SPECSI,SPECSU,SPECSQ,obs):   
    pd.options.mode.chained_assignment = None
    bincount = 9
    binsize = len(SPECSI)
    SPECS = SPECSI.groupby(np.arange(binsize)//bincount).mean()

    SPECSU['U_RESP*Ra/delE1'] =  SPECSU['RESP*Ra1/delE1']
    SPECSU['U_RESP*Ra/delE2'] =  SPECSU['RESP*Ra2/delE2']
    SPECSU['U_RESP*Ra/delE3'] =  SPECSU['RESP*Ra3/delE3']
    #SPECSU = SPECSU.drop(columns=['RESP*Ra1/delE1','RESP*Ra2/delE2','RESP*Ra3/delE3'])

    SPECS['I_RESP*Ra/delE1'] = SPECS['RESP*Ra1/delE1']
    SPECS['I_RESP*Ra/delE2'] = SPECS['RESP*Ra2/delE2']
    SPECS['I_RESP*Ra/delE3'] = SPECS['RESP*Ra3/delE3']
    #SPECS = SPECS.drop(columns=['RESP*Ra1/delE1','RESP*Ra2/delE2','RESP*Ra3/delE3'])

    SPECSQ['Q_RESP*Ra/delE1'] =  SPECSQ['RESP*Ra1/delE1']
    SPECSQ['Q_RESP*Ra/delE2'] =  SPECSQ['RESP*Ra2/delE2']
    SPECSQ['Q_RESP*Ra/delE3'] =  SPECSQ['RESP*Ra3/delE3']
    #SPECSQ = SPECSQ.drop(columns=['RESP*Ra1/delE1','RESP*Ra2/delE2','RESP*Ra3/delE3'])




    SPECS['E']= SPECSI['AVG_E']


    SPECS = pd.merge(SPECS,SPECSU,on='CH1')
    SPECSQ['CH1'] = SPECS['CH1']
    SPECS = pd.merge(SPECS,SPECSQ,on='CH1')

    SPECS['U_I1']= SPECS['U_RESP*Ra/delE1']/SPECS['I_RESP*Ra/delE1']
    SPECS['U_I2']= SPECS['U_RESP*Ra/delE2']/SPECS['I_RESP*Ra/delE2']
    SPECS['U_I3']= SPECS['U_RESP*Ra/delE3']/SPECS['I_RESP*Ra/delE3']

    SPECS['Q_I1']= SPECS['Q_RESP*Ra/delE1']/SPECS['I_RESP*Ra/delE1']
    SPECS['Q_I2']= SPECS['Q_RESP*Ra/delE2']/SPECS['I_RESP*Ra/delE2']
    SPECS['Q_I3']= SPECS['Q_RESP*Ra/delE3']/SPECS['I_RESP*Ra/delE3']

    SPECS['U1STD'] = SPECS['U1STD']/SPECS['I_RESP*Ra/delE1']
    SPECS['U2STD'] = SPECS['U2STD']/SPECS['I_RESP*Ra/delE2']
    SPECS['U3STD'] = SPECS['U3STD']/SPECS['I_RESP*Ra/delE3']
    SPECS['Q1STD'] = SPECS['Q1STD']/SPECS['I_RESP*Ra/delE1']
    SPECS['Q2STD'] = SPECS['Q2STD']/SPECS['I_RESP*Ra/delE2']
    SPECS['Q3STD'] = SPECS['Q3STD']/SPECS['I_RESP*Ra/delE3']



    SPECS['E'] = SPECS['AVG_E']

    SPECS = SPECS.drop(columns=['E1_AVG_x', 'E2_AVG_x', 'E3_AVG_x', 'delE1_x', 'Ra1_x', 
                                'RESP*Ra1_x', 'RESP*Ra1/delE1_x', 'CH2_x','delE2_x' ,'Ra2_x' ,'RESP*Ra2_x'
                                ,'RESP*Ra2/delE2_x' ,'CH3_x' ,'delE3_x' ,'Ra3_x' ,'RESP*Ra3_x' ,'RESP*Ra3/delE3_x',
                               'RESP*RATE_x' ,'AVG_E_x','RESP*Ra/delE_x',
                               'Ra3','RESP*Ra3' ,'RESP*Ra3/delE3' ,'RESP*RATE' ,'AVG_E' ,'RESP*Ra/delE',
                               'RESP*Ra2','RESP*Ra2/delE2', 'CH3', 'E1_AVG', 'E2_AVG', 'E3_AVG' ,'delE3',
                               'Q_RESP*Ra/delE1' ,'Q_RESP*Ra/delE2' ,'Q_RESP*Ra/delE3',
                               'E1_AVG_y', 'E2_AVG_y', 'E3_AVG_y', 'delE1_y', 'Ra1_y',
                               'I_RESP*Ra/delE1' ,'I_RESP*Ra/delE2' ,'I_RESP*Ra/delE3',
                               'RESP*Ra1_y' ,'RESP*Ra1/delE1_y' ,'CH2_y' ,'delE2_y' ,'Ra2_y' ,'RESP*Ra2_y' ,'RESP*Ra2/delE2_y', 'CH3_y'])
    SPECS = SPECS.reset_index()

    P = np.linspace(2,4,18)
    P = np.append(P, (5,7))

    for i in range(0,len(P)-1):


        S = SPECS.loc[SPECS['E'].between(P[i],P[i+1])]
        Sl = len(S)
        
        S['USTD1_dup'] = S['U1STD']
        S['USTD2_dup'] = S['U2STD']
        S['USTD3_dup'] = S['U3STD']
        S['QSTD1_dup'] = S['Q1STD']
        S['QSTD2_dup'] = S['Q2STD']
        S['QSTD3_dup'] = S['Q3STD']
        
        
        
        S['USTD1**2'] = S['U1STD']**2
        S['USTD2**2'] = S['U2STD']**2
        S['USTD3**2'] = S['U3STD']**2
        S['QSTD1**2'] = S['Q1STD']**2
        S['QSTD2**2'] = S['Q2STD']**2
        S['QSTD3**2'] = S['Q3STD']**2

        S['USTD1'] = np.sqrt(S['USTD1**2'].sum())
        S['USTD2'] = np.sqrt(S['USTD2**2'].sum())
        S['USTD3'] = np.sqrt(S['USTD3**2'].sum())
        S['QSTD1'] = np.sqrt(S['QSTD1**2'].sum())
        S['QSTD2'] = np.sqrt(S['QSTD2**2'].sum())
        S['QSTD3'] = np.sqrt(S['QSTD3**2'].sum())

        S.drop(columns = ['USTD1**2','USTD2**2','USTD3**2','QSTD1**2','QSTD2**2','QSTD3**2'])


        S['U_I1'] = S['U_I1'].sum()/(P[i+1]-P[i])
        S['U_I2'] = S['U_I2'].sum()/(P[i+1]-P[i])
        S['U_I3'] = S['U_I3'].sum()/(P[i+1]-P[i])

        S['Q_I1'] = S['Q_I1'].sum()/(P[i+1]-P[i])
        S['Q_I2'] = S['Q_I2'].sum()/(P[i+1]-P[i])
        S['Q_I3'] = S['Q_I3'].sum()/(P[i+1]-P[i])

        S['USTD1'] = S['USTD1']/(P[i+1]-P[i])
        S['USTD2'] = S['USTD2']/(P[i+1]-P[i])
        S['USTD3'] = S['USTD3']/(P[i+1]-P[i])
        S['QSTD1'] = S['QSTD1']/(P[i+1]-P[i])
        S['QSTD2'] = S['QSTD2']/(P[i+1]-P[i])
        S['QSTD3'] = S['QSTD3']/(P[i+1]-P[i])
        S['Eerror'] = (P[i+1]-P[i])/2


        S['USTD'] = np.sqrt((S['USTD1']**2)+(S['USTD2']**2)+(S['USTD3']**2))/3
        S['QSTD'] = np.sqrt((S['QSTD1']**2)+(S['QSTD2']**2)+(S['QSTD3']**2))/3


        idx = S['E'].sub((P[i]+P[i+1])/2).abs()
        S = S.drop(index= (S.iloc[(S.index != idx.index)]).index)
        S['E'] = (P[i+1]+P[i])/2
        
        S['PD1'] = np.sqrt((S['U_I1']**2)+(S['Q_I1']**2))*100
        S['PD2'] = np.sqrt((S['U_I2']**2)+(S['Q_I2']**2))*100
        S['PD3'] = np.sqrt((S['U_I3']**2)+(S['Q_I3']**2))*100
        
        S['PA1'] = (0.5)*np.arctan2((S['U_I1']),(S['Q_I1']))*180/np.pi
        S['PA2'] = (0.5)*np.arctan2((S['U_I2']),(S['Q_I2']))*180/np.pi
        S['PA3'] = (0.5)*np.arctan2((S['U_I3']),(S['Q_I3']))*180/np.pi
        
        
        S['U_I'] = (S['U_I1'] + S['U_I2']+S['U_I3'])/3
        S['Q_I'] = (S['Q_I1'] + S['Q_I2']+S['Q_I3'])/3
        S['PD'] = np.sqrt((S['U_I']**2)+(S['Q_I']**2))*100
        S['PA'] = (0.5)*np.arctan2((S['U_I']),(S['Q_I']))*180/np.pi


        #S['PDe'] = (np.sqrt(((S['USTD']*(S['U_I']**(2-1))*2)**2) + ((S['QSTD']*(S['Q_I']**(2-1))*2)**2))/(S['PD']/100)**2)*(1/2)*S['PD']
        S['PDe'] = np.sqrt((((S['U_I']*S['USTD'])**2)+((S['Q_I']*S['QSTD'])**2))
                           /((S['U_I']*S['U_I'])+(S['Q_I']*S['Q_I'])))*100
        S['PAe'] = np.sqrt((((S['Q_I']*S['USTD'])**2)+((S['U_I']*S['QSTD'])**2))
                           /(((S['U_I']*S['U_I'])+(S['Q_I']*S['Q_I']))**2))*0.5*180/np.pi
        #S['PAe'] = np.sqrt((((S['Q_I']*S['USTD'])**2)+
         #                  ((S['U_I']*S['QSTD'])**2))/
          #                 (((S['U_I']**2)+(S['Q_I']**2))**2))*0.5*180/np.pi

        SPECS = SPECS.loc[~(SPECS['E'].between(P[i],P[i+1]))]
        SPECS= pd.concat([SPECS,S])
    SPECS = SPECS.sort_values(by=['E'])


    plt.figure()
    s=4 #size of the scatter plot points
    plt.errorbar(SPECS['E'],SPECS['U_I1'],yerr = SPECS['USTD1'], xerr=SPECS['Eerror'],fmt='o',markersize = s,color = 'red',label = 'Det1')
    plt.errorbar(SPECS['E'],SPECS['U_I2'],yerr = SPECS['USTD2'], xerr=SPECS['Eerror'],fmt='o',markersize = s,color = 'green',label = 'Det2')
    plt.errorbar(SPECS['E'],SPECS['U_I3'],yerr = SPECS['USTD3'], xerr=SPECS['Eerror'],fmt='o',markersize = s,color = 'orange',label = 'Det3')
    #plt.errorbar(SPECSQ['E'],SPECSQ['RESP*Ra2/delE2'],yerr = SPECSUstd, xerr=0,fmt='o',markersize = s,color ='green',label = 'Det2')
    #plt.errorbar(SPECSQ['E'],SPECSQ['RESP*Ra3/delE3'],yerr = SPECSUstd, xerr=0,fmt='o',markersize = s,color = 'orange',label = 'Det3')

    plt.xlim(2,7)
    #plt.ylim(-0.010,0.010)
    #plt.axvspan(2,4,label = '2-4EV', color = 'purple', alpha =0.25)
    #plt.axvspan(4,8,label = '4-8EV', color = 'pink', alpha =0.5)
    plt.title('Normalized Stokes Parameter U/I Observation ' + str(obs))
    plt.xlabel('Energy [KeV]')
    plt.ylabel('U/I')
    plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/U_IObs{obs}.jpeg')
    plt.legend()
    plt.show()
    plt.close()

    plt.figure()
    s=4 #size of the scatter plot points
    plt.errorbar(SPECS['E'],SPECS['Q_I1'],yerr = SPECS['QSTD1'], xerr=SPECS['Eerror'],fmt='o',markersize = s,color = 'red',label = 'Det1')
    plt.errorbar(SPECS['E'],SPECS['Q_I2'],yerr = SPECS['QSTD2'], xerr=SPECS['Eerror'],fmt='o',markersize = s,color = 'green',label = 'Det2')
    plt.errorbar(SPECS['E'],SPECS['Q_I3'],yerr = SPECS['QSTD3'], xerr=SPECS['Eerror'],fmt='o',markersize = s,color = 'orange',label = 'Det3')
    #plt.errorbar(SPECSQ['E'],SPECSQ['RESP*Ra2/delE2'],yerr = SPECSUstd, xerr=0,fmt='o',markersize = s,color ='green',label = 'Det2')
    #plt.errorbar(SPECSQ['E'],SPECSQ['RESP*Ra3/delE3'],yerr = SPECSUstd, xerr=0,fmt='o',markersize = s,color = 'orange',label = 'Det3')

    plt.xlim(2,7)
    #plt.ylim(-0.05,0.05)
    #plt.axvspan(2,4,label = '2-4EV', color = 'purple', alpha =0.25)
    #plt.axvspan(4,8,label = '4-8EV', color = 'pink', alpha =0.5)
    plt.title('Normalized Stokes Parameter Q/I Observation ' + str(obs))
    plt.xlabel('Energy [KeV]')
    plt.ylabel('Q/I')
    plt.legend()
    plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/Q_IObs{obs}.jpeg')
    plt.show()
    plt.close()





    plt.figure()
    s=4 
    plt.errorbar(SPECS['E'],SPECS['PD1'],yerr = SPECS['PDe'], xerr=SPECS['Eerror'],fmt='o',markersize = s,color = 'red',label = 'Det1')
    plt.errorbar(SPECS['E'],SPECS['PD2'],yerr = SPECS['PDe'], xerr=SPECS['Eerror'],fmt='o',markersize = s,color = 'green',label = 'Det2')
    plt.errorbar(SPECS['E'],SPECS['PD3'],yerr = SPECS['PDe'], xerr=SPECS['Eerror'],fmt='o',markersize = s,color = 'orange',label = 'Det3')
    plt.xlim(2,7)
    #plt.ylim(-0.10,0.10)
    plt.title('Average Polarization Degree Observation ' +str(obs))
    plt.xlabel('Energy [KeV]')
    plt.ylabel('[%]')
    plt.legend()
    plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/PDObs{obs}.jpeg')
    plt.show()
    plt.close()

    plt.figure()
    s=4 
    plt.errorbar(SPECS['E'],SPECS['PA1'],yerr = SPECS['PAe'], xerr=SPECS['Eerror'],fmt='o',markersize = s,color = 'red',label = 'Det1')
    plt.errorbar(SPECS['E'],SPECS['PA2'],yerr = SPECS['PAe'], xerr=SPECS['Eerror'],fmt='o',markersize = s,color = 'green',label = 'Det2')
    plt.errorbar(SPECS['E'],SPECS['PA3'],yerr = SPECS['PAe'], xerr=SPECS['Eerror'],fmt='o',markersize = s,color = 'orange',label = 'Det3')
    plt.xlim(2,7)
    #plt.ylim(-0.10,0.10)
    plt.title('Average Polarization Angle Observation ' + str(obs))
    plt.xlabel('Energy [KeV]')
    plt.ylabel('[Degrees]')
    plt.legend()
    plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/PAObs{obs}.jpeg')
    plt.show()
    plt.close()
    pd.options.mode.chained_assignment = 'warn'
    return SPECSI,SPECSU,SPECSQ,SPECS



SPECSI1,SPECSU2,SPECSQ1,SPECS1 = PolarizationGraphs(SPECSI1,SPECSU1,SPECSQ1,1)
SPECSI2,SPECSU2,SPECSQ2,SPECS2 = PolarizationGraphs(SPECSI2,SPECSU2,SPECSQ2,2)
SPECSI3,SPECSU3,SPECSQ3,SPECS3 = PolarizationGraphs(SPECSI3,SPECSU3,SPECSQ3,3)

s=4
plt.figure()
plt.errorbar(SPECS1['E'],SPECS1['PA'],yerr = SPECS1['PAe'], xerr=SPECS1['Eerror'],fmt='o',markersize = s,color = 'red',label = 'obs1')
plt.errorbar(SPECS2['E'],SPECS2['PA'],yerr = SPECS2['PAe'], xerr=SPECS2['Eerror'],fmt='o',markersize = s,color = 'green',label = 'obs2')
plt.errorbar(SPECS3['E'],SPECS3['PA'],yerr = SPECS3['PAe'], xerr=SPECS3['Eerror'],fmt='o',markersize = s,color = 'orange',label = 'obs3')
plt.title('Polarization Angle All observations')
plt.xlabel('Energy [KeV]')
plt.ylabel('[Degrees]')
plt.legend()
plt.show()
plt.close()


plt.figure()
plt.errorbar(SPECS1['E'],SPECS1['PD'],yerr = SPECS1['PDe'], xerr=SPECS1['Eerror'],fmt='o',markersize = s,color = 'red',label = 'obs1')
plt.errorbar(SPECS2['E'],SPECS2['PD'],yerr = SPECS2['PDe'], xerr=SPECS2['Eerror'],fmt='o',markersize = s,color = 'green',label = 'obs2')
plt.errorbar(SPECS3['E'],SPECS3['PD'],yerr = SPECS3['PDe'], xerr=SPECS3['Eerror'],fmt='o',markersize = s,color = 'orange',label = 'obs3')
plt.title('Polarization Angle All observations')
plt.xlabel('Energy [KeV]')
plt.ylabel('[%]')
plt.legend()
plt.show()
plt.close()

In [ ]:
def PAPDPhaseResolvedSpectraPlots(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,obs,P,*arg,**kwargs):
    numofcolumns = 5  
    
    pd.options.mode.chained_assignment = None
    fig, axs = plt.subplots(int(np.ceil(((3*len(P)+1))/numofcolumns)+1),numofcolumns,figsize=(10,20))
    plt.suptitle(f"I,Q,U Phase Resolved Spectra for Observation: {obs}",y=0.99)
    s=4
    for i in range(len(P)-1):
        SPECSI,SPECSU,SPECSQ = SpectraAnalysis(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,obs,Pmax = P[i+1],Pmin=P[i])
        
        
        pd.options.mode.chained_assignment = None
        bincount = 9
        binsize = len(SPECSI)
        SPECS = SPECSI.groupby(np.arange(binsize)//bincount).mean()

        SPECSU['U_RESP*Ra/delE1'] =  SPECSU['RESP*Ra1/delE1']
        SPECSU['U_RESP*Ra/delE2'] =  SPECSU['RESP*Ra2/delE2']
        SPECSU['U_RESP*Ra/delE3'] =  SPECSU['RESP*Ra3/delE3']
        #SPECSU = SPECSU.drop(columns=['RESP*Ra1/delE1','RESP*Ra2/delE2','RESP*Ra3/delE3'])

        SPECS['I_RESP*Ra/delE1'] = SPECS['RESP*Ra1/delE1']
        SPECS['I_RESP*Ra/delE2'] = SPECS['RESP*Ra2/delE2']
        SPECS['I_RESP*Ra/delE3'] = SPECS['RESP*Ra3/delE3']
        #SPECS = SPECS.drop(columns=['RESP*Ra1/delE1','RESP*Ra2/delE2','RESP*Ra3/delE3'])

        SPECSQ['Q_RESP*Ra/delE1'] =  SPECSQ['RESP*Ra1/delE1']
        SPECSQ['Q_RESP*Ra/delE2'] =  SPECSQ['RESP*Ra2/delE2']
        SPECSQ['Q_RESP*Ra/delE3'] =  SPECSQ['RESP*Ra3/delE3']
        #SPECSQ = SPECSQ.drop(columns=['RESP*Ra1/delE1','RESP*Ra2/delE2','RESP*Ra3/delE3'])




        SPECS['E']= SPECSI['AVG_E']


        SPECS = pd.merge(SPECS,SPECSU,on='CH1')
        SPECSQ['CH1'] = SPECS['CH1']
        SPECS = pd.merge(SPECS,SPECSQ,on='CH1')

        SPECS['U_I1']= SPECS['U_RESP*Ra/delE1']/SPECS['I_RESP*Ra/delE1']
        SPECS['U_I2']= SPECS['U_RESP*Ra/delE2']/SPECS['I_RESP*Ra/delE2']
        SPECS['U_I3']= SPECS['U_RESP*Ra/delE3']/SPECS['I_RESP*Ra/delE3']

        SPECS['Q_I1']= SPECS['Q_RESP*Ra/delE1']/SPECS['I_RESP*Ra/delE1']
        SPECS['Q_I2']= SPECS['Q_RESP*Ra/delE2']/SPECS['I_RESP*Ra/delE2']
        SPECS['Q_I3']= SPECS['Q_RESP*Ra/delE3']/SPECS['I_RESP*Ra/delE3']

        SPECS['U1STD'] = SPECS['U1STD']/SPECS['I_RESP*Ra/delE1']
        SPECS['U2STD'] = SPECS['U2STD']/SPECS['I_RESP*Ra/delE2']
        SPECS['U3STD'] = SPECS['U3STD']/SPECS['I_RESP*Ra/delE3']
        SPECS['Q1STD'] = SPECS['Q1STD']/SPECS['I_RESP*Ra/delE1']
        SPECS['Q2STD'] = SPECS['Q2STD']/SPECS['I_RESP*Ra/delE2']
        SPECS['Q3STD'] = SPECS['Q3STD']/SPECS['I_RESP*Ra/delE3']



        SPECS['E'] = SPECS['AVG_E']

        SPECS = SPECS.drop(columns=['E1_AVG_x', 'E2_AVG_x', 'E3_AVG_x', 'delE1_x', 'Ra1_x', 
                                    'RESP*Ra1_x', 'RESP*Ra1/delE1_x', 'CH2_x','delE2_x' ,'Ra2_x' ,'RESP*Ra2_x'
                                    ,'RESP*Ra2/delE2_x' ,'CH3_x' ,'delE3_x' ,'Ra3_x' ,'RESP*Ra3_x' ,'RESP*Ra3/delE3_x',
                                   'RESP*RATE_x' ,'AVG_E_x','RESP*Ra/delE_x',
                                   'Ra3','RESP*Ra3' ,'RESP*Ra3/delE3' ,'RESP*RATE' ,'RESP*Ra/delE',
                                   'RESP*Ra2','RESP*Ra2/delE2', 'CH3', 'E1_AVG', 'E2_AVG', 'E3_AVG' ,'delE3',
                                   'E1_AVG_y', 'E2_AVG_y', 'E3_AVG_y', 'delE1_y', 'Ra1_y',
                                   'RESP*Ra1_y' ,'RESP*Ra1/delE1_y' ,'CH2_y' ,'delE2_y' ,'Ra2_y' ,'RESP*Ra2_y' ,'RESP*Ra2/delE2_y', 'CH3_y'])
        SPECS = SPECS.reset_index()

        E = np.linspace(2,4,18)
        E = np.append(E, (5,7))

        for j in range(0,len(E)-1):


            S = SPECS.loc[SPECS['E'].between(E[j],E[j+1])]
            Sl = len(S)
            
            S['USTD1_dup'] = S['U1STD']
            S['USTD2_dup'] = S['U2STD']
            S['USTD3_dup'] = S['U3STD']
            S['QSTD1_dup'] = S['Q1STD']
            S['QSTD2_dup'] = S['Q2STD']
            S['QSTD3_dup'] = S['Q3STD']
            
            S['USTD1**2'] = S['U1STD']**2
            S['USTD2**2'] = S['U2STD']**2
            S['USTD3**2'] = S['U3STD']**2
            S['QSTD1**2'] = S['Q1STD']**2
            S['QSTD2**2'] = S['Q2STD']**2
            S['QSTD3**2'] = S['Q3STD']**2

            S['USTD1'] = np.sqrt(S['USTD1**2'].sum())
            S['USTD2'] = np.sqrt(S['USTD2**2'].sum())
            S['USTD3'] = np.sqrt(S['USTD3**2'].sum())
            S['QSTD1'] = np.sqrt(S['QSTD1**2'].sum())
            S['QSTD2'] = np.sqrt(S['QSTD2**2'].sum())
            S['QSTD3'] = np.sqrt(S['QSTD3**2'].sum())

            S.drop(columns = ['USTD1**2','USTD2**2','USTD3**2','QSTD1**2','QSTD2**2','QSTD3**2'])


            S['U_I1'] = S['U_I1'].sum()/(E[j+1]-E[j])
            S['U_I2'] = S['U_I2'].sum()/(E[j+1]-E[j])
            S['U_I3'] = S['U_I3'].sum()/(E[j+1]-E[j])

            S['Q_I1'] = S['Q_I1'].sum()/(E[j+1]-E[j])
            S['Q_I2'] = S['Q_I2'].sum()/(E[j+1]-E[j])
            S['Q_I3'] = S['Q_I3'].sum()/(E[j+1]-E[j])

            S['USTD1'] = S['USTD1']/(E[j+1]-E[j])
            S['USTD2'] = S['USTD2']/(E[j+1]-E[j])
            S['USTD3'] = S['USTD3']/(E[j+1]-E[j])
            S['QSTD1'] = S['QSTD1']/(E[j+1]-E[j])
            S['QSTD2'] = S['QSTD2']/(E[j+1]-E[j])
            S['QSTD3'] = S['QSTD3']/(E[j+1]-E[j])
            S['Eerror'] = (E[j+1]-E[j])/2


            S['USTD'] = np.sqrt((S['USTD1']**2)+(S['USTD2']**2)+(S['USTD3']**2))/3
            S['QSTD'] = np.sqrt((S['QSTD1']**2)+(S['QSTD2']**2)+(S['QSTD3']**2))/3


            idx = S['E'].sub((E[j]+E[j+1])/2).abs()
            S = S.drop(index= (S.iloc[(S.index != idx.index)]).index)
            S['E'] = (E[j+1]+E[j])/2

            S['PD1'] = np.sqrt((S['U_I1']**2)+(S['Q_I1']**2))*100
            S['PD2'] = np.sqrt((S['U_I2']**2)+(S['Q_I2']**2))*100
            S['PD3'] = np.sqrt((S['U_I3']**2)+(S['Q_I3']**2))*100

            S['PA1'] = (0.5)*np.arctan2((S['U_I1']),(S['Q_I1']))*180/np.pi
            S['PA2'] = (0.5)*np.arctan2((S['U_I2']),(S['Q_I2']))*180/np.pi
            S['PA3'] = (0.5)*np.arctan2((S['U_I3']),(S['Q_I3']))*180/np.pi


            S['U_I'] = (S['U_I1'] + S['U_I2']+S['U_I3'])/3
            S['Q_I'] = (S['Q_I1'] + S['Q_I2']+S['Q_I3'])/3
            S['PD'] = np.sqrt((S['U_I']**2)+(S['Q_I']**2))*100
            S['PA'] = (0.5)*np.arctan2((S['U_I']),(S['Q_I']))*180/np.pi


            #S['PDe'] = (np.sqrt(((S['USTD']*(S['U_I']**(2-1))*2)**2) + ((S['QSTD']*(S['Q_I']**(2-1))*2)**2))/(S['PD']/100)**2)*(1/2)*S['PD']
            S['PDe'] = np.sqrt((((S['U_I']*S['USTD'])**2)+((S['Q_I']*S['QSTD'])**2))
                               /((S['U_I']*S['U_I'])+(S['Q_I']*S['Q_I'])))*100
            S['PAe'] = np.sqrt((((S['Q_I']*S['USTD'])**2)+((S['U_I']*S['QSTD'])**2))
                               /(((S['U_I']*S['U_I'])+(S['Q_I']*S['Q_I']))**2))*0.5*180/np.pi
            #S['PAe'] = np.sqrt((((S['Q_I']*S['USTD'])**2)+
             #                  ((S['U_I']*S['QSTD'])**2))/
              #                 (((S['U_I']**2)+(S['Q_I']**2))**2))*0.5*180/np.pi

            SPECS = SPECS.loc[~(SPECS['E'].between(E[j],E[j+1]))]
            SPECS= pd.concat([SPECS,S])
        SPECS = SPECS.sort_values(by=['E'])

        ##Currecnt Problem##
        #Error bar definitions conflict between the definition
        
        #plt.figure()
        #plt.plot(SPECSI['E1_AVG'],SPECSI['RESP*Ra1/delE1'], label = 'Det1', color = 'red')
        #plt.show()
        
        #with pd.option_context('display.max_rows',None):
        #    print(SPECSU)
        
        axs[int(np.floor(i)),0].set_title('I Spectra Plot, Phase='+str(P[i])+' - '+str(P[i+1]))
        axs[int(np.floor(i)),0].plot(SPECS['AVG_E'],SPECS['I_RESP*Ra/delE1'], label = 'Det1', color = 'red')
        axs[int(np.floor(i)),0].plot(SPECS['AVG_E'],SPECS['I_RESP*Ra/delE2'], label = 'Det2', color = 'green')
        axs[int(np.floor(i)),0].plot(SPECS['AVG_E'],SPECS['I_RESP*Ra/delE3'], label = 'Det3', color = 'orange')
        axs[int(np.floor(i)),0].set_ylim([0.0005,0.5])
        axs[int(np.floor(i)),0].set_xlim([2,7])
        axs[int(np.floor(i)),0].set_yscale('log')
        #axs[int(np.floor(i/numofcolumns)),i%numofcolumns].scatter(SPECSI['E1_AVG'],SPECSI['RESP*Ra1/delE1'], label = 'Det1', color = 'red')
        
        axs[int((np.floor(i))),1].set_title('Q SpectraPlot, Phase='+str(P[i])+' - '+str(P[i+1]))
        axs[int((np.floor(i))),1].errorbar(SPECS['AVG_E'],SPECS['Q_RESP*Ra/delE1'],yerr = SPECS['QSTD1_dup'],label = 'Det1', color = 'red',fmt='o',markersize=s)#, label = 'Det1', color = 'red')
        axs[int((np.floor(i))),1].errorbar(SPECS['AVG_E'],SPECS['Q_RESP*Ra/delE2'],yerr = SPECS['QSTD2_dup'],label = 'Det2', color = 'green',fmt='o',markersize=s)
        axs[int((np.floor(i))),1].errorbar(SPECS['AVG_E'],SPECS['Q_RESP*Ra/delE3'],yerr = SPECS['QSTD3_dup'],label = 'Det3', color = 'orange',fmt='o',markersize=s)#
        axs[int((np.floor(i))),1].set_xlim([2,7])
        axs[int(np.floor(i)),1].set_ylim([-0.01,0.01])
        
    
        axs[int(np.floor(i)),2].set_title('U SpectraPlot, Phase='+str(P[i])+' - '+str(P[i+1]))
        axs[int(np.floor(i)),2].errorbar(SPECS['AVG_E'],SPECS['U_RESP*Ra/delE1'],yerr = SPECS['USTD1_dup'], label = 'Det1', color = 'red',fmt='o',markersize=s)
        axs[int(np.floor(i)),2].errorbar(SPECS['AVG_E'],SPECS['U_RESP*Ra/delE2'],yerr = SPECS['USTD2_dup'], label = 'Det2', color = 'green',fmt='o',markersize=s)
        axs[int(np.floor(i)),2].errorbar(SPECS['AVG_E'],SPECS['U_RESP*Ra/delE3'],yerr = SPECS['USTD3_dup'], label = 'Det3', color = 'orange',fmt='o',markersize=s)
        axs[int(np.floor(i)),2].set_xlim([2,7])
        axs[int(np.floor(i)),2].set_ylim([-0.01,0.01])
        
        
        axs[int((np.floor(i))),3].set_title('Q/I Plot, Phase='+str(P[i])+' - '+str(P[i+1]))
        axs[int((np.floor(i))),3].errorbar(SPECS['E'],SPECS['Q_I1'],yerr = SPECS['QSTD1'],label = 'Det1', color = 'red',fmt='o',markersize=s)#, label = 'Det1', color = 'red')
        axs[int((np.floor(i))),3].errorbar(SPECS['E'],SPECS['Q_I2'],yerr = SPECS['QSTD2'],label = 'Det2', color = 'green',fmt='o',markersize=s)
        axs[int((np.floor(i))),3].errorbar(SPECS['E'],SPECS['Q_I3'],yerr = SPECS['QSTD3'],label = 'Det3', color = 'orange',fmt='o',markersize=s)#
        axs[int((np.floor(i))),3].set_xlim([2,7])
        axs[int(np.floor(i)),3].set_ylim([-0.2,0.2])
        
        axs[int((np.floor(i))),4].set_title('U/I Plot, Phase='+str(P[i])+' - '+str(P[i+1]))
        axs[int((np.floor(i))),4].errorbar(SPECS['E'],SPECS['U_I1'],yerr = SPECS['USTD1'],label = 'Det1', color = 'red',fmt='o',markersize=s)#, label = 'Det1', color = 'red')
        axs[int((np.floor(i))),4].errorbar(SPECS['E'],SPECS['U_I2'],yerr = SPECS['USTD2'],label = 'Det2', color = 'green',fmt='o',markersize=s)
        axs[int((np.floor(i))),4].errorbar(SPECS['E'],SPECS['U_I3'],yerr = SPECS['USTD3'],label = 'Det3', color = 'orange',fmt='o',markersize=s)#
        axs[int((np.floor(i))),4].set_xlim([2,7])
        axs[int(np.floor(i)),4].set_ylim([-0.2,0.2])
        
    plt.tight_layout(pad = 1)
    plt.savefig(f'/home/jacob/Desktop/PulseProfiles/Obs{obs}/PAPDPhaseResolavedQUPlotsObs{obs}.jpeg')
    plt.show()
    pd.options.mode.chained_assignment = 'warn'
    
    
obs =1
P = [0,0.1,0.2,0.5,0.75,0.875,1]
PAPDPhaseResolvedSpectraPlots(Det1S,Det1B,Det2S,Det2B,Det3S,Det3B,obs,P)

for i in range(3):
    print("")

obs = 2
#P = [0,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.775,0.875,0.925,1]

PAPDPhaseResolvedSpectraPlots(Det12S,Det12B,Det22S,Det22B,Det32S,Det32B,obs,P)

        

In [ ]:
pd.options.display.max_rows = 30
pd.options.mode.chained_assignment = None

In [ ]:
ChR1S = CHCount(Det1S,1,1) 
ChR1bkg = CHCount(Det1B,1,1)

ChR2S = CHCount(Det2S,2,1) 
ChR2bkg = CHCount(Det2B,2,1)

ChR3S = CHCount(Det3S,3,1) 
ChR3bkg = CHCount(Det3B,3,1)


In [ ]:
CH1S = ChR1S[:,0]
Ra1IS = ChR1S[:,2]
CH2S = ChR2S[:,0]
Ra2IS = ChR2S[:,2]
CH3S = ChR3S[:,0]
Ra3IS = ChR3S[:,2]

SPECSI = ARFRMF(CH1S,Ra1IS,CH2S,Ra2IS,CH3S,Ra3IS)



CH1 = ChR1bkg[:,0]
Ra1I = ChR1bkg[:,2]
CH2 = ChR2bkg[:,0]
Ra2I = ChR2bkg[:,2]
CH3 = ChR3bkg[:,0]
Ra3I = ChR3bkg[:,2]

SPECSbkg = ARFRMF(CH1,Ra1I,CH2,Ra2I,CH3,Ra3I)

#SPECSI = SPECBackgroundSubtraction(SPECSI,SPECSbkg,26.923,53.846,107.693)





#ChR[:,3] is the u, non-normalized stokes parameter = 2sin(2*psi)
Ra1U = ChR1S[:,2]*ChR1S[:,3]/ChR1S[:,5]
Ra2U = ChR2S[:,2]*ChR2S[:,3]/ChR2S[:,5]
Ra3U = ChR3S[:,2]*ChR3S[:,3]/ChR3S[:,5]

SPECSU = ARFRMF(CH1S,Ra1U,CH2S,Ra2U,CH3S,Ra3U)

Ra1U = ChR1bkg[:,2]*ChR1bkg[:,3]
Ra2U = ChR2bkg[:,2]*ChR2bkg[:,3]
Ra3U = ChR3bkg[:,2]*ChR3bkg[:,3]

SPECSUbkg = ARFRMF(CH1,Ra1U,CH2,Ra2U,CH3,Ra3U)

#SPECSU = SPECBackgroundSubtraction(SPECSU,SPECSUbkg,26.923,53.846,107.693)

SPECSU = SPECSU.loc[(SPECSU['AVG_E']>2) & (SPECSU['AVG_E']<7)]
bincount = 4
binsize = len(SPECSU)
SPECSU = SPECSU.groupby(np.arange(binsize)//bincount).mean()
SPECSUstd = SPECSU['RESP*Ra/delE'].std()


    
    



#ChR[:,4] is the q, non-normalized stokes parameter = 2cos(2*psi)
Ra1Q = ChR1S[:,2]*ChR1S[:,4]/ChR1S[:,5]
Ra2Q = ChR2S[:,2]*ChR2S[:,4]/ChR2S[:,5]
Ra3Q = ChR3S[:,2]*ChR3S[:,4]/ChR3S[:,5]

SPECSQ = ARFRMF(CH1S,Ra1Q,CH2S,Ra2Q,CH3S,Ra3Q)

Ra1Q = ChR1bkg[:,2]*ChR1bkg[:,4]
Ra2Q = ChR2bkg[:,2]*ChR2bkg[:,4]
Ra3Q = ChR3bkg[:,2]*ChR3bkg[:,4]

SPECSQbkg = ARFRMF(CH1,Ra1Q,CH2,Ra2Q,CH3,Ra3Q)

#SPECSQ = SPECBackgroundSubtraction(SPECSQ,SPECSQbkg,26.923,53.846,107.693)
SPECSQ= SPECSQ.loc[(SPECSQ['AVG_E']>2) & SPECSQ['AVG_E']<7]

bincount = 9
binsize = len(SPECSQ)

SPECSQ = SPECSQ.groupby(np.arange(binsize)//bincount).mean()

SPECSQstd = SPECSQ['RESP*Ra/delE'].std()


In [ ]:
plt.figure()
#plt.plot(SPECSI['AVG_E'],SPECSI['RESP*Ra/delE'], label = 'Summed Spectrum', zorder=4)
#SPECSI.plot(x='AVG_E',y=['RESP*Ra1/delE1',
 #                   'RESP*Ra2/delE2','RESP*Ra3/delE3'] ,
  #          label = ['Det1', 'Det2','Det3'], zorder=4)
plt.plot(SPECSI['E1_AVG'],SPECSI['RESP*Ra1/delE1'], label = 'Det1', color = 'red')
plt.plot(SPECSI['E2_AVG'],SPECSI['RESP*Ra2/delE2'], label = 'Det2', color = 'green')
plt.plot(SPECSI['E3_AVG'],SPECSI['RESP*Ra3/delE3'], label = 'Det3', color = 'orange')
plt.xlim(2,7)
#plt.ylim(0.01,0.3)
plt.yscale('log')
plt.axvspan(2,4,label = '2-4EV', color = 'purple', alpha =0.25)
plt.axvspan(4,7,label = '4-8EV', color = 'pink', alpha =0.5)
plt.title('Spectrum Python, Stokes Parameter I Observation 1')
plt.xlabel('Energy [KeV]')
plt.ylabel('Rate [cst/s/KeV]')
plt.text(3.4,0.01, '2-4KeV', verticalalignment = 'bottom', horizontalalignment='left')
plt.text(4.1,0.01, '4-7KeV', verticalalignment = 'bottom', horizontalalignment='left')
plt.tick_params('both',labelright = True,right=True,direction ='in',which = 'minor')
plt.legend()
plt.show()

In [ ]:
plt.figure()
s=4 #size of the scatter plot points
#plt.plot(SPECSU['AVG_E'],SPECSU['RESP*Ra/delE'], label = 'Summed Spectrum')
#plt.errorbar(SPECSU['AVG_E'],SPECSU['RESP*Ra/delE'],yerr = SPECSUstd, xerr=0, fmt='o',markersize = s+2,color = 'teal',label = 'Summed Spectrum')
plt.errorbar(SPECSU['AVG_E'],SPECSU['RESP*Ra1/delE1'],yerr = SPECSUstd, xerr=0,fmt='o',markersize = s,color = 'red',label = 'Det1')
plt.errorbar(SPECSU['AVG_E'],SPECSU['RESP*Ra2/delE2'],yerr = SPECSUstd, xerr=0,fmt='o',markersize = s,color ='green',label = 'Det2')
plt.errorbar(SPECSU['AVG_E'],SPECSU['RESP*Ra3/delE3'],yerr = SPECSUstd, xerr=0,fmt='o',markersize = s,color = 'orange',label = 'Det3')
#plt.axvspan(2,4,label = '2-4EV', color = 'purple', alpha =0.25)
#plt.axvspan(4,8,label = '4-8EV', color = 'pink', alpha =0.5)
#SPECSU.plot(x='AVG_E',y=['RESP*Ra/delE'] ,
 #           label = ['Summed Spectrum'], zorder=4)

#plt.xlim(2,7)
#plt.ylim(-0.02,0.02)
plt.title('Spectrum Python, Stokes Parameter U Observation 1')
plt.xlabel('Energy [KeV]')
plt.ylabel('Rate [cst/s/KeV]')
plt.legend()
plt.show()

#mupltiply by equations 3a,3b to get u and q
#ds9 for region files to link x and y to radius for backround subtraction

print(len(SPECSU))

In [ ]:
plt.figure()
s=4 #size of the scatter plot points
#plt.plot(SPECSQ['AVG_E'],SPECSQ['RESP*Ra/delE'], label = 'Summed Spectrum', zorder=4)
#plt.errorbar(SPECSQ['AVG_E'],SPECSQ['RESP*Ra/delE'],yerr = SPECSUstd, xerr=0, fmt='o',markersize = s+2,color = 'teal',label = 'Summed Spectrum')
plt.errorbar(SPECSQ['AVG_E'],SPECSQ['RESP*Ra1/delE1'],yerr = SPECSUstd, xerr=0,fmt='o',markersize = s,color = 'red',label = 'Det1')
plt.errorbar(SPECSQ['AVG_E'],SPECSQ['RESP*Ra2/delE2'],yerr = SPECSUstd, xerr=0,fmt='o',markersize = s,color ='green',label = 'Det2')
plt.errorbar(SPECSQ['AVG_E'],SPECSQ['RESP*Ra3/delE3'],yerr = SPECSUstd, xerr=0,fmt='o',markersize = s,color = 'orange',label = 'Det3')
#SPECSQ.plot(x='AVG_E',y=['RESP*Ra/delE'] ,
           # label = ['Summed Spectrum'], zorder=4)
plt.xlim(2,7)
#plt.ylim(-0.010,0.010)
#plt.axvspan(2,4,label = '2-4EV', color = 'purple', alpha =0.25)
#plt.axvspan(4,8,label = '4-8EV', color = 'pink', alpha =0.5)
plt.title('Spectrum Python, Stokes Parameter Q Observation 1')
plt.xlabel('Energy [KeV]')
plt.ylabel('Rate [cst/s/KeV]')
plt.legend()
plt.show()

In [ ]:
pd.options.mode.chained_assignment = 'warn'
#This code takes a while so we play a audio sound at the end of running it.

sr = 2000
T = 0.20
t = np.linspace(0,T,int(T*sr),endpoint=False)
ipd.Audio(np.sin(2*np.pi*300*t), rate = sr,autoplay = True)